# Работа с метриками по WB

### Импортруемые библиотеки

In [1]:
from tqdm import tqdm
import pandas as pd
import re
pd.options.mode.chained_assignment = None
import datetime
import numpy as np
import pyjokes
import xlrd
from operator import itemgetter
import glob
import requests
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from sys import exit
import sys
link_to_table = 'https://docs.google.com/spreadsheets/d/1A1TyZPeq9r2HiDi-CHfmTfRpuAQYToib1vqyTx4ZyRE/edit#gid=0'
df = pd.read_excel('/'.join(link_to_table.split('/')[:-1])+'/export?format=xlsx')
PATH, df = [dict(df.loc[x]) for x in list(df.index)], None
i = 0
while len([py for py in glob.glob(f"{PATH[i]['path_downloads']}*.xlsx")]) == 0:
    i +=1
path_downloads = PATH[i]['path_downloads']
path_algoritm = PATH[i]['path_algoritm']
sys.path.append(f'{path_algoritm}Модули')
import constants_for_marketplace_metrics_dags as const
import working_with_marketplace_metrics_dags as wwm
import maslow_working_module as mwm

import wb_markups

In [2]:
for i in range(1, 20):
    locals()[f"path_WB_new_metric{i}"] = None

#### Остатки по складам, фильтр "Невинномысск", "Екатеринбург 2" и "Новосибирск"

In [3]:
## Аналитика - отчеты - Отчёт с перечнем номенклатур
path_stocks_balances = path_downloads + f"report_{mwm.dt_now.date().strftime('%Y_%-m_%-d')}.xlsx"

#### Логистика и индекс локализации

In [4]:
## Тарифы - выгрузть в ексель 
path_logistic = path_downloads + f"warehouse coefficients {mwm.dt_now.date().strftime('%Y-%m-%d')}.xlsx"

#### Стоимость базовая для логистики и хранения: (x рублей за 5 л. + y рублей за каждый дополнительный литр) * коэффициент склада; Монопаллета: (x руб/день) * коэффициент паллетного хранения склада.

In [5]:
## Тарифы - Комиссия и тарифы на услуги
Log_b = [30, 7]  # Стоимость логистики до клиента

Hr_b_korob = [0.16, 0.12] # Стоимость хранения, короб
Hr_b_monop = 32 # Стоимость хранения, монопалета

quantity_liters_for_logistic = 1 # количество доп. литров в логистике
quantity_liters_for_hr = 1 # количество доп. литров в хранении
localization_index_wb = 0.90 # индекс локализации

#### Файл с объемом в литрах, для формирования "Стоимости логистики"

In [6]:
## Аналитика - Отчеты - Отчет по остаткам на складе (Настройка таблицы - выставить все "галочки")
path_WB_size = path_downloads + f"report_{mwm.dt_now.date().strftime('%Y_%-m_%-d')}.xlsx.xlsx"

#### Акции

In [7]:
## Цены и скидки - календарь акций
WB_ac = [
         'Товары для акции_Распродажа неходового товара_08.04.2024 08.32.42.xlsx',
         'Товары для акции_СКИДКИ ПРОСТО КОСМОС_08.04.2024 08.32.52.xlsx',
          ]

WB_ac_feature = [
                 ]

#### Процент комиссии

In [8]:
## Тарифы - Комиссия - Выгрузить (обновляется редко)
path_commission_percentage = path_downloads + 'Комиссия от 19 февраля.xlsx'

#### Отчет с Озона

In [9]:
## С озона: Аналитика - Отчеты - Все товары и видимость на Ozon - Скачать
path_ozon_stocks_new = wwm.return_path('products*.csv')

#### Цены

In [10]:
## Цены и скидки - Цены и скидки - Скачать Excel по всем товарам - Сформировать - Скачать
path_price = [py for py in \
glob.glob(f"{path_downloads}Шаблон обновления цен и скидок от {mwm.dt_now.date().strftime('%d.%m.%Y')} *.xlsx")][0]

#### Остатки и заказы

In [11]:
## Аналитика - Отчеты - Продажи - Продажи за определенный день
path_WB_new_metric1 = path_downloads + 'supplier-goods-33694-2024-04-08-2024-04-08-ejwajzgwj.xlsx'

# path_WB_new_metric2 = path_downloads + 'supplier-goods-33694-2024-04-06-2024-04-06-wpascrups.xlsx'

# path_WB_new_metric3 = path_downloads + 'supplier-goods-33694-2024-04-07-2024-04-07-hixectjvh.xlsx'



In [12]:
link_to_table

'https://docs.google.com/spreadsheets/d/1A1TyZPeq9r2HiDi-CHfmTfRpuAQYToib1vqyTx4ZyRE/edit#gid=0'

In [13]:
flag_SH = True
reestr = wwm.Reestr()
header_params = const.header_params_wb_fbs_stocks

# flag_stop_nac = True
flag_stop_nac = False

# flag_pr_koledino = True
flag_pr_koledino = False

flag_pr_kaz = True
# flag_pr_kaz = False

# flag_pr_atak = True
flag_pr_atak = False

In [14]:
def dt_col(data):
    date_col = []
    for row in data.columns:
        try:
            pd.to_datetime(row)
            date_col.append(row)
        except:
            None
    return date_col

In [15]:
def uniq(data):

    data['Баркод'] = data['Баркод'].astype(int)

    L = len(data) - len(list(set(data['Баркод'])))

    return L

In [16]:
pr = pd.read_excel(path_price)
pr = pr.rename(columns={'Категория': 'Предмет',
                        'Текущая цена': 'Текущая розн. цена (до скидки)',
                        'Текущая скидка': 'Текущая скидка на сайте, %'})
pr.to_excel(path_price)
pr = None

In [17]:
paths = []
for i in range(1, 20):
    try:
        if locals()[f"path_WB_new_metric{i}"] != None:
            paths.append(locals()[f"path_WB_new_metric{i}"])
    except:
        None

### Статус рекламы

In [18]:
st_r = pd.read_excel(path_downloads + 'ВБ. Мониторинг авторекламы (товары в авторекламе) (1).xlsx',
                     dtype={'Артикул': 'str'},
                     usecols=['Артикул', 'статус рекламы'])
if len(st_r) - len(list(set(st_r['Артикул']))) != 0:
    print('error')
    int('error')


### Новый баркод

In [19]:
nbar = pd.read_excel(path_downloads + 'Реестр (1).xlsx')
nbar = nbar.dropna()
nbar = nbar.drop_duplicates().reset_index(drop=True)
nbar['Баркод'] = nbar['Баркод'].astype(int)
nbar['Новый баркод'] = nbar['Новый баркод'].astype(int)
len(nbar) - len(list(set(nbar['Баркод'])))

0

### Остатки по складам, фильтр "Невинномысск", "Екатеринбург 2" и "Новосибирск"

In [20]:
stocks_balances = pd.read_excel(path_stocks_balances)
nvm_stocks = stocks_balances[['Баркод', 'Невинномысск']].copy()
nvm_stocks = nvm_stocks.fillna(0)
nvm_stocks = nvm_stocks[nvm_stocks['Невинномысск'] > 0]
try:
    nvm_stocks['Баркод'] = nvm_stocks['Баркод'].astype(int)
except:
    print('Ошибка в файле с сотактками в Невинномысске!!!!!!!!!!!!!!!!')
nvm_stocks.columns = ['Баркод', 'Остаток в Невинномысске']
ekb2_nsk_stocks = stocks_balances[['Баркод', 
                                   'Екатеринбург - Испытателей 14г', 
                                   'Екатеринбург - Перспективный 12', 
                                   'Новосибирск']].copy()
ekb2_nsk_stocks = ekb2_nsk_stocks.fillna(0)
ekb2_nsk_stocks = ekb2_nsk_stocks[(ekb2_nsk_stocks['Екатеринбург - Испытателей 14г'] > 0) | (ekb2_nsk_stocks['Екатеринбург - Перспективный 12'] > 0) | (ekb2_nsk_stocks['Новосибирск'] > 0)]
try:
    ekb2_nsk_stocks['Баркод'] = ekb2_nsk_stocks['Баркод'].astype(int)
except:
    print('Ошибка в файле с сотактками в Екатеринбург 2 и Новосибирск!!!!!!!!!!!!!!!!')
ekb2_nsk_stocks.columns = ['Баркод', 
                           'Остаток в Екатеринбург - Испытателей 14г', 
                           'Остаток в Екатеринбург - Перспективный 12', 
                           'Остаток в Новосибирске']

### Логистика и индекс локализации

In [21]:
logistic_korob = pd.read_excel(path_logistic,
                               sheet_name='Короба')

logistic_monop = pd.read_excel(path_logistic,
                               sheet_name='Монопаллеты')

In [22]:
logistic_monop['Логистика, %'] = logistic_monop['Коэффициент, %']
logistic_monop['Хранение, %'] = logistic_monop['Коэффициент, %']
logistic_monop['за 1 л'] = logistic_monop['Логистика за 1 л']
logistic_monop['1 паллета'] = logistic_monop['Хранение за 1 паллету']
logistic_monop = logistic_monop[['Склад', 'Логистика, %', 'за 1 л', 'доп. л', 'Хранение, %', '1 паллета']]

In [23]:
if logistic_korob.columns[0] == 'Unnamed: 0':
    logistic_korob.columns = ['Склад', 'Логистика и хранение, %', 'Логистика за 1л', 'доп. л',
       'Хранение за 1л', 'доп. л.1']
    logistic_monop.columns = ['Склад', 'Логистика, %', 'за 1 л', 'доп. л', 'Хранение, %',
       '1 паллета']
elif logistic_korob.columns[0] == 'Склад':
    logistic_korob.columns = ['Склад', 'Логистика и хранение, %', 'Логистика за 1л', 'доп. л',
       'Хранение за 1л', 'доп. л.1']
    logistic_monop.columns = ['Склад', 'Логистика, %', 'за 1 л', 'доп. л', 'Хранение, %',
       '1 паллета']
    
logistic_korob['Логистика и хранение, %'] = logistic_korob['Логистика и хранение, %'].astype(int)
Korob = {}
for i in range(len(logistic_korob)):
    Korob[logistic_korob.loc[i]['Склад']] = logistic_korob.loc[i]['Логистика и хранение, %']
    
logistic_monop['Логистика, %'] = logistic_monop['Логистика, %'].astype(int)
logistic_monop['Хранение, %'] = logistic_monop['Хранение, %'].astype(int)
Monop = {}
for i in range(len(logistic_monop)):
    Monop[logistic_monop.loc[i]['Склад']] = [logistic_monop.loc[i]['Логистика, %'], logistic_monop.loc[i]['Хранение, %']]
    
SK = sorted(list(set(list(Korob.keys()) + list(Monop.keys()))))
LOG = pd.DataFrame(columns=['Склад', 
     'Стоимость логистики монопалета (база)', 
     'Стоимость логистики монопалета (доп. литр)',
     'Стоимость хранения монопалета (за день)',
     'Стоимость логистики короб (база)', 
     'Стоимость логистики короб (доп. литр)',
     'Стоимость хранения короб (база)',
     'Стоимость хранения короб (доп. литр)'])
LOG['Склад'] = SK
for i in range(len(LOG)):
    s = LOG.loc[i]['Склад']
    if s not in list(Korob.keys()):
        LOG['Стоимость логистики монопалета (база)'].loc[i] = Monop[s][0] * (Log_b[0] / 100)
        LOG['Стоимость логистики монопалета (доп. литр)'].loc[i] = Monop[s][1] * (Log_b[1] / 100)
        LOG['Стоимость хранения монопалета (за день)'].loc[i] = Monop[s][1] * (Hr_b_monop / 100)
    elif s not in list(Monop.keys()):
        LOG['Стоимость логистики короб (база)'].loc[i] = Korob[s] * (Log_b[0] / 100)
        LOG['Стоимость логистики короб (доп. литр)'].loc[i] = Korob[s] * (Log_b[1] / 100)
        LOG['Стоимость хранения короб (база)'].loc[i] = Korob[s] * (Hr_b_korob[0] / 100) 
        LOG['Стоимость хранения короб (доп. литр)'].loc[i] = Korob[s] * (Hr_b_korob[1] / 100)
    elif s in list(Korob.keys()) and s in list(Monop.keys()):
        LOG['Стоимость логистики монопалета (база)'].loc[i] = Monop[s][0] * (Log_b[0] / 100)
        LOG['Стоимость логистики монопалета (доп. литр)'].loc[i] = Monop[s][1] * (Log_b[1] / 100)
        LOG['Стоимость хранения монопалета (за день)'].loc[i] = Monop[s][1] * (Hr_b_monop / 100)  
        LOG['Стоимость логистики короб (база)'].loc[i] = Korob[s] * (Log_b[0] / 100)
        LOG['Стоимость логистики короб (доп. литр)'].loc[i] = Korob[s] * (Log_b[1] / 100)
        LOG['Стоимость хранения короб (база)'].loc[i] = Korob[s] * (Hr_b_korob[0] / 100) 
        LOG['Стоимость хранения короб (доп. литр)'].loc[i] = Korob[s] * (Hr_b_korob[1] / 100)
    else:
        print('Склады в базе и в метриках не соответствуют!')

Log_pr_koledino_1 = LOG[LOG['Склад']=='Коледино']['Стоимость логистики короб (база)'].values[0]
Log_pr_koledino_2 = LOG[LOG['Склад']=='Коледино']['Стоимость логистики короб (доп. литр)'].values[0]
Hr_pr_koledino_1 =  LOG[LOG['Склад']=='Коледино']['Стоимость хранения короб (база)'].values[0]
Hr_pr_koledino_2 =  LOG[LOG['Склад']=='Коледино']['Стоимость хранения короб (доп. литр)'].values[0]

Log_pr_marketplace_1 = LOG[LOG['Склад']=='Маркетплейс']['Стоимость логистики короб (база)'].values[0]
Log_pr_marketplace_2 = LOG[LOG['Склад']=='Маркетплейс']['Стоимость логистики короб (доп. литр)'].values[0]
Hr_pr_marketplace_1 =  LOG[LOG['Склад']=='Маркетплейс']['Стоимость хранения короб (база)'].values[0]
Hr_pr_marketplace_2 =  LOG[LOG['Склад']=='Маркетплейс']['Стоимость хранения короб (доп. литр)'].values[0]

Log_pr_kaz_1 = LOG[LOG['Склад']=='Казань']['Стоимость логистики короб (база)'].values[0]
Log_pr_kaz_2 = LOG[LOG['Склад']=='Казань']['Стоимость логистики короб (доп. литр)'].values[0]
Hr_pr_kaz_1 =  LOG[LOG['Склад']=='Казань']['Стоимость хранения короб (база)'].values[0]
Hr_pr_kaz_2 =  LOG[LOG['Склад']=='Казань']['Стоимость хранения короб (доп. литр)'].values[0]

Log_pr_atak_1 = LOG[LOG['Склад']=='Атакент']['Стоимость логистики короб (база)'].values[0]
Log_pr_atak_2 = LOG[LOG['Склад']=='Атакент']['Стоимость логистики короб (доп. литр)'].values[0]
Hr_pr_atak_1 =  LOG[LOG['Склад']=='Атакент']['Стоимость хранения короб (база)'].values[0]
Hr_pr_atak_2 =  LOG[LOG['Склад']=='Атакент']['Стоимость хранения короб (доп. литр)'].values[0]

In [24]:
def unloading():
    
    with pd.ExcelWriter(path_algoritm + "Все файлы для ежедневной презентации/Логистика и хранение WB.xlsx") as writer:


        LOG.to_excel(writer, sheet_name="Логистика и хранение", index=False)
        for column in LOG:
            column_width = max(LOG[column].astype(str).map(len).max(), len(column))
            col_idx = LOG.columns.get_loc(column)
            writer.sheets['Логистика и хранение'].set_column(col_idx, col_idx, column_width)



    writer.save()

In [25]:
try:
    unloading()
except:
    unloading()

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


### Доп. признак для маркетинга

In [26]:
# Спринт WB
path_spr_wb = path_downloads + 'Спринт вб.xlsx'
# Спринт OZON
path_spr_oz = path_downloads + 'Спринт озон.xlsx'

### Файл с объемом в литрах, для формирования "Стоимости логистики"

In [27]:
wbz = pd.read_excel(path_WB_size)
wbz = wbz.rename(columns={list(filter(lambda x: 'Объем, л' in x, list(wbz.columns)))[0]: 'Объем, л'})
wbz.to_excel(path_WB_size, index=False)

path_WB_size_metrics = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Метрики по объему/WB_size_metrics.xlsx'

### Остатки FBS 

In [28]:
path_WB_ofbs_metrics = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/ОСТАТКИ FBS/WB_ofbs_metrics.csv'

# # Загружается каждый день (управление остатками на вб)

# path_WB_ofbs_1 = path_downloads + 'stocks.xlsx'
# path_WB_ofbs_2 = path_downloads + 'stocks (1).xlsx'
# path_WB_ofbs_3 = path_downloads + 'stocks (2).xlsx'
# path_WB_ofbs_4 = path_downloads + 'stocks (3).xlsx'

# fbs1 = pd.read_excel(path_WB_ofbs_1)
# fbs2 = pd.read_excel(path_WB_ofbs_2)
# fbs3 = pd.read_excel(path_WB_ofbs_3)
# fbs4 = pd.read_excel(path_WB_ofbs_4)
# fbs = pd.concat([fbs1, fbs2], ignore_index=True)
# fbs = pd.concat([fbs, fbs3], ignore_index=True)
# fbs = pd.concat([fbs, fbs4], ignore_index=True)
# fbs.to_excel(path_downloads + 'stocks_concat.xlsx', index=False)

path_WB_ofbs = path_downloads + 'stocks_concat.xlsx'

## Пути к ежедневно обновляемым файлам  

### Заказы, остатки и цены

In [29]:
# # Загружается каждый день 

# for i in range(1, 20):
#     locals()[f"path_WB_new_metric{i}"] = None

# # файл с ЦЕНАМИ последними (из Загрузок)


# # path_price = path_downloads + 'Шаблон обновления цен и скидок от 25.03.2024 08.31.xlsx'    # (цены и скидки на вб)

# path_price = [py for py in \
# glob.glob(f"{path_downloads}Шаблон обновления цен и скидок от {mwm.dt_now.date().strftime('%d.%m.%Y')} *.xlsx")][0]

# pr = pd.read_excel(path_price)
# pr = pr.rename(columns={'Категория': 'Предмет',
#                         'Текущая цена': 'Текущая розн. цена (до скидки)',
#                         'Текущая скидка': 'Текущая скидка на сайте, %'})
# pr.to_excel(path_price)
# pr = None


# # файлы с ОСТАТКАМИ и ЗАКАЗАМИ за день (из Загрузок)

# path_WB_new_metric1 = path_downloads + 'supplier-goods-33694-2024-04-07-2024-04-07-sybwdwcar.xlsx'   # (отчет 3 на вб)

# # path_WB_new_metric2 = path_downloads + 'supplier-goods-33694-2024-04-06-2024-04-06-wpascrups.xlsx'

# # path_WB_new_metric3 = path_downloads + 'supplier-goods-33694-2024-04-07-2024-04-07-hixectjvh.xlsx'

# # path_WB_new_metric4 = path_downloads + 'supplier-goods-33694-2024-03-10-2024-03-10-jolotaewr.xlsx'

# # path_WB_new_metric5 = path_downloads + 'supplier-goods-33694-2024-01-06-2024-01-06-flcyxdykb.xlsx'

# # path_WB_new_metric6 = path_downloads + 'supplier-goods-33694-2023-01-04-2023-01-04-mgglbsduw.XLSX'

# # path_WB_new_metric7 = path_downloads + 'supplier-goods-33694-2023-01-05-2023-01-05-xmryohitv.XLSX'

# # path_WB_new_metric8 = path_downloads + 'supplier-goods-33694-2023-01-06-2023-01-06-oaliebwqp.XLSX'

# # path_WB_new_metric9 = path_downloads + 'supplier-goods-33694-2023-01-07-2023-01-07-rhxhvyxnd.XLSX'

# # path_WB_new_metric10 = path_downloads + 'supplier-goods-33694-2023-01-08-2023-01-08-whtzkzfcm.XLSX'





# paths = []
# for i in range(1, 20):
#     try:
#         if locals()[f"path_WB_new_metric{i}"] != None:
#             paths.append(locals()[f"path_WB_new_metric{i}"])
#     except:
#         None
        
# paths

## Метрики

### Метрики, к которым происходит подгрузка

In [30]:
# файл с "историческими" метриками

path_WB_orders_metrics = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Исторические метрики /WB_orders_metrics.csv'

# файл с "обезличенными" товарами

path_WB_depersonalized_metric = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Обезличенные товары/WB_depersonalized_metric.csv'

# файл с метриками цен

path_WB_price_metric = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Цены по дням /WB_price_metric.csv'

# файл с метриками остатков

path_stocks_WB_metric = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Остатки по дням /WB_Stocks.csv'

### Метрики, которые каждый раз обновляются

In [31]:
# файл со всеми обработанными признаками

path_WB_full_features_metric = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Метрики с расчетами и выгрузкой за день /WB_full_features_metric.csv'

# Путь для файла "метрики финальные для презентации"

path_WB_full_df = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Метрики для презентации /WB_full_df.csv'

## Пути к подгружаемым и ежедневно не обновляемым файлам

In [32]:
# файл с признаком "Предмет" (для WB)

path_object_df = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Подгружаемые признаки по WB/"Предмет"/object_df.csv'

# файл с признаками "Проценты и стоимости" (для WB)

path_percent_cost = path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Подгружаемые признаки по WB/"Проценты и стоимости"/percent_cost.csv'

# файл с признаком "Себестоимость" (общий)

path_cost_price = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/Себестоимость /cost_price 2.0.xlsx'

# файл с признаком "РЦ" (общий)

path_rc = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/РЦ/РЦ_обработанные.xlsx'


# файл с признаком "В резерве" (общий)

path_vr = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/В резерве/В резерве_обработанные.xlsx'


In [33]:
def print_duplicates_art(data):
    
    if 'Артикул' in data.columns:
        print(f"Уникальных артикулов: {len(list(set(data['Артикул'])))}")
        print(f"Всего артикулов: {len(list(data['Артикул']))}")
        print()
        print()
        print()
        data = data.reset_index(drop=True)
        for i in range(len(data)):
            art = data.loc[i]['Артикул']
            if len(data[data['Артикул']==art]) > 1:
                display(data[data['Артикул']==art])
    else:
        None

In [34]:
def pda(data):
    
    if 'Артикул' in data.columns:
        print(f"Уникальных артикулов: {len(list(set(data['Артикул'])))}")
        print(f"Всего артикулов: {len(list(data['Артикул']))}")
        print()
        print()
        print()
        data = data.reset_index(drop=True)
        for i in range(len(data)):
            art = data.loc[i]['Артикул']
            if len(data[data['Артикул']==art]) > 1:
                display(data[data['Артикул']==art])
    else:
        None

### Удалить последний день из метрик

In [35]:
def Apocalypse():
    
    WB_orders_metrics = pd.read_csv(path_WB_orders_metrics, sep=';')
    WB_orders_metrics = WB_orders_metrics.iloc[:, :-1]
    WB_orders_metrics.to_csv(path_WB_orders_metrics, index=False, sep=";", encoding='utf-8-sig')
    
#     WB_depersonalized_metric = pd.read_csv(path_WB_depersonalized_metric, sep=';')
#     WB_depersonalized_metric = WB_depersonalized_metric.iloc[:, :-2]
#     WB_depersonalized_metric.to_csv(path_WB_depersonalized_metric, index=False, sep=";", encoding='utf-8-sig')
    
    WB_price_metric = pd.read_csv(path_WB_price_metric, sep=';')
    WB_price_metric = WB_price_metric.iloc[:, :-1]
    WB_price_metric.to_csv(path_WB_price_metric, index=False, sep=";", encoding='utf-8-sig')
    
    stocks_WB_metric = pd.read_csv(path_stocks_WB_metric, sep=';')
    stocks_WB_metric = stocks_WB_metric.iloc[:, :-1]
    stocks_WB_metric.to_csv(path_stocks_WB_metric, index=False, sep=";", encoding='utf-8-sig')
    

In [36]:
# Apocalypse()

## Функции работы с таблицами по WB

### Функция замены русских букв, если они есть в "Артикулах"

In [37]:
def letter_substitution(data):
    
    data = data.rename(columns={data.columns[0]: 'Артикул'})
    data['Артикул'] = data['Артикул'].astype(str)
    data = data.reset_index(drop=True)
    
    fa = list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(data['Артикул'])))

    if len(fa) > 0:

        Q = {'е': 'e', 
             'Е': 'E', 
             'Т': 'T', 
             'у': 'y', 
             'о': 'o', 
             'О': 'O', 
             'р': 'p', 
             'Р': 'P', 
             'а': 'a', 
             'А': 'A', 
             'Н': 'H',
             'К': 'K',
             'х': 'x',
             'Х': 'X',
             'с': 'c',
             'С': 'C',
             'В': 'B',
             'М': 'M'}

        for i in range(len(data)):

            for row in list(Q.keys()):

                if row in data.loc[i]['Артикул']:

                    data['Артикул'].loc[i] = data.loc[i]['Артикул'].replace(row, Q[row])


    fa2 = list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(data['Артикул'])))

    if len(fa2) == 0:

        return data




### Функция сортировки столбцов в порядке течения времени

In [38]:
def data_columns_sorted(data, Ost):
        
    R = []
    Q = []
    w = list(data.columns)[:5]
    
    if Ost == True:
        
        w1 = [list(data.columns)[-1]]
        

        for row in list(data.columns)[5:-1]:
            R.append(datetime.datetime.strptime(row, '%d-%B-%Y'))
        R.sort()

        for row in R:
            Q.append(row.strftime('%d-%B-%Y'))
        data = data[w + Q + w1]
    
    
    elif Ost == False:
        
        
        for row in list(data.columns)[5:]:
            R.append(datetime.datetime.strptime(row, '%d-%B-%Y'))
        R.sort()
        
        for row in R:
            Q.append(row.strftime('%d-%B-%Y'))
            
        data = data[w + Q]
        
        
    return data



### Функция обработки заказов с WB: меняем значения для "неуникальных" артикулов

In [39]:
def drop_products(data):
    
    data['Баркод'] = data['Баркод'].astype(int)
    
    
    
    
    for i in range(len(data)):
        b = data.loc[i]['Баркод']
        if len(data[data['Баркод']==b]) > 1:
            if b == 4648894810020:
                ind = sorted(list(WB_orders_metrics[WB_orders_metrics['Баркод']==4648894810020].index))
                if len(ind) == 2:
                    data.loc[ind[1], 'Баркод'] = 4648877486747
                
                
                
                
    
    if (len(data[(data['Артикул']=='27601') & (data['Баркод']==7432141550858)]) != 0 and
        len(data[(data['Артикул']=='27601') & (data['Баркод']==7432141550847)]) != 0):
        
        index1 = data[(data['Артикул']=='27601') & (data['Баркод']==7432141550858)].index[0]
        index2 = data[(data['Артикул']=='27601') & (data['Баркод']==7432141550847)].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
        
        
        
        
        
    if (len(data[(data['Артикул']=='с20502') & (data['Баркод']==8112389834753)]) != 0 and
        len(data[(data['Артикул']=='с20502') & (data['Баркод']==7432141550853)]) != 0):
        
        index1 = data[(data['Артикул']=='с20502') & (data['Баркод']==8112389834753)].index[0]
        index2 = data[(data['Артикул']=='с20502') & (data['Баркод']==7432141550853)].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
        
        
        
        
    if (len(data[(data['Артикул']=='G0132') & (data['Баркод']==7432141556745)]) != 0 and
        len(data[(data['Артикул']=='G0132') & (data['Баркод']==7432141556750)]) != 0):
        
        index1 = data[(data['Артикул']=='G0132') & (data['Баркод']==7432141556745)].index[0]
        index2 = data[(data['Артикул']=='G0132') & (data['Баркод']==7432141556750)].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
        
        
        
#     if (len(data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556871)]) != 0 and
#         len(data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556874)]) != 0):
        
#         index1 = data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556871)].index[0]
#         index2 = data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556874)].index[0]
        
#         for row in list(data.columns[5:]):
    
#             data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
#         data = data.drop(index=index2)
        
        
        
    if (len(data[(data['Артикул']=='50170') & (data['Баркод']==7432141557064)]) != 0 and
        len(data[(data['Артикул']=='50170') & (data['Баркод']==7432141557063)]) != 0):
        
        index1 = data[(data['Артикул']=='50170') & (data['Баркод']==7432141557064)].index[0]
        index2 = data[(data['Артикул']=='50170') & (data['Баркод']==7432141557063)].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
        
        
        
    if len(data[data['Артикул']=='286011']) != 0:
        
        index1 = data[data['Артикул']=='286011'].index[0]
        
        data['Артикул'].loc[index1] = '28611'
        
        
    if len(data[data['Артикул']=='286012']) != 0:
        
        index1 = data[data['Артикул']=='286012'].index[0]
        
        data['Артикул'].loc[index1] = '28612' 
        
        

    if len(data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556874)]) != 0:
        
        index1 = data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556874)].index[0]
        
        data['Артикул'].loc[index1] = 'G0276' 
        
        
        
        
        
        
    if len(data[(data['Артикул']!='G0275') & (data['Баркод']==7432141556872)]) != 0:
        
        index1 = data[(data['Артикул']!='G0275') & (data['Баркод']==7432141556872)].index[0]
        
        data['Артикул'].loc[index1] = 'G0275' 
        
        
        
        
        
        
    if len(data[(data['Артикул']!='G0274') & (data['Баркод']==7432141556871)]) != 0:
        
        index1 = data[(data['Артикул']!='G0274') & (data['Баркод']==7432141556871)].index[0]
        
        data['Артикул'].loc[index1] = 'G0274' 
        
        
        
        
    if len(data[(data['Артикул']!='G0276') & (data['Баркод']==7432141556874)]) != 0:
        
        index1 = data[(data['Артикул']!='G0276') & (data['Баркод']==7432141556874)].index[0]
        
        data['Артикул'].loc[index1] = 'G0276'
        
        
        
        
        
        
        
        
        
    if len(data[(data['Артикул']=='G0283') & (data['Баркод']==4648957260304)]) != 0:
        
        index1 = data[(data['Артикул']=='G0283') & (data['Баркод']==4648957260304)].index[0]
        
        data['Артикул'].loc[index1] = 'G0284' 
        
        
        
        
    if len(data[(data['Артикул']=='G0283') & (data['Баркод']==4648877486747)]) != 0:
        
        index1 = data[(data['Артикул']=='G0283') & (data['Баркод']==4648877486747)].index[0]
        
        data['Артикул'].loc[index1] = 'G0285'  
        
        
        
        
    if len(data[(data['Артикул']=='G0700') & (data['Баркод']==4654613663099)]) != 0:
        
        index1 = data[(data['Артикул']=='G0700') & (data['Баркод']==4654613663099)].index[0]
        
        data['Артикул'].loc[index1] = 'G0701'  
        
        
        
    if len(data[(data['Артикул']=='G0700') & (data['Баркод']==4654632683702)]) != 0:
        
        index1 = data[(data['Артикул']=='G0700') & (data['Баркод']==4654632683702)].index[0]
        
        data['Артикул'].loc[index1] = 'G0702'  
        
        
        
    if len(data[(data['Артикул']=='G0703') & (data['Баркод']==4654806520703)]) != 0:
        
        index1 = data[(data['Артикул']=='G0703') & (data['Баркод']==4654806520703)].index[0]
        
        data['Артикул'].loc[index1] = 'G0705'  
        

        
        
    if len(data[(data['Артикул']=='G0703') & (data['Баркод']==4654762199616)]) != 0:
        
        index1 = data[(data['Артикул']=='G0703') & (data['Баркод']==4654762199616)].index[0]
        
        data['Артикул'].loc[index1] = 'G0704'  
        
        
        
    if len(data[(data['Артикул']=='G0706') & (data['Баркод']==4654957036634)]) != 0:
        
        index1 = data[(data['Артикул']=='G0706') & (data['Баркод']==4654957036634)].index[0]
        
        data['Артикул'].loc[index1] = 'G0707'  
        
        
        
        
        
    if len(data[(data['Артикул']=='G0706') & (data['Баркод']==4654990326389)]) != 0:
        
        index1 = data[(data['Артикул']=='G0706') & (data['Баркод']==4654990326389)].index[0]
        
        data['Артикул'].loc[index1] = 'G0708' 
        
        
        
        
        
        
    if len(data[(data['Артикул']=='G0709') & (data['Баркод']==4655387531096)]) != 0:
        
        index1 = data[(data['Артикул']=='G0709') & (data['Баркод']==4655387531096)].index[0]
        
        data['Артикул'].loc[index1] = 'G0710' 
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
    if len(data[(data['Артикул']!='50301') & (data['Баркод']==7432141557124)]) != 0:
        
        index1 = data[(data['Артикул']!='50301') & (data['Баркод']==7432141557124)].index[0]
        
        data['Артикул'].loc[index1] = '50301'
        
        

    if len(data[(data['Артикул']!='50532') & (data['Баркод']==7432141557215)]) != 0:
        
        index1 = data[(data['Артикул']!='50532') & (data['Баркод']==7432141557215)].index[0]
        
        data['Артикул'].loc[index1] = '50532'
        
        
        
    if len(data[(data['Артикул']!='50533') & (data['Баркод']==7432141557220)]) != 0:
        
        index1 = data[(data['Артикул']!='50533') & (data['Баркод']==7432141557220)].index[0]
        
        data['Артикул'].loc[index1] = '50533'
        
        
        
        
        

        
        

        
        
        
    if len(data[data['Артикул']=='28611']) != 0 and len(data[data['Артикул']=='286011']) != 0:
        
        index1 = data[data['Артикул']=='28611'].index[0]
        index2 = data[data['Артикул']=='286011'].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
                                                             
                                                             
    if len(data[data['Артикул']=='28612']) != 0 and len(data[data['Артикул']=='286012']) != 0:
        
        index1 = data[data['Артикул']=='28612'].index[0]
        index2 = data[data['Артикул']=='286012'].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)




        
        
        
        
        
        
    N = {'50171': 7432141557066,
         '50531': 7432141557217,
         'D0030': 7432141559042,
         'D0031': 7432141559045,
         'D0032': 7432141559048,
         'D0033': 7432141559051,
         'G0193': 7432141556813,
         'G0194': 7432141556816,
         'J0400': 0}
    
    for row in N:
        
        if len(data[(data['Артикул']==row) & (data['Баркод']==N[row])]) != 0:
            
            index = data[(data['Артикул']==row) & (data['Баркод']==N[row])].index[0]
                
                
            data = data.drop(index=index)
            
            
            
    L = {'7432141556855': ['Наколенник с гелевым защитным кольцом size M ', 'G0260', 36537523],
         '7432141556860': ['Наколенник с гелевым защитным кольцом, size L ', 'G0261', 36537523],
         '7432141556863': ['Наколенник с гелевым защитным кольцом, Sixe XL', 'G0262', 36537523],
         '7432141556864': ['Наколенник, size S', 'G0270', 38099518],
         '7432141556867': ['Наколенник, size M', 'G0271', 38099518],
         '7432141556870': ['Наколенник, size L', 'G0272', 38099518],
         '7432141556873': ['Наколенник, Sixe XL', 'G0273', 38099518],
         '7432141556876': ['Наколенники, комплект 2 шт, size M', 'G0280', 36654736],
         '7432141556879': ['Наколенники, комплект 2 шт, size L', 'G0281', 36654736],
         '7432141556882': ['Наколенники, комплект 2 шт, Sixe XL', 'G0282', 36654736], 
         '7432141557627': ['Ортопедический корсет, размер S', 'G0370', 37341958],
         '7432141557630': ['Ортопедический корсет, размер M', 'G0371', 37341958],
         '7432141557633': ['Ортопедический корсет, размер L', 'G0372', 37341958],
         '7432141557636': ['Ортопедический корсет, размер XL', 'G0373', 37341958],
         '7432141557639': ['Ортопедический корсет, размер XXL', 'G0374', 37341958],
         '7432141557651': ['Спортивный корсет, размер S', 'G0400', 37325184],
         '7432141557654': ['Спортивный корсет, размер M', 'G0401', 37325184], 
         '7432141557657': ['Спортивный корсет, размер L', 'G0402', 37325184],
         '7432141557660': ['Спортивный корсет, размер XL', 'G0403', 37325184], 
         '7432141557663': ['Спортивный корсет, размер XXL', 'G0404', 37325184],
         '7432141557666': ['Спортивный пояс, размер M', 'G0410', 37338500],
         '7432141557669': ['Спортивный пояс, размер L', 'G0411', 37338500],
         '7432141557672': ['Спортивный пояс, размер XL', 'G0412', 37338500],
         '4683663822330': ['Спортивный пояс, размер XXL', 'G0413', 37338500]}
         
         
         
    for row in list(set(data['Баркод'])):
         
        if data[data['Баркод']==row]['Баркод'].values[0] in list(map(int, list(L.keys()))):
            
            index = data[data['Баркод']==row].index[0]
            
            
            data['Артикул'].loc[index] = L[str(row)][1]
            
            data['Наименование'].loc[index] = L[str(row)][0]
            
            data['Номенклатура'].loc[index] = L[str(row)][2]

            
    data2 = data.copy()
    
    data2 = data2.drop(['Артикул', 'Наименование'], axis=1)
    
    data2 = data2.drop_duplicates()
    
    k = list(data2.index)

    data = data.query('index in @k')
    
    data = data.reset_index(drop=True)
    
    data2 = None
        

    return data
        

### Функция обработки заказов с WB: меняем значения для "неуникальных" артикулов (для расчета динамического прогноза)

In [40]:
def drop_products_for_dynamics(data):
    
    data['Баркод'] = data['Баркод'].astype(int)
    
    if (len(data[(data['Артикул']=='27601') & (data['Баркод']==7432141550858)]) != 0 and
        len(data[(data['Артикул']=='27601') & (data['Баркод']==7432141550847)]) != 0):
        
        index1 = data[(data['Артикул']=='27601') & (data['Баркод']==7432141550858)].index[0]
        index2 = data[(data['Артикул']=='27601') & (data['Баркод']==7432141550847)].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
        
          
        
        
        
    if (len(data[(data['Артикул']=='с20502') & (data['Баркод']==8112389834753)]) != 0 and
        len(data[(data['Артикул']=='с20502') & (data['Баркод']==7432141550853)]) != 0):
        
        index1 = data[(data['Артикул']=='с20502') & (data['Баркод']==8112389834753)].index[0]
        index2 = data[(data['Артикул']=='с20502') & (data['Баркод']==7432141550853)].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
        
        

        
    if (len(data[(data['Артикул']=='G0132') & (data['Баркод']==7432141556745)]) != 0 and
        len(data[(data['Артикул']=='G0132') & (data['Баркод']==7432141556750)]) != 0):
        
        index1 = data[(data['Артикул']=='G0132') & (data['Баркод']==7432141556745)].index[0]
        index2 = data[(data['Артикул']=='G0132') & (data['Баркод']==7432141556750)].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
        
        
        
#     if (len(data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556871)]) != 0 and
#         len(data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556874)]) != 0):
        
#         index1 = data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556871)].index[0]
#         index2 = data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556874)].index[0]
        
#         for row in list(data.columns[5:]):
    
#             data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
#         data = data.drop(index=index2)
        
        
        
        
        
    if (len(data[(data['Артикул']=='50170') & (data['Баркод']==7432141557064)]) != 0 and
        len(data[(data['Артикул']=='50170') & (data['Баркод']==7432141557063)]) != 0):
        
        index1 = data[(data['Артикул']=='50170') & (data['Баркод']==7432141557064)].index[0]
        index2 = data[(data['Артикул']=='50170') & (data['Баркод']==7432141557063)].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
        
        
        
    if len(data[data['Артикул']=='286011']) != 0:
        
        index1 = data[data['Артикул']=='286011'].index[0]
        
        data['Артикул'].loc[index1] = '28611'
        
        
    if len(data[data['Артикул']=='286012']) != 0:
        
        index1 = data[data['Артикул']=='286012'].index[0]
        
        data['Артикул'].loc[index1] = '28612'            

        
        
        
        
    if len(data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556874)]) != 0:
        
        index1 = data[(data['Артикул']=='G0274') & (data['Баркод']==7432141556874)].index[0]
        
        data['Артикул'].loc[index1] = 'G0276' 
        
        
    if len(data[(data['Артикул']!='G0275') & (data['Баркод']==7432141556872)]) != 0:
        
        index1 = data[(data['Артикул']!='G0275') & (data['Баркод']==7432141556872)].index[0]
        
        data['Артикул'].loc[index1] = 'G0275' 
        
        
        
        
        
        
    if len(data[(data['Артикул']!='G0274') & (data['Баркод']==7432141556871)]) != 0:
        
        index1 = data[(data['Артикул']!='G0274') & (data['Баркод']==7432141556871)].index[0]
        
        data['Артикул'].loc[index1] = 'G0274' 
        
        
        
        
    if len(data[(data['Артикул']!='G0276') & (data['Баркод']==7432141556874)]) != 0:
        
        index1 = data[(data['Артикул']!='G0276') & (data['Баркод']==7432141556874)].index[0]
        
        data['Артикул'].loc[index1] = 'G0276'
        
        
        
        
        
    if len(data[(data['Артикул']!='50301') & (data['Баркод']==7432141557124)]) != 0:
        
        index1 = data[(data['Артикул']!='50301') & (data['Баркод']==7432141557124)].index[0]
        
        data['Артикул'].loc[index1] = '50301'
        
        

    if len(data[(data['Артикул']!='50532') & (data['Баркод']==7432141557215)]) != 0:
        
        index1 = data[(data['Артикул']!='50532') & (data['Баркод']==7432141557215)].index[0]
        
        data['Артикул'].loc[index1] = '50532'
        
        
        
    if len(data[(data['Артикул']!='50533') & (data['Баркод']==7432141557220)]) != 0:
        
        index1 = data[(data['Артикул']!='50533') & (data['Баркод']==7432141557220)].index[0]
        
        data['Артикул'].loc[index1] = '50533'        
        
        
        
        
        
    if len(data[data['Артикул']=='28611']) != 0 and len(data[data['Артикул']=='286011']) != 0:
        
        index1 = data[data['Артикул']=='28611'].index[0]
        index2 = data[data['Артикул']=='286011'].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
                                                             
                                                             
    if len(data[data['Артикул']=='28612']) != 0 and len(data[data['Артикул']=='286012']) != 0:
        
        index1 = data[data['Артикул']=='28612'].index[0]
        index2 = data[data['Артикул']=='286012'].index[0]
        
        for row in list(data.columns[5:]):
    
            data[row].loc[index1] = data.loc[index1][row] + data.loc[index2][row]
        
        data = data.drop(index=index2)
        
        
        
        
        
        
    N = {'50171': 7432141557066,
         '50531': 7432141557217,
         'D0030': 7432141559042,
         'D0031': 7432141559045,
         'D0032': 7432141559048,
         'D0033': 7432141559051,
         'G0193': 7432141556813,
         'G0194': 7432141556816,
         'J0400': 0}
    
    for row in N:
        
        if len(data[(data['Артикул']==row) & (data['Баркод']==N[row])]) != 0:
            
            index = data[(data['Артикул']==row) & (data['Баркод']==N[row])].index[0]
                
                
            data = data.drop(index=index)
            
            
    M = {'Z0010': [7432141559103, 7432141559104],
         'Z0011': [7432141559106, 7432141559107], 
         'Z0020': [7432141559110, 7432141559109],
         'Z0021': [7432141559113, 7432141559112]}
    
    
    for row in M:
        
        
        if (len(data[(data['Артикул']==row) & (data['Баркод']==M[row][0])]) != 0 and
            len(data[(data['Артикул']==row) & (data['Баркод']==M[row][1])]) != 0):
        
            index1 = data[(data['Артикул']==row) & (data['Баркод']==M[row][0])].index[0]
            
            index2 = data[(data['Артикул']==row) & (data['Баркод']==M[row][1])].index[0]
        
            for row_2 in list(data.columns[5:]):
    
                data[row_2].loc[index1] = data.loc[index1][row_2] + data.loc[index2][row_2]
        
            data = data.drop(index=index2)
            
            
            
            
    L = {'7432141556855': ['Наколенник с гелевым защитным кольцом size M ', 'G0260', 36537523],
         '7432141556860': ['Наколенник с гелевым защитным кольцом, size L ', 'G0261', 36537523],
         '7432141556863': ['Наколенник с гелевым защитным кольцом, Sixe XL', 'G0262', 36537523],
         '7432141556864': ['Наколенник, size S', 'G0270', 38099518],
         '7432141556867': ['Наколенник, size M', 'G0271', 38099518],
         '7432141556870': ['Наколенник, size L', 'G0272', 38099518],
         '7432141556873': ['Наколенник, Sixe XL', 'G0273', 38099518],
         '7432141556876': ['Наколенники, комплект 2 шт, size M', 'G0280', 36654736],
         '7432141556879': ['Наколенники, комплект 2 шт, size L', 'G0281', 36654736],
         '7432141556882': ['Наколенники, комплект 2 шт, Sixe XL', 'G0282', 36654736], 
         '7432141557627': ['Ортопедический корсет, размер S', 'G0370', 37341958],
         '7432141557630': ['Ортопедический корсет, размер M', 'G0371', 37341958],
         '7432141557633': ['Ортопедический корсет, размер L', 'G0372', 37341958],
         '7432141557636': ['Ортопедический корсет, размер XL', 'G0373', 37341958],
         '7432141557639': ['Ортопедический корсет, размер XXL', 'G0374', 37341958],
         '7432141557651': ['Спортивный корсет, размер S', 'G0400', 37325184],
         '7432141557654': ['Спортивный корсет, размер M', 'G0401', 37325184], 
         '7432141557657': ['Спортивный корсет, размер L', 'G0402', 37325184],
         '7432141557660': ['Спортивный корсет, размер XL', 'G0403', 37325184], 
         '7432141557663': ['Спортивный корсет, размер XXL', 'G0404', 37325184],
         '7432141557666': ['Спортивный пояс, размер M', 'G0410', 37338500],
         '7432141557669': ['Спортивный пояс, размер L', 'G0411', 37338500],
         '7432141557672': ['Спортивный пояс, размер XL', 'G0412', 37338500],
         '4683663822330': ['Спортивный пояс, размер XXL', 'G0413', 37338500]}
         
         
         
    for row in list(set(data['Баркод'])):
         
        if data[data['Баркод']==row]['Баркод'].values[0] in list(map(int, list(L.keys()))):
            
            index = data[data['Баркод']==row].index[0]
            
            
            data['Артикул'].loc[index] = L[str(row)][1]
            
            data['Наименование'].loc[index] = L[str(row)][0]
            
            data['Номенклатура'].loc[index] = L[str(row)][2]
    
    
    
    data2 = data.copy()
    
    data2 = data2.drop(['Артикул', 'Наименование'], axis=1)
    
    data2 = data2.drop_duplicates()
    
    k = list(data2.index)

    data = data.query('index in @k')
    
    data = data.reset_index(drop=True)
    
    data2 = None
        

    return data
        
        


        

### Функция обработки WB заказов и остатков в ежедневных выгрузках (+отделение "обезличенных" товаров)

In [41]:
def wb_orders_and_stocks_new_redact(data, flag_SH=flag_SH):
    
    if flag_SH == True:

        for ind in range(len(data)):
            if str(data.loc[ind]['Unnamed: 10']) == 'Санкт-Петербург Шушары':
                data.loc[ind, 'Unnamed: 15'] = 0

    
    data  = data[[data.columns[5]] +
                 [data.columns[4]] +
                 [data.columns[6]] +
                 [data.columns[7]] +
                 [data.columns[0]] +
                 [data.columns[-5]] +
                 [data.columns[-1]]]
    
    data = data.loc[1:]
    
    
    R = []
    for i in range(1,4):
        R.append(re.findall('(\d+)', list(data.columns)[4])[i])
    
    R = R[0] + '-' + R[2] + '-' + R[1]
    R = pd.to_datetime(R)
    R = R.strftime('%d-%B-%Y')
    
    data = data.rename(columns={data.columns[0]: 'Артикул',
                                data.columns[1]: 'Наименование',
                                data.columns[2]: 'Номенклатура',
                                data.columns[3]: 'Баркод',
                                data.columns[4]: 'ТМ',
                                data.columns[5]: R, 
                                data.columns[6]: 'Остаток'})
    
    data = data.reset_index(drop=True)
    
    data['Артикул'] = data['Артикул'].astype(str)
    
    
    for i in range(len(data)):
        data['Артикул'].loc[i] = data.loc[i]['Артикул'].replace('/', '')
        
        
        
    Q = []

    for i in range(len(data)):

        if 'od' in data.loc[i]['Артикул']:
            Q.append(i)
        elif 'ol' in data.loc[i]['Артикул']:
            Q.append(i)
        elif 'ba' in data.loc[i]['Артикул']:
            Q.append(i)
        elif len(data.loc[i]['Артикул']) > 9:
            Q.append(i)
        else:
            None

    data['Баркод'] = data['Баркод'].fillna('none')       
    data['Баркод'] = data['Баркод'].astype(str) 

    for i in range(len(data)):

        if 'non' in data.loc[i]['Баркод']:
            Q.append(i)
        elif '/' in data.loc[i]['Баркод']:
            Q.append(i)
        elif len(data.loc[i]['Баркод']) < 8:
            Q.append(i)
        else:
            None

    Q = list(set(Q))
    
    
    deperson_data = data.query('index in @Q')
    
    deperson_data = deperson_data.reset_index(drop=True)
    
    deperson_data = deperson_data[['Артикул', R, 'Остаток']]
    
    
    deperson_data = deperson_data.groupby('Артикул')[R, 'Остаток'].sum().reset_index(drop=False)
    
#     R = (pd.to_datetime(deperson_data.columns[-2], format='%d-%B-%Y') + datetime.timedelta(days=1)).strftime(format='%d-%B-%Y')
    
#     deperson_data = deperson_data.rename(columns={'Остаток': R+'-Ost'})
    
    data = data.query('index not in @Q')
    
    data = data.reset_index(drop=True)
    
    #data = drop_products(data)
    
    data2 = data.copy()

    data2 = data2[[data2.columns[-4]] + 
                  [data2.columns[-2]] +
                  [data2.columns[-1]]]


    data2 = data2.groupby(data2.columns[0])[
        [data2.columns[1], data2.columns[2]]].sum()

    data2['Баркод'] = list(data2.index)

    data2.reset_index(drop=True, inplace=True)

    data2 = data2[[data2.columns[-1]] + 
            [data2.columns[-3]] +
            [data2.columns[-2]]]

    data2['Баркод'] = data2['Баркод'].astype('int')


    data = data.drop([data.columns[-2], data.columns[-1]], axis=1)
    data['Баркод'] = data['Баркод'].astype(int)

    data  = data.drop_duplicates().reset_index(drop=True)

    data = data.merge(data2, on='Баркод', how='right')

    data2 = None
    
    data['Баркод'] = data['Баркод'].astype(int)
    
    return data, deperson_data
    
    #return data

### Функция добавления "обезличенных" товаров к метрикам по "обезличенным" товарам

In [42]:
def add_deperson_data(data, data3): #data3 - исторические метрики по обезличенным товарам, data - новые
    
    if len(data) > 0:
        
        if 'Остаток' in list(data3.columns):
            
            data3 = data3.drop('Остаток', axis=1)

        data3['Артикул'] = data3['Артикул'].astype(str)
        data['Артикул'] = data['Артикул'].astype(str)

        #R = (pd.to_datetime(deperson_data.columns[-2], format='%d-%B-%Y') + datetime.timedelta(days=1)).strftime(format='%d-%B-%Y')

        if data.columns[-2] not in list(data3.columns):

            data3 = data3.merge(data, on='Артикул', how='outer')
    else:
        None
        
#         data2 = data.copy()

#         data2 = data2[['Артикул']+list(data2.columns[-2:])]

#         data3 = data3.merge(data2, on='Артикул', how='outer')

#         data3 = data3.fillna(0)

#         for i in range(len(data3)):

#             row = data3.loc[i]['Артикул']

#             if data3.loc[i]['Наименование'] ==0:

#                 try:
#                     data3['Наименование'].loc[i] = data[data['Артикул']==row]['Наименование'].values[0]

#                 except:

#                     None


#             if data3.loc[i]['Номенклатура'] ==0:

#                 try:
#                     data3['Номенклатура'].loc[i] = data[data['Артикул']==row]['Номенклатура'].values[0]

#                 except:

#                     None

#             if data3.loc[i]['ТМ'] ==0:

#                 try:
#                     data3['ТМ'].loc[i] = data[data['Артикул']==row]['ТМ'].values[0]

#                 except:

#                     None
                    
#     else:
        
#         None
        
        
    return data3
                


### Функция добавления новых товаров в метрики (если такие имеются в ежедневной выгрузке)

In [43]:
def addition_metric(data, data2):
    
    # Все элементы первого массива, которых нет во втором 

    main_list = np.setdiff1d(list(set(data2['Баркод'])), list(set(data['Баркод'])))
    
    main_list = list(set(main_list))


    if len(main_list) > 0:
        
        dop_prod = data2.copy()
        
        dop_prod = dop_prod.query('Баркод in @main_list')
        
        dop_prod = dop_prod.iloc[:, :-2]
        
        dop_prod = dop_prod.reset_index(drop=True)
            
        data = pd.concat([data, dop_prod], ignore_index=True)
        
        data = data.fillna(0)
        
        for row in list(data.columns[5:]):
        
            data[row] = data[row].astype(int)
        
        dop_prod = None
        
    else:
        
        None
        
    data['Баркод'] = data['Баркод'].astype(int)
    
    return data
        

### Функция сведения "исторических" метрик с "новыми"

In [44]:
def merge_metric_with_new_metric(data, data2):
    
    
    data['Баркод'] = data['Баркод'].astype(int)
    
    data2['Баркод'] = data2['Баркод'].astype(int)
    
    data2 = data2[[data2.columns[-4], data2.columns[-2], data2.columns[-1]]]
    
    if data2.columns[1] not in list(data.columns):
        
        
        data = data.merge(data2, on='Баркод', how='outer')
        
        data = data.fillna(0)
        
        for row in list(data.columns[5:]):
        
            data[row] = data[row].astype(int)
        
        data = data.reset_index(drop=True)
        
        data = data_columns_sorted(data, Ost=True)
        
    else:
        
        None
        
        
    data['Баркод'] = data['Баркод'].astype(int)
    
    return data



### Функция добавления "Остатков" по дню к метрикам с "остатками"

In [45]:
def add_stocks_in_stocks_metric(data, data2): # data - stocks_WB_metric, data2 - WB_orders_metrics
    
    data['Баркод'] = data['Баркод'].astype(int)
    data2['Баркод'] = data2['Баркод'].astype(int)
    
    R = (pd.to_datetime(data2.columns[-2], format='%d-%B-%Y') + datetime.timedelta(days=1)).strftime(format='%d-%B-%Y')
    
    if 'Остаток' in (data2.columns):
    
        if R not in list(data.columns):
            
            main_list = np.setdiff1d(list(set(data2['Баркод'])), list(set(data['Баркод'])))
            
            if len(main_list) > 0:
                
                data3 = data2.copy()
                
                data3 = data3.query('Баркод in @main_list')
            
                data3 = data3.iloc[:, :5]

                data3 = data3.reset_index(drop=True)

                data = pd.concat([data, data3], ignore_index=True)
                
                data = data.fillna(0)
                
                data[list(data.columns[5:])] = data[list(data.columns[5:])].astype(int)
                
                data3 = None
                
            else:
                
                None
                
            data3 = data2.copy()
            
            data3 = data3[['Баркод', 'Остаток']]
            
            data3 = data3.rename(columns={'Остаток': R})
            
            data = data.merge(data3, on='Баркод', how='left')
            
            data = data.fillna(0)
            
            data[list(data.columns[5:])] = data[list(data.columns[5:])].astype(int)
            
            data = data_columns_sorted(data, Ost=False)
            
            data['Баркод'] = data['Баркод'].astype(int)
            
            data3 = None
            
            return data
        
        
        else:
            
            data['Баркод'] = data['Баркод'].astype(int)
                
            return data
            
            
            
    else:
        
        return 'В метриках нет "Остатка"'
    
              

### Функция удаления "Остатка" из метрик

In [46]:
def drop_stocks(data):
    
    if 'Остаток' in list(data.columns):
        
        data = data.drop('Остаток', axis=1)
        
    elif 'остаток' in list(data.columns):
        
        data = data.drop('остаток', axis=1)
        
    else:
        None
        
    return data

### Функция "укорачивания" датасета с метриками

In [47]:
def drop_time_columns(data):
    
    if len(list(data.columns)) > 45:
        
        data = data[list(data.columns[:5]) + list(data.columns[-50:-1]) + list(data.columns[-1:])]
        
    else:
        
        None
        
    return data

### Функция создания "Предмета" из файла с ежедневными ценам

In [48]:
def object_price_create():
    
    object_price = pd.read_excel(path_price)
    
    object_price = object_price[['Предмет', 'Артикул продавца', 'Последний баркод']]
    object_price = object_price.dropna().reset_index(drop=True)
    object_price = object_price.rename(columns={'Артикул продавца': 'Артикул', 'Последний баркод': 'Баркод'})
    object_price = object_price[['Артикул', 'Предмет', 'Баркод']]

    Q = []

    for row in list(object_price['Баркод']):
        try:
            int(row)
            Q.append(row)
        except:
            None

    object_price = object_price.query('Баркод in @Q')
    object_price = object_price.reset_index(drop=True)

    for i in range(len(object_price)):
        object_price['Артикул'].loc[i] = object_price.loc[i]['Артикул'].replace('/', '')

    object_price = object_price.drop_duplicates().reset_index(drop=True)

    object_price['Баркод'] = object_price['Баркод'].astype(int)
    object_price['Артикул'] = object_price['Артикул'].astype(str)
    
    
    return object_price

### Функция добаления признака "Предмет" в метрики

In [49]:
def add_object(data):
    
    data['Баркод'] = data['Баркод'].astype(int)

    object_df = pd.read_csv(path_object_df, sep=';')
    
    object_df['Баркод'] = object_df['Баркод'].astype(int)
    
    object_df = object_df.drop_duplicates().reset_index(drop=True)
    
    main_list = np.setdiff1d(list(set(data['Баркод'])), list(set(object_df['Баркод'])))
    
    Q = []
    
    if len(main_list) > 0:
        
        object_price = object_price_create()


        for row in list(main_list):

            if (data[data['Баркод'] == row]['Артикул']).values[0] in list(object_df['Артикул']):
                
                object_df2 = object_df.copy()
                object_df2 = object_df2[object_df2['Артикул'] == ((data[data['Баркод'] == row]['Артикул']).values[0])]
                object_df2['Баркод'] = row
                object_df = pd.concat([object_df, object_df2], ignore_index=True)
                object_df.to_csv(path_object_df, index=False, sep=";", encoding='utf-8-sig')
                
            else:
                
                if row in list(set(object_price['Баркод'])):
                    
                    object_df = pd.concat([object_df, object_price.loc[object_price[object_price['Баркод']==row].index[0]:object_price[object_price['Баркод']==row].index[0]]], ignore_index=True)
                    object_df['Баркод'] = object_df['Баркод'].astype(int)
                    object_df.to_csv(path_object_df, index=False, sep=";", encoding='utf-8-sig')
                    
                else:
                    
                    Q.append((data[data['Баркод'] == row]['Артикул']).values[0])
                    

    else:
        
        None
        
    if len(Q) == 0:
        
        object_df = object_df[['Баркод', 'Предмет']]
        
        data = data.merge(object_df, on='Баркод', how='left')
        
        data['Баркод'] = data['Баркод'].astype(int)
        
        return data
    
    
    else:
        
        return 'Добавьте в датафрейм "Предмет" следующие артикулы:', Q
        

### Функция добаления признака процент комиссии в метрики

In [50]:
def add_commission_percentage(data):
    
    pc = pd.read_excel(path_commission_percentage)   
    
    pc = pc[['Предмет', 'Склад WB, %']]
    pc.columns = ['Предмет', 'Процент комиссии']
    pc['Предмет'] = pc['Предмет'].astype(str)
    pc['Процент комиссии'] = pc['Процент комиссии'].astype(int)
    pc = pc.drop_duplicates().reset_index(drop=True)
    
    if data['Предмет'].isna().sum() != 0:

        return 'Добавьте в метрики недастоющий признак "предмет" в метрики'

    else:

        main_list = np.setdiff1d(list(set(data['Предмет'])), list(set(pc['Предмет'])))

        if len(main_list) != 0:

            return 'Добавьте в "Процент комиссии" недостающие предметы:', main_list

        else:

            data = data.merge(pc, on='Предмет', how='left')

            data['Баркод'] = data['Баркод'].astype(int)

            return data   

In [51]:
# def add_percent_cost(data):
    
#     percent_cost = pd.read_csv(path_percent_cost, sep=';')
    
    
#     if data['Предмет'].isna().sum() != 0:
        
#         return 'Добавьте в метрики недастоющий признак "предмет" в метрики'
    
#     else:
        
#         main_list = np.setdiff1d(list(set(data['Предмет'])), list(set(percent_cost['Предмет'])))
        
#         if len(main_list) != 0:
            
#             return 'Добавьте в "Проценты и стоимости" недостающие предметы:', main_list
        
#         else:
            
#             data = data.merge(percent_cost, on='Предмет', how='left')
            
#             data['Баркод'] = data['Баркод'].astype(int)
            
#             return data
        

### Функция добавления в "Себестоимость" новых артикулов

In [52]:
def add_art_for_cost_price(art, c_p): # len(art) == len(c_p)
    
    cost_price = pd.read_excel(path_cost_price)
    
    for i in range(len(art)):
        
        cost_price2 = cost_price.copy()
        cost_price2 = cost_price2.loc[0:0]
        cost_price2['Артикул'].loc[0] = art[i]
        cost_price2['Себестоимость'].loc[0] = c_p[i]
        
        cost_price = pd.concat([cost_price, cost_price2], ignore_index=True)
        
    
    cost_price = cost_price.reset_index(drop=True)
    cost_price.to_excel(path_cost_price, index=False) 
    

### Функция добаления признака "Себестоимость"

In [53]:
def add_cost_price(data):
    
    cost_price = pd.read_excel(path_cost_price)
    
    Q = []
    for row in list(set(data['Артикул'])):
        
        if row not in list(set(cost_price['Артикул'])):
            
            Q.append(row)
            
        else:
            None
            
    if len(Q) != 0:
        
        return 'Добавьте в "Себестоимость" недостающие артикулы функцией "add_art_for_cost_price:"', Q
    
    else:
        
        data = data.merge(cost_price, on='Артикул', how='left')
        
    if data.isna().sum().sum() != 0:
        
        return 'В датафрейме "Метрики" есть пропуски!'
    
    else:
        
        data['Баркод'] = data['Баркод'].astype(int)
        
        return data
    

### Функция редактирования "Цены" (изменение баркодов и удаление лишних строк)

In [54]:
def drop_products_for_price(data):

    N = {'50171': 7432141557066,
         '50531': 7432141557217,
         'D0030': 7432141559042,
         'D0031': 7432141559045,
         'D0032': 7432141559048,
         'D0033': 7432141559051,
         'J0400': 0}
    
    for row in N:
        
        if len(data[(data['Артикул']==row) & (data['Баркод']==N[row])]) != 0:
            
            index = data[(data['Артикул']==row) & (data['Баркод']==N[row])].index[0]
                
                
            data = data.drop(index=index)
            
            
    data = data.reset_index(drop=True)
            
            
    K = {'G0260': 7432141556855,
         'G0261': 7432141556860,
         'G0262': 7432141556863,
         'G0270': 7432141556864,
         'G0271': 7432141556867,
         'G0272': 7432141556870,
         'G0273': 7432141556873,
         'G0280': 7432141556876,
         'G0281': 7432141556879,
         'G0282': 7432141556882,
         'G0370': 7432141557627, 
         'G0371': 7432141557630,
         'G0372': 7432141557633,
         'G0373': 7432141557636,
         'G0374': 7432141557639,
         'G0400': 7432141557651,
         'G0401': 7432141557654,
         'G0402': 7432141557657,
         'G0403': 7432141557660,
         'G0404': 7432141557663,
         'G0410': 7432141557666,
         'G0411': 7432141557669,
         'G0412': 7432141557672,
         'G0413': 4683663822330,
         'G0274': 7432141556871,
         'G0275': 7432141556872,
         'G0276': 7432141556874,
         '50301': 7432141557124,
         '50532': 7432141557215,
         '50533': 7432141557220}
    
    
    for i in range(len(data)):
        
        if data.loc[i]['Артикул'] in list(K.keys()):
            
            data['Баркод'].loc[i] = K[data.loc[i]['Артикул']]
            
    data = data.drop_duplicates().reset_index(drop=True)
    
    data['Баркод'] = data['Баркод'].astype(int)

            

    return data
        

### Функция добавления в "ежедневные" цены размерных артикулов

In [55]:
def add_dop_art_to_price(data):
    
    if 'G0270' in list(data['Артикул']):

        if 'G0271' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0271', 
                                         7432141556867, 
                                         data[data['Артикул']=='G0270']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0270']['Скидка'].values[0],
                                         data[data['Артикул']=='G0270']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0271']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0271'].index[0])

                    data.loc[len(data.index)] = ['G0271', 
                                             7432141556867, 
                                             data[data['Артикул']=='G0270']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0270']['Скидка'].values[0],
                                             data[data['Артикул']=='G0270']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
        if 'G0272' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0272', 
                                         7432141556870, 
                                         data[data['Артикул']=='G0270']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0270']['Скидка'].values[0],
                                         data[data['Артикул']=='G0270']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0272']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0272'].index[0])

                    data.loc[len(data.index)] = ['G0272', 
                                             7432141556870, 
                                             data[data['Артикул']=='G0270']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0270']['Скидка'].values[0],
                                             data[data['Артикул']=='G0270']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
        if 'G0273' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0273', 
                                         7432141556873, 
                                         data[data['Артикул']=='G0270']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0270']['Скидка'].values[0],
                                         data[data['Артикул']=='G0270']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0273']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0273'].index[0])

                    data.loc[len(data.index)] = ['G0273', 
                                             7432141556873, 
                                             data[data['Артикул']=='G0270']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0270']['Скидка'].values[0],
                                             data[data['Артикул']=='G0270']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
    else:
        
        None
                
                
                
                
                
    if 'G0282' in list(data['Артикул']):
        
        
        if 'G0280' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0280', 
                                         7432141556876, 
                                         data[data['Артикул']=='G0282']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0282']['Скидка'].values[0],
                                         data[data['Артикул']=='G0282']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0280']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0280'].index[0])

                    data.loc[len(data.index)] = ['G0280', 
                                             7432141556876, 
                                             data[data['Артикул']=='G0282']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0282']['Скидка'].values[0],
                                             data[data['Артикул']=='G0282']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
        if 'G0281' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0281', 
                                         7432141556879, 
                                         data[data['Артикул']=='G0282']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0282']['Скидка'].values[0],
                                         data[data['Артикул']=='G0282']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0281']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0281'].index[0])

                    data.loc[len(data.index)] = ['G0281', 
                                             7432141556879, 
                                             data[data['Артикул']=='G0282']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0282']['Скидка'].values[0],
                                             data[data['Артикул']=='G0282']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
    else:
        
        None
                
                
                
                
                
    if 'G0370' in list(data['Артикул']):
        
        
        
        if 'G0371' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0371', 
                                         7432141557630, 
                                         data[data['Артикул']=='G0370']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0370']['Скидка'].values[0],
                                         data[data['Артикул']=='G0370']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0371']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0371'].index[0])

                    data.loc[len(data.index)] = ['G0371', 
                                             7432141557630, 
                                             data[data['Артикул']=='G0370']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0370']['Скидка'].values[0],
                                             data[data['Артикул']=='G0370']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
                
        if 'G0372' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0372', 
                                         7432141557633, 
                                         data[data['Артикул']=='G0370']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0370']['Скидка'].values[0],
                                         data[data['Артикул']=='G0370']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0372']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0372'].index[0])

                    data.loc[len(data.index)] = ['G0372', 
                                             7432141557633, 
                                             data[data['Артикул']=='G0370']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0370']['Скидка'].values[0],
                                             data[data['Артикул']=='G0370']['Цена'].values[0]]

                else:

                    None

            except:

                None

                
        if 'G0373' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0373', 
                                         7432141557636, 
                                         data[data['Артикул']=='G0370']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0370']['Скидка'].values[0],
                                         data[data['Артикул']=='G0370']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0373']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0373'].index[0])

                    data.loc[len(data.index)] = ['G0373', 
                                             7432141557636, 
                                             data[data['Артикул']=='G0370']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0370']['Скидка'].values[0],
                                             data[data['Артикул']=='G0370']['Цена'].values[0]]

                else:

                    None

            except:

                None



        if 'G0374' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0374', 
                                         7432141557639, 
                                         data[data['Артикул']=='G0370']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0370']['Скидка'].values[0],
                                         data[data['Артикул']=='G0370']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0374']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0374'].index[0])

                    data.loc[len(data.index)] = ['G0374', 
                                             7432141557639, 
                                             data[data['Артикул']=='G0370']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0370']['Скидка'].values[0],
                                             data[data['Артикул']=='G0370']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
    else:
        
        None
        
        
        



    if 'G0400' in list(data['Артикул']):
        
        
        if 'G0401' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0401', 
                                         7432141557654, 
                                         data[data['Артикул']=='G0400']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0400']['Скидка'].values[0],
                                         data[data['Артикул']=='G0400']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0401']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0401'].index[0])

                    data.loc[len(data.index)] = ['G0401', 
                                             7432141557654, 
                                             data[data['Артикул']=='G0400']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0400']['Скидка'].values[0],
                                             data[data['Артикул']=='G0400']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
                
                
        if 'G0402' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0402', 
                                         7432141557657, 
                                         data[data['Артикул']=='G0400']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0400']['Скидка'].values[0],
                                         data[data['Артикул']=='G0400']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0402']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0402'].index[0])

                    data.loc[len(data.index)] = ['G0402', 
                                             7432141557657, 
                                             data[data['Артикул']=='G0400']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0400']['Скидка'].values[0],
                                             data[data['Артикул']=='G0400']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
        if 'G0403' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0403', 
                                         7432141557660, 
                                         data[data['Артикул']=='G0400']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0400']['Скидка'].values[0],
                                         data[data['Артикул']=='G0400']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0403']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0403'].index[0])

                    data.loc[len(data.index)] = ['G0403', 
                                             7432141557660, 
                                             data[data['Артикул']=='G0400']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0400']['Скидка'].values[0],
                                             data[data['Артикул']=='G0400']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
                
        if 'G0404' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0404', 
                                         7432141557663, 
                                         data[data['Артикул']=='G0400']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0400']['Скидка'].values[0],
                                         data[data['Артикул']=='G0400']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0404']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0404'].index[0])

                    data.loc[len(data.index)] = ['G0404', 
                                             7432141557663, 
                                             data[data['Артикул']=='G0400']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0400']['Скидка'].values[0],
                                             data[data['Артикул']=='G0400']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
    else:
        
        None
        
        
        
        
        
        
        
    if 'G0410' in list(data['Артикул']):     
        
        if 'G0411' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0411', 
                                         7432141557669, 
                                         data[data['Артикул']=='G0410']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0410']['Скидка'].values[0],
                                         data[data['Артикул']=='G0410']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0411']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0411'].index[0])

                    data.loc[len(data.index)] = ['G0411', 
                                             7432141557669, 
                                             data[data['Артикул']=='G0410']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0410']['Скидка'].values[0],
                                             data[data['Артикул']=='G0410']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
        if 'G0412' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0412', 
                                         7432141557672, 
                                         data[data['Артикул']=='G0410']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0410']['Скидка'].values[0],
                                         data[data['Артикул']=='G0410']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0412']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0412'].index[0])

                    data.loc[len(data.index)] = ['G0412', 
                                             7432141557672, 
                                             data[data['Артикул']=='G0410']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0410']['Скидка'].values[0],
                                             data[data['Артикул']=='G0410']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
                
        if 'G0413' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0413', 
                                         4683663822330, 
                                         data[data['Артикул']=='G0410']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0410']['Скидка'].values[0],
                                         data[data['Артикул']=='G0410']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0413']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0413'].index[0])

                    data.loc[len(data.index)] = ['G0413', 
                                             4683663822330, 
                                             data[data['Артикул']=='G0410']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0410']['Скидка'].values[0],
                                             data[data['Артикул']=='G0410']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
                
                
                
                
        if 'G0275' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0275', 
                                         7432141556872, 
                                         data[data['Артикул']=='G0274']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0274']['Скидка'].values[0],
                                         data[data['Артикул']=='G0274']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0275']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0275'].index[0])

                    data.loc[len(data.index)] = ['G0275', 
                                             7432141556872, 
                                             data[data['Артикул']=='G0274']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0274']['Скидка'].values[0],
                                             data[data['Артикул']=='G0274']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
                
                
                
                
                
        if 'G0276' not in list(data['Артикул']):

            data.loc[len(data.index)] = ['G0276', 
                                         7432141556874, 
                                         data[data['Артикул']=='G0274']['Цена до скидки'].values[0],
                                         data[data['Артикул']=='G0274']['Скидка'].values[0],
                                         data[data['Артикул']=='G0274']['Цена'].values[0]]
        else:

            try:

                if data[data['Артикул']=='G0276']['Цена'].values[0] == 0:

                    data = data.drop(index=data[data['Артикул']=='G0276'].index[0])

                    data.loc[len(data.index)] = ['G0276', 
                                             7432141556874, 
                                             data[data['Артикул']=='G0274']['Цена до скидки'].values[0],
                                             data[data['Артикул']=='G0274']['Скидка'].values[0],
                                             data[data['Артикул']=='G0274']['Цена'].values[0]]

                else:

                    None

            except:

                None
                
                
                
                
                
                
                
                
                
                
                
                
                
                
                
#         if 'G0283' not in list(data['Артикул']):

#             data.loc[len(data.index)] = ['G0283', 
#                                          4648894810020, 
#                                          data[data['Артикул']=='G0284']['Цена до скидки'].values[0],
#                                          data[data['Артикул']=='G0284']['Скидка'].values[0],
#                                          data[data['Артикул']=='G0284']['Цена'].values[0]]
#         else:

#             try:

#                 if data[data['Артикул']=='G0283']['Цена'].values[0] == 0:

#                     data = data.drop(index=data[data['Артикул']=='G0283'].index[0])

#                     data.loc[len(data.index)] = ['G0283', 
#                                              4648894810020, 
#                                              data[data['Артикул']=='G0284']['Цена до скидки'].values[0],
#                                              data[data['Артикул']=='G0284']['Скидка'].values[0],
#                                              data[data['Артикул']=='G0284']['Цена'].values[0]]

#                 else:

#                     None

#             except:

#                 None
                
                
#         if 'G0285' not in list(data['Артикул']):

#             data.loc[len(data.index)] = ['G0285', 
#                                          4648877486747, 
#                                          data[data['Артикул']=='G0284']['Цена до скидки'].values[0],
#                                          data[data['Артикул']=='G0284']['Скидка'].values[0],
#                                          data[data['Артикул']=='G0284']['Цена'].values[0]]
#         else:

#             try:

#                 if data[data['Артикул']=='G0285']['Цена'].values[0] == 0:

#                     data = data.drop(index=data[data['Артикул']=='G0285'].index[0])

#                     data.loc[len(data.index)] = ['G0285', 
#                                              4648877486747, 
#                                              data[data['Артикул']=='G0284']['Цена до скидки'].values[0],
#                                              data[data['Артикул']=='G0284']['Скидка'].values[0],
#                                              data[data['Артикул']=='G0284']['Цена'].values[0]]

#                 else:

#                     None

#             except:

#                 None
                
                
                



                
                
                
                
#         if '50533' not in list(data['Артикул']):

#             data.loc[len(data.index)] = ['50533', 
#                                          7432141557215, 
#                                          data[data['Артикул']=='50532']['Цена до скидки'].values[0],
#                                          data[data['Артикул']=='50532']['Скидка'].values[0],
#                                          data[data['Артикул']=='50532']['Цена'].values[0]]
#         else:

#             try:

#                 if data[data['Артикул']=='50533']['Цена'].values[0] == 0:

#                     data = data.drop(index=data[data['Артикул']=='50533'].index[0])

#                     data.loc[len(data.index)] = ['50533', 
#                                              7432141557215, 
#                                              data[data['Артикул']=='50532']['Цена до скидки'].values[0],
#                                              data[data['Артикул']=='50532']['Скидка'].values[0],
#                                              data[data['Артикул']=='50532']['Цена'].values[0]]

#                 else:

#                     None

#             except:

#                 None 
                
                
                
                
                
                
                
    else:
        
        None
                
            
            
    data = data.drop_duplicates().reset_index(drop=True)
    
    data['Баркод'] = data['Баркод'].astype(int)
                
    return data
                
 

### Функция обработки и добавления цены (выгружаемая ежедневно)

In [56]:
def add_price(data, price):
    
    data['Баркод'] = data['Баркод'].astype(int)

    #price = pd.read_excel(path_price)
    
    price = price[['Артикул продавца', 'Последний баркод', 'Текущая розн. цена (до скидки)', 'Текущая скидка на сайте, %']]
    price = price.rename(columns={'Артикул продавца': 'Артикул', 'Последний баркод': 'Баркод', 'Текущая розн. цена (до скидки)': 'Цена до скидки', 'Текущая скидка на сайте, %': 'Скидка'})
    price = price.fillna('xxx')
    price = price[price['Баркод'] !='xxx']
    price['Баркод'] = price['Баркод'].astype('str')
    U=[]
    for row in price['Баркод']:
        if '/' in row:
            U.append(row)
    price = price.query('Баркод not in @U')
    price['Баркод'] = price['Баркод'].astype('int')
    price = price.reset_index(drop=True)
    price['Цена'] = (price['Цена до скидки'] / 100) * (100 - price['Скидка'])
    for i in range(len(price)):
        if price['Скидка'][i] == 0:
            price['Цена'][i] = price['Цена до скидки'][i]
    price['Цена'] = price['Цена'].astype('int')
    
    price['Артикул'] = list(map(lambda x: x.replace('/', ''), price['Артикул']))
    
    price = drop_products_for_price(price)
    
    price = add_dop_art_to_price(price)
    
    price = price.drop_duplicates().reset_index(drop=True)
    
    price = price.drop('Артикул', axis=1)
    
    
    data = data.merge(price, on='Баркод', how='left')
    
    data['Цена до скидки'] = data['Цена до скидки'].fillna(0)
    data['Скидка'] = data['Скидка'].fillna(0)
    data['Цена'] = data['Цена'].fillna(0)
    
    data['Баркод'] = data['Баркод'].astype(int)
    
    
    
    return data

### Функция создания вычисляемых признаков

In [57]:
def creating_calculated_features(data):
    
    data['Комиссия в рублях'] = (data['Цена'] / 100) * (data['Процент комиссии'] + 3)
    
    data['Эквайринг'] = data['Цена'] / 100 * 0.75
    
    data['НДС'] =  ((data['Цена'] * 0.2 / 1.2) - (data['Себестоимость'] * 0.2 / 1.2) - 
                    (data['Комиссия в рублях'] * 0.2 / 1.2) - 
                    (data['Стоимость логистики'] * 0.2 / 1.2))


    R = ((data['Цена'] - (data['Цена'] * 0.2 / 1.2)) - 
         (data['Себестоимость'] - (data['Себестоимость'] * 0.2 / 1.2)) - 
         (data['Комиссия в рублях'] - (data['Комиссия в рублях'] * 0.2 / 1.2)) - 
         (data['Стоимость логистики'] - (data['Стоимость логистики'] * 0.2 / 1.2)) - (data['Эквайринг']))
    
    
    data['Налог на прибыль'] = R*0.2
    
    data['Налог'] = data['НДС'] + data['Налог на прибыль']
    
    data['Прибыль'] = (data['Цена'] - 
                       data['Себестоимость'] - 
                       data['Комиссия в рублях'] - 
                       data['Стоимость логистики'] - 
                       data['Налог'] -
                       data['Эквайринг'])
    
    
    data['Наценка'] = (data['Прибыль'] / data['Себестоимость']) * 100
    
    data['Процент налога'] = (data['Налог'] * 100) / data['Цена']
    
    
    
    for i in range(len(data)):
        
        if data['Цена'][i] <= 0:
            data['Налог на прибыль'][i] = 0
            data['Прибыль'][i] = 0
            data['Процент налога'][i] = 0
            data['Процент налога'][i] = 0
        
        
        
    data['Наценка'] = data['Наценка'].round(2)
    
    data = data.drop('Эквайринг', axis=1)
    
    return data



    
    

### Функция для удаления ненужных столбцев для презентации

In [58]:
def drop_columns_for_presentation_df(data):
    
    drop_col =  ['Предмет',
                 'Процент комиссии',
                 'Стоимость логистики',
                 'Стоимость хранения',
                 'Цена до скидки',
                 'Скидка',
                 'Себестоимость',
                 'Комиссия в рублях',
                 'НДС',
                 'Налог на прибыль',
                 'Налог',
                 'Прибыль',
                 'Процент налога']
    
    data = data.drop(drop_col, axis=1)
    
    return data

### Функция добавления РЦ

In [59]:
def add_rc(data):
    
    rc = pd.read_excel(path_rc)
    
    if len(rc) - len(rc['Артикул'].unique()) != 0:
        
        return 'В РЦ есть неуникальные артикулы!'
    
    else:
        
        data = data.merge(rc, on='Артикул', how='left')
        
        data = data.fillna(0)
        
        data = data.reset_index(drop=True)
        
        data['Баркод'] = data['Баркод'].astype(int)
        
        return data

### Функция добавления признака "Статус"

In [60]:
# def add_status(data):
    
#     st = pd.read_excel(path_st)
    
#     st = st.reset_index(drop=True)
    
#     data = data.reset_index(drop=True)
    
    
#     if len(st) - len(st['Артикул'].unique()) != 0:
        
#         return 'В "статусах" есть неуникальные артикулы!'
    
#     else:
        
#         Q = []
        
#         for i in range(len(data)):
        
#             row = data.loc[i]['Артикул']
        
#             if row in list(set(st['Артикул'])):
                
#                 Q.append(st[st['Артикул']==row]['Статус'].values[0])
                
#             else:
                
#                 Q.append('статус не определен')
                
        
#         data.insert(5, 'Статус', Q)
        
#         data = data.reset_index(drop=True)
        
#         data['Баркод'] = data['Баркод'].astype(int)
        
#         return data
        

In [61]:
# def add_status(data):
    
#     rc_irc = pd.read_excel(path_downloads + 'РЦ Иркутск.xlsx', 
#                     dtype={'Артикул': 'str'})
    
#     if 'Статус' in list(data.columns):
        
#         data = data.drop('Статус', axis=1)
        
#     else:
        
#         None
    
#     st = pd.read_excel(path_st)
    
#     st = st.reset_index(drop=True)
    
#     data = data.reset_index(drop=True)
    
    
    
#     if len(st) - len(st['Артикул'].unique()) != 0:
        
#         return 'В "статусах" есть неуникальные артикулы!'
    
#     else:
        
#         Q = []
        
#         for i in range(len(data)):
        
#             row = data.loc[i]['Артикул']
        
#             if row in list(set(st['Артикул'])):
                
#                 Q.append(st[st['Артикул']==row]['Статус'].values[0])
                
#             else:
                
#                 Q.append('статус не определен')
                
        
#         data.insert(5, 'Статус', Q)
        
#         data = data.reset_index(drop=True)
        
#         data['Баркод'] = data['Баркод'].astype(int)
        
        
        
#     st_oz = pd.read_excel(path_algoritm + 'OZON_ORDERS/OZON_ORDERS_NEW_ALGORITM/Статусы/Статусы.xlsx')  
#     st_oz['Артикул'] = st_oz['Артикул'].astype(str)
    
    
    
#     c = 0
#     W = []
#     S = []
#     for i in range(len(data)):
#         if data.loc[i]['Статус'] == 'статус не определен':
#             if data.loc[i]['Артикул'] in list(set(st_oz['Артикул'])):
#                 c +=1
#                 if data.loc[i]['Артикул'] in list(set(st['Артикул'])):
#                     st = st[st['Артикул']!=data.loc[i]['Артикул']].reset_index(drop=True)
#                     st.loc[len(st)] = [data.loc[i]['Артикул'], 'новинка Даша']
#                 else:
#                     st.loc[len(st)] = [data.loc[i]['Артикул'], 'новинка Даша']
                    
#         elif data.loc[i]['Статус'] == 'товары в пути':
#             if data.loc[i]['Остаток на РЦ'] > 0:
#                 data.loc[i, 'Статус'] = 'новинка Даша'
#                 W.append(data.loc[i]['Артикул'])
#             elif data.loc[i]['Артикул'] in list(set(rc_irc['Артикул'])):
#                 if rc_irc[
#                     ['Артикул']==data.loc[i]['Артикул']]['Остаток РЦ Иркутск'].values[0] > 0:
#                     data.loc[i, 'Статус'] = 'новинка Даша'
#                     W.append(data.loc[i]['Артикул'])                    
#         elif data.loc[i]['Статус'] == 'новинка Даша':
#             if len([x for x in list(WB_orders_metrics_for_status.loc[i][dt_col(WB_orders_metrics_for_status)]) if x > 0]) >= 30:
#                 data.loc[i, 'Статус'] = 'текущий Дарья'
#                 S.append(data.loc[i]['Артикул'])
#         else:
#             None
#     W = list(set(W))
#     S = list(set(S))
    
    
#     if len(S) > 0:
#         for row in S:
#             st[st['Артикул']==row].index[0]
#             st.loc[st[st['Артикул']==row].index[0], 'Статус'] = 'текущий Дарья'
#         if len(st) - len(list(set(st['Артикул']))) != 0:
#             print('error')
#             int('error')
#         else:
#             None
            
            
#     if len(W) > 0:
#         for row in W:
#             st[st['Артикул']==row].index[0]
#             st.loc[st[st['Артикул']==row].index[0], 'Статус'] = 'новинка Даша'
#         if len(st) - len(list(set(st['Артикул']))) != 0:
#             print('error')
#             int('error')
#         else:
#             None
        
                    
#     if c != 0 or len(W) > 0 or len(S) > 0:
#         st.to_excel(path_st, index=False)
        
        
#         if 'Статус' in list(data.columns):

#             data = data.drop('Статус', axis=1)

#         else:

#             None

#         st = pd.read_excel(path_st)

#         st = st.reset_index(drop=True)

#         data = data.reset_index(drop=True)


#         if len(st) - len(st['Артикул'].unique()) != 0:

#             return 'В "статусах" есть неуникальные артикулы!'

#         else:

#             Q = []

#             for i in range(len(data)):

#                 row = data.loc[i]['Артикул']

#                 if row in list(set(st['Артикул'])):

#                     Q.append(st[st['Артикул']==row]['Статус'].values[0])

#                 else:

#                     Q.append('статус не определен')


#             data.insert(5, 'Статус', Q)

#             data = data.reset_index(drop=True)

#             data['Баркод'] = data['Баркод'].astype(int)
    
                
                
#     return data
        

### Функция добавления промежуточных расчетов суммы заказов (Сумма за неделю)

In [62]:
def weekly_amount(data):
    
    data1 = data.copy()

    z = list(data.iloc[:, 6:-4].columns)

    for i in range(len(z)):
        if pd.to_datetime(z[i]).dayofweek == 6 and i > 6:
            data1.insert((data1.columns.get_loc(z[i]))+1, f"{pd.to_datetime(z[i-6]):%d-%B}-{pd.to_datetime(z[i]):%d-%B}", 0)



    data = data1

    R = []
    for i in range(len(list(data.columns))):
        try:
            if len((re.findall('(\d+)',list(data.columns)[i]))[1]) != 4:
                R.append(list(data.columns)[i])

        except:
            None

    for row in R:
        T = []
        for i in range(len(data)):
            T.append((data.iloc[i, data.columns.get_loc(row)-7:data.columns.get_loc(row)]).sum())

        data[row] = T
        
    
    data['Баркод'] = data['Баркод'].astype(int)
        
    return data

### Функция добавления "Итого"

In [63]:
def add_total(data):
    
    if data.loc[len(data)-1]['Артикул'] != 'Итого':
    
        data = data.sort_values(by='Артикул')

        data = data.reset_index(drop=True)

        data.insert(len(data.columns)-4, '     ', '     ')

        data2 = data.copy()

        data2 = data2.loc[0:0]

        data2['Артикул'].loc[0] = 'Итого'
        data2['Наименование'].loc[0] = None
        data2['Номенклатура'].loc[0] = None
        data2['Баркод'].loc[0] = None
        data2['ТМ'].loc[0] = None
        data2['Статус'].loc[0] = None

        for row in list(data2.columns[6:-5]):

            data2[row].loc[0] = sum(data[row])

        data2['     '].loc[0] = '     '
        data2['Остаток'].loc[0] = sum(data['Остаток'])
        data2['Цена'].loc[0] = None
        data2['Наценка'].loc[0] = None
        data2['Остаток на РЦ'].loc[0] = sum(data['Остаток на РЦ'])

        data = pd.concat([data, data2], ignore_index=True)

        return data
    
    else:
        
        None

### Функция добавления "Цены" в метрики по ценам

In [64]:
def add_price_in_WB_price_metric(data, data2):      # data - WB_price_metric,
                                                    # data2 - WB_full_features_metric
        
        
    data['Баркод'] = data['Баркод'].astype(int)
    
    data2['Баркод'] = data2['Баркод'].astype(int)
    
    R = (pd.to_datetime(data2.columns[-17], format='%d-%B-%Y') + datetime.timedelta(days=1)).strftime(format='%d-%B-%Y')
    
    if R not in list(data.columns):
            
        main_list = np.setdiff1d(list(set(data2['Баркод'])), list(set(data['Баркод'])))
        
        if len(main_list) > 0:
            
            data3 = data2.copy()
            
            data3 = data3.query('Баркод in @main_list')
            
            data3 = data3.reset_index(drop=True)
            
            data3 = data3.iloc[:, :5]
            
            data = pd.concat([data, data3], ignore_index=True)
            
            data = data.fillna(0)
            
            data[list(data.columns[5:])] = data[list(data.columns[5:])].astype(int)
            
            data3 = None
            
        else:
            
            None
            
        data3 = data2.copy()
        
        data3 = data3[['Баркод', 'Цена']]
        
        data3 = data3.rename(columns={'Цена': R})
        
        data = data.merge(data3, on='Баркод', how='left')
        
        data = data.fillna(0)

        data[list(data.columns[5:])] = data[list(data.columns[5:])].astype(int)
        
        data3 = None
        
        return data
        
        
    else:
        
        return data
            
            

### Функция редактирования "Объема"

In [65]:
def redact_wb_size(data):
    
    data2 = data.copy()
    
    
    data2 = data2[['Артикул продавца', 'Артикул WB', 'Баркод', 'Предмет', 'Бренд']]

    data2 = data2.rename(columns={'Артикул продавца': 'Артикул', 'Артикул WB': 'Номенклатура'})

    Q = []

    for i in range(len(data2)):

        try:

            int(data2.loc[i]['Баркод'])

            Q.append(i)

        except:

            None

    data2 = data2.query('index in @Q').reset_index(drop=True)
    data2['Баркод'] = data2['Баркод'].astype(int)
    
    
    data2['Артикул'] = list(map(lambda x: x.replace('/', ''), data2['Артикул']))

    data2['Артикул'] = data2['Артикул'].astype(str)
    data2['Номенклатура'] = data2['Номенклатура'].astype(int)
    data2['Предмет'] = data2['Предмет'].astype(str)
    data2['Бренд'] = data2['Бренд'].astype(str)
    
    

    
    data = data[['Баркод', 'Объем, л']]
    data = data.dropna().reset_index(drop=True)

    Q = []

    for i in range(len(data)):

        try:

            int(data.loc[i]['Баркод'])
            float(data.loc[i]['Объем, л'])

            Q.append(i)

        except:

            None

    data = data.query('index in @Q').reset_index(drop=True)
    data['Баркод'] = data['Баркод'].astype(int)
    data['Объем, л'] = data['Объем, л'].astype(float)
    
    return data, data2

### Функция редактирования метрик по "объемам", добавление новых дней

In [66]:
def redact_WB_size_metrics(data, data2, data3):       # data -  WB_size_for_metric, data2 - WB_size_day
                                                      # data3 - метрики
    


    WB_size_metrics = pd.read_excel(path_WB_size_metrics)

    if 'Diff' in list(WB_size_metrics.columns):

        WB_size_metrics = WB_size_metrics.drop('Diff', axis=1)

    else:

        None


    data = data.rename(columns={data.columns[-1]: pd.to_datetime(datetime.datetime.now()).strftime('%d-%B-%Y')})



    if data.columns[-1] not in list(WB_size_metrics.columns):


        WB_size_metrics = WB_size_metrics.merge(data, on='Баркод', how='outer')

    else:

        None

    D = WB_size_metrics[WB_size_metrics.columns[-1]] - WB_size_metrics[WB_size_metrics.columns[-2]]

    WB_size_metrics['Diff'] = D

    for i in range(len(WB_size_metrics)):

        row = WB_size_metrics.loc[i]['Баркод']
        
        if row in list(set(data2['Баркод'])):

            for col in ['Артикул', 'Номенклатура', 'Баркод', 'Предмет', 'Бренд']:


                if WB_size_metrics.loc[i:i][col].isna().values[0] == True:

                    WB_size_metrics[col].loc[i] = data2[data2['Баркод']==row][col].values[0]

                else:

                    None
                    
        else:
            
            None
            
            
        if row in list(set(data3['Баркод'])):
            
            if WB_size_metrics.loc[i:i]['Наименование'].isna().values[0] == True:
                
                WB_size_metrics['Наименование'].loc[i] = data3[data3['Баркод']==row]['Наименование'].values[0]
                
            else:
                
                None
        
        else:
            
            None
               
    


    return WB_size_metrics

### Функция редактирования "Бренда"

In [67]:
def add_and_redact_brend(data):

    def xxx_brend(y):
        try:
            int(y) 
            y = y
        except:
            y = 'None'
        return y
    
    def yyy_brend(y):
        try:
            if '0' in y:
                y = 'no_name'
            elif '\xa0' in y:
                y = y.replace('\xa0', '')
            elif '/' in y:
                y = y.replace('/', '')
            elif 'Limpid Side' in y:
                y = 'Limpide Side'
            elif 'REVOLUY' in y:
                y = 'REVOLUT'
            elif 'DENT & DONT' in y:
                y = 'DENT & DON’T'
            elif 'Revolut' in y:
                y = 'REVOLUT'
            else:
                None
        except:
            None
            
        return y
    
    
    dfb = pd.read_excel(path_WB_size)
    dfb = dfb[['Баркод', 'Бренд']]
    dfb = dfb.query(f'Баркод in {list(filter(lambda x:  "None" != xxx_brend(x), list(set(dfb["Баркод"]))))}').reset_index(drop=True)
    dfb['Баркод'] = dfb['Баркод'].astype(int)
    dfb['Бренд'] = dfb['Бренд'].astype(str)
    dfb = dfb.rename(columns={'Бренд': 'ТМ'})
    dfb = dfb.dropna().reset_index(drop=True)
    dfb['ТМ'] = list(map(lambda x: yyy_brend(x), list(dfb['ТМ'])))
    dfb['ТМ'] = list(map(lambda x: yyy_brend(x), list(dfb['ТМ'])))

    
    flag = 0
    if 'Бренд' in list(data.columns):
        flag = 1
        data = data.rename(columns={'Бренд': 'ТМ'})
    else:
        None
    

    data['ТМ'] = data['ТМ'].fillna('no_name')    
    data['ТМ'] = list(map(lambda x: yyy_brend(x), list(data['ТМ'])))
    data['ТМ'] = list(map(lambda x: yyy_brend(x), list(data['ТМ'])))

    for i in range(len(data)):
        if data.loc[i]['ТМ'] == 'no_name':
            try:
                data['Баркод'].loc[i] = int(data.loc[i]['Баркод'])
                if data.loc[i]['Баркод'] in list(set(dfb['Баркод'])):
                    data['ТМ'].loc[i] = dfb[dfb['Баркод']==data.loc[i]['Баркод']]['ТМ'].values[0]
                else:
                    None
            except:
                data['ТМ'].loc[i] = None    
    
    
    if flag == 1:
        data = data.rename(columns={'ТМ': 'Бренд'})
    else:
        None
        
    return data

In [68]:
# def add_commission_percentage(data):
    
#     pc = pd.read_excel(path_commission_percentage)   
    
#     pc = pc[['Предмет', 'Склад WB, %']]
#     pc.columns = ['Предмет', 'Процент комиссии']
#     pc['Предмет'] = pc['Предмет'].astype(str)
#     pc['Процент комиссии'] = pc['Процент комиссии'].astype(int)
#     pc = pc.drop_duplicates().reset_index(drop=True)
    
#     if data['Предмет'].isna().sum() != 0:

#         return 'Добавьте в метрики недастоющий признак "предмет" в метрики'

#     else:

#         main_list = np.setdiff1d(list(set(data['Предмет'])), list(set(pc['Предмет'])))

#         if len(main_list) != 0:

#             return 'Добавьте в "Проценты и стоимости" недостающие предметы:', main_list

#         else:

#             data = data.merge(pc, on='Предмет', how='left')

#             data['Баркод'] = data['Баркод'].astype(int)

#             return data   

## Работа с WB метриками

### Замена стутусов

In [69]:
# df = pd.read_excel(path_corr_st, sheet_name='вб')
# add_st_new(df)
# df = None

### Загрузка основных "исторических" метрик

In [70]:
WB_orders_metrics = pd.read_csv(path_WB_orders_metrics, sep=";")

### Загрузка "Цены" (из "Загрузок")

In [71]:
price = pd.read_excel(path_price)

### Проверка "Цены" на наличие баркодов, замена в случае ошибки

In [72]:
dop_price2 = pd.read_excel(path_downloads + 'Для цена на вб.xlsx')

In [73]:
dpp = list(filter(lambda x: x not in list(set(dop_price2['Артикул WB'])), list(set(price['Артикул WB']))))
dpp2 = pd.read_excel(path_WB_size)
dpp2 = dpp2[['Артикул WB', 'Баркод']]
dpp2.columns = ['Артикул_WB', 'Баркод']
dpp2 = dpp2.query('Артикул_WB in @dpp').reset_index(drop=True)

In [74]:
if len(dpp2) - len(list(set(dpp2['Артикул_WB']))) == 0:
    for i in range(len(dpp2)):
        dop_price2.loc[len(dop_price2)] = list(dpp2.loc[i].values)
    dop_price2.to_excel(path_downloads + 'Для цена на вб.xlsx', index=False)
else:
    None

In [75]:
if len(price) - price['Последний баркод'].isna().sum() < 50:
    dop_price2 = pd.read_excel(path_downloads + 'Для цена на вб.xlsx')
    price = price.drop('Последний баркод', axis=1)
    price = price.merge(dop_price2, on='Артикул WB', how='inner') 
else:
    None

### Загрузка метрик по "обезличенным" товарам

In [76]:
WB_depersonalized_metric = pd.read_csv(path_WB_depersonalized_metric, sep=';')
# WB_depersonalized_metric = WB_depersonalized_metric.drop_duplicates().reset_index(drop=True)

In [77]:
WB_depersonalized_metric

,Артикул,07-February-2023,08-February-2023,09-February-2023,10-February-2023,11-February-2023,12-February-2023,13-February-2023,14-February-2023,15-February-2023,...,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток
0,27701ba,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"2,86091E+11",1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29720-слоновая кость,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28800Mod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,286091234541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30000-сиреневый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0
6,Неопознанный товар,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0


In [78]:
path_WB_depersonalized_metric

'/Users/dmitriymaslov/Мой диск/Алгоритмы Маслов/WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Обезличенные товары/WB_depersonalized_metric.csv'

### Загрузка заказов и остатков по дню, сведение с метриками

In [79]:
for path in paths:
    
    
    WB_orders_metrics = drop_stocks(WB_orders_metrics) # Удаляем "Остатки", если они есть 
   
    WB_orders_metrics = drop_products(WB_orders_metrics) # Меняем значения для "неуникальных" артикулов
     
    WB_new_metric = pd.read_excel(path) # Загружаем заказы и остатки по дню 
    
#     if flag_SH == True:

#         for ind in range(len(WB_new_metric)):
#             if str(WB_new_metric.loc[ind]['Unnamed: 10']) == 'Санкт-Петербург Шушары':
#                 WB_new_metric.loc[ind, 'Unnamed: 16'] = 0
    
    WB_new_metric, deperson_data = wb_orders_and_stocks_new_redact(WB_new_metric) # Отделяем "обезличенные" товары
    
    
#     deperson_data2 = deperson_data.iloc[:, :5].copy()
#     deperson_data2 = deperson_data2.drop_duplicates().reset_index(drop=True)
#     deperson_data = deperson_data.groupby('Артикул')[list(deperson_data.columns[5:])].sum().reset_index(drop=False)
#     deperson_data = deperson_data2.merge(deperson_data, on='Артикул', how='inner')
#     display(deperson_data)

    
#     if WB_new_metric.columns[-2]+'-Ost' in list(WB_depersonalized_metric.columns):

#         WB_depersonalized_metric = WB_depersonalized_metric.drop(WB_new_metric.columns[-2]+'-Ost', axis=1)

#     WB_depersonalized_metric = WB_depersonalized_metric.drop_duplicates().reset_index(drop=True)

#     deperson_data = deperson_data.drop_duplicates().reset_index(drop=True)



    WB_depersonalized_metric = add_deperson_data(deperson_data, WB_depersonalized_metric) # Добавляем "обезличенные" товары
                                                                                          # к метрикам по "обезличенным" товарам
    display(WB_depersonalized_metric)
    
    WB_new_metric = drop_products(WB_new_metric) # Меняем значения для "неуникальных" артикулов в загруженных товарах по дню
    
    WB_orders_metrics = addition_metric(WB_orders_metrics, WB_new_metric) # Добавляем новые товары в метрики из загрузок, если такие появлись
    
    WB_orders_metrics = merge_metric_with_new_metric(WB_orders_metrics, WB_new_metric) # Сводим метрки с заказами и остатками по дню
    
    WB_orders_metrics = data_columns_sorted(WB_orders_metrics, Ost=True) # Сортировка столбцов дат по времени
    
    if 'Остаток' not in (WB_orders_metrics.columns):
        
        WB_orders_metrics['Баркод'] = WB_orders_metrics['Баркод'].astype(int)
        
        WB_new_metric['Баркод'] = WB_new_metric['Баркод'].astype(int)
        
        WB_orders_metrics = WB_orders_metrics.merge(WB_new_metric[['Баркод', 'Остаток']], on='Баркод', how='left')
        
        WB_orders_metrics = WB_orders_metrics.fillna(0)
        
        WB_orders_metrics[list(WB_orders_metrics.columns[5:])] = WB_orders_metrics[list(WB_orders_metrics.columns[5:])].astype(int)
        
    
    display(WB_new_metric)
    
    print()
    
    display(WB_new_metric[WB_new_metric['Артикул']=='Z0010'])
    
    print(WB_new_metric['Остаток'].sum())

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/var/folders/kd/z6thdxpn5wgf06_21113yy0c0000gq/T/ipykernel_44766/3687215995.py:86: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  deperson_data = deperson_data.groupby('Артикул')[R, 'Остаток'].sum().reset_index(drop=False)


,Артикул,07-February-2023,08-February-2023,09-February-2023,10-February-2023,11-February-2023,12-February-2023,13-February-2023,14-February-2023,15-February-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,27701ba,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"2,86091E+11",1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29720-слоновая кость,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28800Mod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,286091234541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30000-сиреневый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,Неопознанный товар,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


,Артикул,Наименование,Номенклатура,Баркод,ТМ,08-April-2024,Остаток
0,J0073,Органайзер подставка для ручек и карандашей,149328955,4600434586475,Birdhouse,2,386
1,J0074,Фруктовница металлическая/ Ваза для фруктов на...,149328974,4600436646801,Shiny Kitchen,2,0
2,G0164,Коврик для ванной противоскользящий быстросохн...,149329084,4601024336050,Birdhouse,1,893
3,50710,Ультразвуковой отпугиватель мышей и тараканов,149328768,4601389226515,SS&Y Group,0,788
4,50711,Ультразвуковой отпугиватель мышей и тараканов,149328769,4601431742987,SS&Y Group,0,448
...,...,...,...,...,...,...,...
847,28621,Валик массажный для спины и фитнеса МФР 45 см,146501403,7432141950743,Shark Fit,2,314
848,28622,Валик массажный для спины и фитнеса МФР 45 см,146501404,7432141950746,Shark Fit,1,325
849,B0217,Корзина металлическая для хранения серебристая...,146502279,7432141950750,Birdhouse,0,896
850,B0218,Корзина металлическая для хранения серебристая...,146506030,7432141950753,Birdhouse,2,348


,Артикул,Наименование,Номенклатура,Баркод,ТМ,08-April-2024,Остаток
588,Z0010,Прозрачная кружка с двойными стенками,48157058,7432141559103,Limpid Side,2,1888


361809


In [80]:
# WB_new_metric[WB_new_metric['Артикул']=='Z0247']

In [81]:
# WB_orders_metrics[WB_orders_metrics['Артикул']=='Z0247']

In [82]:
WB_orders_metrics['Остаток'].sum()

361809

In [83]:
# WB_depersonalized_metric = WB_depersonalized_metric.drop_duplicates().reset_index(drop=True)
# WB_depersonalized_metric

In [84]:
WB_orders_metrics_for_status = WB_orders_metrics.copy()

In [85]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [86]:
len(WB_orders_metrics) - len(list(set(WB_orders_metrics['Артикул'])))

16

### Загрузка метрик по остаткам

In [87]:
stocks_WB_metric = pd.read_csv(path_stocks_WB_metric, sep=';')

### Добавление в метрики по остаткам "новых остатков" из сведенных метрик с "остатками" и заказами" по дню

In [88]:
stocks_WB_metric = add_stocks_in_stocks_metric(stocks_WB_metric, WB_orders_metrics)

### Замена значения для "неуникальных" артикулов в метриках по остаткам

In [89]:
stocks_WB_metric = drop_products(stocks_WB_metric)

### Замена значения для "неуникальных" артикулов в метриках основных (после сведения)

In [90]:
WB_orders_metrics = drop_products(WB_orders_metrics)

### Проверка метрик

In [91]:
WB_orders_metrics['Остаток'].sum()

361809

In [92]:
stocks_WB_metric[stocks_WB_metric.columns[-1]].sum()

361809

In [93]:
WB_depersonalized_metric

,Артикул,07-February-2023,08-February-2023,09-February-2023,10-February-2023,11-February-2023,12-February-2023,13-February-2023,14-February-2023,15-February-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,27701ba,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"2,86091E+11",1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29720-слоновая кость,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28800Mod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,286091234541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30000-сиреневый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,Неопознанный товар,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


### Создание таблицы с "историческими" метриками без "остатка", для последующей выгрузки

In [94]:
WB_orders_metrics_final = WB_orders_metrics.copy()

WB_orders_metrics_final = WB_orders_metrics_final.drop('Остаток', axis=1)

In [95]:
dfc = 'WB_orders_metrics_final'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


### Редактирование статусов на вб и озон, исходя из появившейся динамики 

In [96]:
# st = pd.read_excel(path_st)
# st['Артикул'] = st['Артикул'].astype(str)
# st['Статус'] = st['Статус'].astype(str)

# st_oz = pd.read_excel(path_st_oz)
# st_oz['Артикул'] = st_oz['Артикул'].astype(str)
# st_oz['Статус'] = st_oz['Статус'].astype(str)


# WB_orders_metrics['Артикул'] = WB_orders_metrics['Артикул'].astype(str)
# wb_art_actual = list(set(WB_orders_metrics['Артикул']))

# art_st_change = list(set(st[(st['Артикул'].isin(wb_art_actual)) & (st['Статус']=='товары в пути')]['Артикул']))

# art_st_change_2 = list(set(st[(st['Артикул'].isin(wb_art_actual)) & (st['Статус']=='новинка Даша')]['Артикул']))

# if len(st) - len(list(set(st['Артикул']))) != 0:
#     print('error')
#     int('error')
# else:
#     for row in art_st_change:
#         st.loc[st[st['Артикул']==row].index[0], 'Статус'] = 'новинка Даша'
        
# if len(st) - len(list(set(st['Артикул']))) != 0:
#     print('error')
#     int('error')
# else:
#     st.to_excel(path_st, index=False)
    
    
# if len(st_oz) - len(list(set(st_oz['Артикул']))) != 0:
#     print('error')
#     int('error')
# else:
#     for row in art_st_change:
#         if row in list(st_oz['Артикул']):
#             st_oz.loc[st_oz[st_oz['Артикул']==row].index[0], 'Статус'] = 'новинка Даша'
#         else:
#             None
            
#     for row in art_st_change_2:
#         if row in list(st_oz['Артикул']):
#             st_oz.loc[st_oz[st_oz['Артикул']==row].index[0], 'Статус'] = 'новинка Даша'
#         else:
#             None
            
            
            
# if len(st_oz) - len(list(set(st_oz['Артикул']))) != 0:
#     print('error')
#     int('error')
# else:
#     st_oz.to_excel(path_st_oz, index=False)  

In [97]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


### Сокращение признаков с заказами в метриках

In [98]:
WB_orders_metrics = drop_time_columns(WB_orders_metrics)

In [99]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


###  Добавление признака "Предмет"

In [100]:
object_df = pd.read_csv(path_object_df, sep=';')
object_df = object_df.drop_duplicates().reset_index(drop=True)
object_df.to_csv(path_object_df, index=False, sep=";", encoding='utf-8-sig')

dfc = 'object_df'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 60


In [101]:
for i in range(len(object_df)):
    art = object_df.loc[i]['Баркод']
    if len(object_df[object_df['Баркод']==art]) > 1:
        display(object_df[object_df['Баркод']==art])

In [102]:
object_df

,Артикул,Предмет,Баркод
0,21300,Ремешки для умных часов,7432141550553
1,2700127001,Термометры электронные,7432141550835
2,19700,Ремешки для умных часов,7432141550579
3,20913,Ремешки для умных часов,7432141550731
4,20005,Ремешки для умных часов,7432141550613
...,...,...,...
1340,J0007,Плитка зеркальная,4664807383498
1341,50457,Расчески,4673613441636
1342,G0950,Подушки автомобильные,4659437081876
1343,G0636,Фонари спортивные,4674582725086


In [103]:
# ['G0800', 'Шнурки для кулонов', 4655849865141]
# object_df[object_df['Баркод'].isin([4655906011061, 
#                                     4655637993650, 
#                                     4655529556437, 
#                                     4655750572909, 
#                                     4655752763770,
#                                     4655849865141,
#                                     4655854909717,
#                                     4656163201394,
#                                     4656268479094])]

In [104]:
WB_orders_metrics = add_object(WB_orders_metrics)

In [105]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [106]:
# df = pd.read_excel(path_downloads + '2024_01_31_Subject_up.xlsx')

In [107]:
# df1 = pd.read_excel(path_downloads + 'Комиссия (3).xlsx')

In [108]:
# df['Склад поставщика - везу на склад WB, %'] = None
# df['Склад поставщика - везу самостоятельно до клиента, %'] = None


In [109]:
# for i in range(len(df)):
#     p = df.loc[i]['Предмет']
#     if p not in list(set(df1['Предмет'])):
#         df1 = pd.concat([df1, df.loc[i:i]], ignore_index=True)
#     else:
#         df1.loc[df1[df1['Предмет']==p].index[0], 'Склад WB, %'] = df.loc[i]['Склад WB, %']

In [110]:
# df1.to_excel(path_downloads + 'Комиссия от 5 февраля.xlsx', index=False)

### Проверка 

In [111]:
# object_df = pd.read_csv(path_object_df, sep=';')
# object_df.loc[len(object_df)] = ['G0709', 'Ошейники', 4655042967628]
# object_df.to_csv(path_object_df, index=False, sep=";", encoding='utf-8-sig')

In [112]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [113]:
# def add_percent_cost(data):
    
#     percent_cost = pd.read_csv(path_percent_cost, sep=';')
    
    
#     if data['Предмет'].isna().sum() != 0:
        
#         return 'Добавьте в метрики недастоющий признак "предмет" в метрики'
    
#     else:
        
#         main_list = np.setdiff1d(list(set(data['Предмет'])), list(set(percent_cost['Предмет'])))
        
#         if len(main_list) != 0:
            
#             return 'Добавьте в "Проценты и стоимости" недостающие предметы:', main_list
        
#         else:
            
#             data = data.merge(percent_cost, on='Предмет', how='left')
            
#             data['Баркод'] = data['Баркод'].astype(int)
            
#             return data

In [114]:
# percent_cost = pd.read_csv(path_percent_cost, sep=';')

In [115]:
# percent_cost

### Добавление признака "Процент комиссии"

In [116]:
WB_orders_metrics = add_commission_percentage(WB_orders_metrics)

In [117]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


### Проверка 

In [118]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Предмет,Процент комиссии
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Чехлы для планшетов,23
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,0,0,0,1,1,...,0,0,0,0,0,0,0,0,Чехлы для планшетов,23
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,2,0,0,1,0,...,0,0,0,0,0,0,0,0,Чехлы для планшетов,23
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Чехлы для планшетов,23
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,1,3,1,0,2,...,2,2,2,3,5,7,4,615,Чехлы для наушников,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,0,1,0,0,1,0,2,0,Расчески,23
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,0,1,2,1,2,8,204,Плитка зеркальная,19
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,0,1,219,Подушки автомобильные,17
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,0,469,Фонари спортивные,18


In [119]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


### Работа с "Объемом"

In [120]:
ozon_size = pd.read_csv(path_ozon_stocks_new, sep=';')

ozon_size = ozon_size[['Артикул', 'FBO OZON SKU ID', 'Barcode', 'Наименование товара', 'Бренд', 'Объем товара, л']]

ozon_size = ozon_size.rename(columns={'FBO OZON SKU ID': 'Номенклaтурa', 
                                      'Barcode': 'Баркод', 
                                      'Наименование товара': 'Название', 
                                      'Бренд': 'ТМ'})

ozon_size['Артикул'] = ozon_size['Артикул'].astype(str)

ozon_size['Артикул'] = list(map(lambda x: x.replace("'", ""), list(ozon_size['Артикул'].values)))

ozon_size_2 = ozon_size.drop('Объем товара, л', axis=1).copy()

ozon_size = ozon_size[['Баркод', 'Объем товара, л']]

ozon_size['Объем товара, л'] = list(map(lambda x: x.replace("'", ""), ozon_size['Объем товара, л']))

ozon_size['Объем товара, л'] = ozon_size['Объем товара, л'].astype(float)
Q = []
for i in range(len(ozon_size)):
    try:
        int(ozon_size.loc[i]['Баркод'])
    except:
        Q.append(i)
ozon_size = ozon_size.query('index not in @Q').reset_index(drop=True)
ozon_size['Баркод'] = ozon_size['Баркод'].astype(int)

In [121]:
len(ozon_size) - len(list(set(ozon_size['Баркод'])))

0

In [122]:
E = []
for i in range(len(ozon_size)):
    
    if len(ozon_size[ozon_size['Баркод']==ozon_size.loc[i]['Баркод']]) > 1:
        
        E.append(list(ozon_size[ozon_size['Баркод']==ozon_size.loc[i]['Баркод']].index)[0])
    else:
        E.append(i)
E = list(set(E))
ozon_size = ozon_size.query('index in @E').reset_index(drop=True) 

In [123]:
len(ozon_size) - len(list(set(ozon_size['Баркод'])))

0

In [124]:
WB_size = pd.read_excel(path_WB_size)

In [125]:
WB_size, WB_size_day = redact_wb_size(WB_size)

In [126]:
WB_size_for_metric = WB_size.copy()

In [127]:
WB_size_for_metric

,Баркод,"Объем, л"
0,7432141550840,0.94
1,7432141559167,1.10
2,7432141950561,14.78
3,7432141950483,0.40
4,7432141950484,0.40
...,...,...
1358,4651867882221,5.50
1359,4652126684235,5.50
1360,4652210873651,5.50
1361,4652433706750,5.50


In [128]:
# print(f'Log_pr_koledino_1: {Log_pr_koledino_1}')
# print()
# print(f'Log_pr_koledino_2: {Log_pr_koledino_2}')


In [129]:
WB_size[WB_size['Объем, л'] == 0]

,Баркод,"Объем, л"


In [130]:
WB_size

,Баркод,"Объем, л"
0,7432141550840,0.94
1,7432141559167,1.10
2,7432141950561,14.78
3,7432141950483,0.40
4,7432141950484,0.40
...,...,...
1358,4651867882221,5.50
1359,4652126684235,5.50
1360,4652210873651,5.50
1361,4652433706750,5.50


In [131]:
# f = pd.read_excel(path_downloads + 'full_WB_df нет данных по лггистике.xlsx', sheet_name='Лист1')

In [132]:
# f

In [133]:
# WB_size.query(f'Баркод in {list(f["Баркод"])}')

In [134]:
ozon_size = ozon_size[ozon_size['Объем товара, л']!=0].reset_index(drop=True)

In [135]:
for i in range(len(WB_size)):
    
    if WB_size.loc[i]['Объем, л'] == 0:
    
        if WB_size.loc[i]['Баркод'] in list(set(ozon_size['Баркод'])):
            
            WB_size['Объем, л'].loc[i] = ozon_size[ozon_size['Баркод']==WB_size.loc[i]['Баркод']]['Объем товара, л'].values[0]

In [136]:
WB_size[WB_size['Объем, л'] == 0]

,Баркод,"Объем, л"


In [137]:
# WB_size.query(f'Баркод in {list(f["Баркод"])}')

In [138]:
WB_size

,Баркод,"Объем, л"
0,7432141550840,0.94
1,7432141559167,1.10
2,7432141950561,14.78
3,7432141950483,0.40
4,7432141950484,0.40
...,...,...
1358,4651867882221,5.50
1359,4652126684235,5.50
1360,4652210873651,5.50
1361,4652433706750,5.50


In [139]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [140]:
# dfc = 'WB_size'

# print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
# print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

In [141]:
WB_size

,Баркод,"Объем, л"
0,7432141550840,0.94
1,7432141559167,1.10
2,7432141950561,14.78
3,7432141950483,0.40
4,7432141950484,0.40
...,...,...
1358,4651867882221,5.50
1359,4652126684235,5.50
1360,4652210873651,5.50
1361,4652433706750,5.50


In [142]:
WB_size666 = WB_size.copy()

In [143]:
WB_size[WB_size['Баркод']==7432141550509]

,Баркод,"Объем, л"
626,7432141550509,0.22


In [144]:
Log_pr_marketplace_1

30.0

In [145]:
Log_pr_marketplace_2

7.000000000000001

In [146]:
WB_size[WB_size['Баркод']==7432141550509]

,Баркод,"Объем, л"
626,7432141550509,0.22


In [147]:
Log_pr_marketplace_1

30.0

In [148]:
print(f"Log_b: {Log_b}")
print(f"Hr_b_korob: {Hr_b_korob}")
print(f"Hr_b_monop: {Hr_b_monop}")
print(f"quantity_liters_for_logistic: {quantity_liters_for_logistic}")
print(f"quantity_liters_for_hr: {quantity_liters_for_hr}")
print(f"localization_index_wb: {localization_index_wb}")

Log_b: [30, 7]
Hr_b_korob: [0.16, 0.12]
Hr_b_monop: 32
quantity_liters_for_logistic: 1
quantity_liters_for_hr: 1
localization_index_wb: 0.9


In [149]:
print(f"Log_b: {Log_b}")
print(f"Hr_b_korob: {Hr_b_korob}")
print(f"Hr_b_monop: {Hr_b_monop}")
print(f"quantity_liters_for_logistic: {quantity_liters_for_logistic}")
print(f"quantity_liters_for_hr: {quantity_liters_for_hr}")
print(f"localization_index_wb: {localization_index_wb}")

Log_b: [30, 7]
Hr_b_korob: [0.16, 0.12]
Hr_b_monop: 32
quantity_liters_for_logistic: 1
quantity_liters_for_hr: 1
localization_index_wb: 0.9


In [150]:
Log_pr_marketplace_1

30.0

In [151]:
quantity_liters_for_logistic

1

In [152]:
Log_pr_marketplace_2

7.000000000000001

In [153]:
LOG

,Склад,Стоимость логистики монопалета (база),Стоимость логистики монопалета (доп. литр),Стоимость хранения монопалета (за день),Стоимость логистики короб (база),Стоимость логистики короб (доп. литр),Стоимость хранения короб (база),Стоимость хранения короб (доп. литр)
0,Астана,NaN,NaN,NaN,43.5,10.15,0.232,0.174
1,Атакент,22.5,5.25,24.0,36.0,8.4,0.192,0.144
2,Белая дача,37.5,8.75,40.0,55.5,12.95,0.296,0.222
3,Белые Столбы,NaN,NaN,NaN,100.5,23.45,0.536,0.402
4,Волгоград,60.0,14.0,64.0,NaN,NaN,NaN,NaN
5,Вёшки,105.0,24.5,112.0,36.0,8.4,0.192,0.144
6,Екатеринбург - Испытателей 14г,NaN,NaN,NaN,55.5,12.95,0.296,0.222
7,Екатеринбург - Перспективный 12,34.5,8.05,36.8,45.0,10.5,0.24,0.18
8,Иваново,40.5,9.45,43.2,NaN,NaN,NaN,NaN
9,Казань,72.0,16.8,76.8,60.0,14.0,0.32,0.24


In [154]:
quantity_liters_for_hr

1

In [155]:
Log_pr_marketplace_1

30.0

In [156]:
WB_size

,Баркод,"Объем, л"
0,7432141550840,0.94
1,7432141559167,1.10
2,7432141950561,14.78
3,7432141950483,0.40
4,7432141950484,0.40
...,...,...
1358,4651867882221,5.50
1359,4652126684235,5.50
1360,4652210873651,5.50
1361,4652433706750,5.50


In [157]:
WB_size_copy_market = WB_size.copy()

Q = None
Q = []

for i in range(len(WB_size_copy_market)):

    if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_logistic and WB_size_copy_market.loc[i]['Объем, л'] > 0:

        Q.append(Log_pr_atak_1)

    elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_logistic:

        Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - quantity_liters_for_logistic) * Log_pr_atak_2)+Log_pr_atak_1)

    elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

        Q.append(0)

WB_size_copy_market['Стоимость логистики'] = Q

WB_size_copy_market['Стоимость логистики'] = WB_size_copy_market['Стоимость логистики'] * localization_index_wb


Q = []

for i in range(len(WB_size_copy_market)):

    if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_hr and WB_size_copy_market.loc[i]['Объем, л'] > 0:

        Q.append(Hr_pr_atak_1)

    elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_hr:

        Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - 3) * Hr_pr_atak_2)+Hr_pr_atak_1)

    elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

        Q.append(0)


WB_size_copy_market['Стоимость хранения'] = Q

WB_size_copy_market = WB_size_copy_market.drop('Объем, л', axis=1)

Q = None

WB_size_copy_market.columns = ['Баркод', 'Стоимость_логистики', 'Стоимость_хранения']

mwm.update_or_create_table_not_date(WB_size_copy_market, 
                                    'wb_kz_log_hr', 
                                    ['Баркод'], 
                                    schema=mwm.ir_schema)

WB_size_copy_market_atak = WB_size_copy_market.copy()
WB_size_copy_market = None

In [158]:
WB_size_copy_market = WB_size.copy()

Q = None
Q = []

for i in range(len(WB_size_copy_market)):

    if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_logistic and WB_size_copy_market.loc[i]['Объем, л'] > 0:

        Q.append(Log_pr_kaz_1)

    elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_logistic:

        Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - quantity_liters_for_logistic) * Log_pr_kaz_2)+Log_pr_kaz_1)

    elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

        Q.append(0)

WB_size_copy_market['Стоимость логистики'] = Q

WB_size_copy_market['Стоимость логистики'] = WB_size_copy_market['Стоимость логистики'] * localization_index_wb


Q = []

for i in range(len(WB_size_copy_market)):

    if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_hr and WB_size_copy_market.loc[i]['Объем, л'] > 0:

        Q.append(Hr_pr_kaz_1)

    elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_hr:

        Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - 3) * Hr_pr_kaz_2)+Hr_pr_kaz_1)

    elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

        Q.append(0)


WB_size_copy_market['Стоимость хранения'] = Q

WB_size_copy_market = WB_size_copy_market.drop('Объем, л', axis=1)

Q = None

In [159]:
WB_size_copy_market

,Баркод,Стоимость логистики,Стоимость хранения
0,7432141550840,54.0,0.32
1,7432141559167,54.0,-0.16
2,7432141950561,230.4,3.20
3,7432141950483,54.0,0.32
4,7432141950484,54.0,0.32
...,...,...,...
1358,4651867882221,117.0,1.04
1359,4652126684235,117.0,1.04
1360,4652210873651,117.0,1.04
1361,4652433706750,117.0,1.04


In [160]:
# if flag_pr_koledino == True:
    
#     WB_size_copy_market = WB_size.copy()

#     Q = None
#     Q = []

#     for i in range(len(WB_size_copy_market)):

#         if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_logistic and WB_size_copy_market.loc[i]['Объем, л'] > 0:

#             Q.append(Log_pr_koledino_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_logistic:

#             Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - quantity_liters_for_logistic) * Log_pr_koledino_2)+Log_pr_koledino_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

#             Q.append(0)

#     WB_size_copy_market['Стоимость логистики'] = Q

#     WB_size_copy_market['Стоимость логистики'] = WB_size_copy_market['Стоимость логистики'] * localization_index_wb


#     Q = []

#     for i in range(len(WB_size_copy_market)):

#         if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_hr and WB_size_copy_market.loc[i]['Объем, л'] > 0:

#             Q.append(Hr_pr_koledino_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_hr:

#             Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - 3) * Hr_pr_koledino_2)+Hr_pr_koledino_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

#             Q.append(0)


#     WB_size_copy_market['Стоимость хранения'] = Q

#     WB_size_copy_market = WB_size_copy_market.drop('Объем, л', axis=1)

#     Q = None

# elif flag_pr_kaz == True:
    
    
    
    
#     WB_size_copy_market = WB_size.copy()

#     Q = None
#     Q = []

#     for i in range(len(WB_size_copy_market)):

#         if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_logistic and WB_size_copy_market.loc[i]['Объем, л'] > 0:

#             Q.append(Log_pr_kaz_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_logistic:

#             Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - quantity_liters_for_logistic) * Log_pr_kaz_2)+Log_pr_kaz_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

#             Q.append(0)

#     WB_size_copy_market['Стоимость логистики'] = Q

#     WB_size_copy_market['Стоимость логистики'] = WB_size_copy_market['Стоимость логистики'] * localization_index_wb


#     Q = []

#     for i in range(len(WB_size_copy_market)):

#         if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_hr and WB_size_copy_market.loc[i]['Объем, л'] > 0:

#             Q.append(Hr_pr_kaz_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_hr:

#             Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - 3) * Hr_pr_kaz_2)+Hr_pr_kaz_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

#             Q.append(0)


#     WB_size_copy_market['Стоимость хранения'] = Q

#     WB_size_copy_market = WB_size_copy_market.drop('Объем, л', axis=1)

#     Q = None

    
# elif flag_pr_atak == True:
    
    
    
#     WB_size_copy_market = WB_size.copy()

#     Q = None
#     Q = []

#     for i in range(len(WB_size_copy_market)):

#         if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_logistic and WB_size_copy_market.loc[i]['Объем, л'] > 0:

#             Q.append(Log_pr_atak_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_logistic:

#             Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - quantity_liters_for_logistic) * Log_pr_atak_2)+Log_pr_atak_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

#             Q.append(0)

#     WB_size_copy_market['Стоимость логистики'] = Q

#     WB_size_copy_market['Стоимость логистики'] = WB_size_copy_market['Стоимость логистики'] * localization_index_wb


#     Q = []

#     for i in range(len(WB_size_copy_market)):

#         if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_hr and WB_size_copy_market.loc[i]['Объем, л'] > 0:

#             Q.append(Hr_pr_atak_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_hr:

#             Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - 3) * Hr_pr_atak_2)+Hr_pr_atak_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

#             Q.append(0)


#     WB_size_copy_market['Стоимость хранения'] = Q

#     WB_size_copy_market = WB_size_copy_market.drop('Объем, л', axis=1)

#     Q = None

    
    
    
    
    
    
# else:
    
#     WB_size_copy_market = WB_size.copy()

#     Q = None
#     Q = []

#     for i in range(len(WB_size_copy_market)):

#         if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_logistic and WB_size_copy_market.loc[i]['Объем, л'] > 0:

#             Q.append(Log_pr_marketplace_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_logistic:

#             Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - quantity_liters_for_logistic) * Log_pr_marketplace_2)+Log_pr_marketplace_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

#             Q.append(0)

#     WB_size_copy_market['Стоимость логистики'] = Q

#     WB_size_copy_market['Стоимость логистики'] = WB_size_copy_market['Стоимость логистики'] * localization_index_wb


#     Q = []

#     for i in range(len(WB_size_copy_market)):

#         if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_hr and WB_size_copy_market.loc[i]['Объем, л'] > 0:

#             Q.append(Hr_pr_marketplace_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_hr:

#             Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - 3) * Hr_pr_marketplace_2)+Hr_pr_marketplace_1)

#         elif WB_size_copy_market.loc[i]['Объем, л'] == 0:

#             Q.append(0)


#     WB_size_copy_market['Стоимость хранения'] = Q

#     WB_size_copy_market = WB_size_copy_market.drop('Объем, л', axis=1)

#     Q = None

In [161]:
# WB_size_copy_market = WB_size.copy()

# Q = None
# Q = []

# for i in range(len(WB_size_copy_market)):
    
#     if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_logistic and WB_size_copy_market.loc[i]['Объем, л'] > 0:
        
#         Q.append(Log_pr_marketplace_1)
        
#     elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_logistic:
        
#         Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - quantity_liters_for_logistic) * Log_pr_marketplace_2)+Log_pr_marketplace_1)
        
#     elif WB_size_copy_market.loc[i]['Объем, л'] == 0:
        
#         Q.append(0)

# WB_size_copy_market['Стоимость логистики'] = Q

# WB_size_copy_market['Стоимость логистики'] = WB_size_copy_market['Стоимость логистики'] * localization_index_wb


# Q = []

# for i in range(len(WB_size_copy_market)):
    
#     if WB_size_copy_market.loc[i]['Объем, л'] <= quantity_liters_for_hr and WB_size_copy_market.loc[i]['Объем, л'] > 0:
        
#         Q.append(Hr_pr_marketplace_1)
        
#     elif WB_size_copy_market.loc[i]['Объем, л'] > quantity_liters_for_hr:
        
#         Q.append(((WB_size_copy_market.loc[i]['Объем, л'].round() - 3) * Hr_pr_marketplace_2)+Hr_pr_marketplace_1)
        
#     elif WB_size_copy_market.loc[i]['Объем, л'] == 0:
        
#         Q.append(0)
        
        
# WB_size_copy_market['Стоимость хранения'] = Q

# WB_size_copy_market = WB_size_copy_market.drop('Объем, л', axis=1)

# Q = None

In [162]:
WB_size_copy_market

,Баркод,Стоимость логистики,Стоимость хранения
0,7432141550840,54.0,0.32
1,7432141559167,54.0,-0.16
2,7432141950561,230.4,3.20
3,7432141950483,54.0,0.32
4,7432141950484,54.0,0.32
...,...,...,...
1358,4651867882221,117.0,1.04
1359,4652126684235,117.0,1.04
1360,4652210873651,117.0,1.04
1361,4652433706750,117.0,1.04


In [163]:
WB_size_copy_market[WB_size_copy_market['Баркод']==7432141550509]

,Баркод,Стоимость логистики,Стоимость хранения
626,7432141550509,54.0,0.32


In [164]:
WB_size = WB_size_copy_market.copy()

In [165]:
WB_size_copy_market[WB_size_copy_market['Баркод']==4600436646801]

,Баркод,Стоимость логистики,Стоимость хранения
1034,4600436646801,117.0,1.04


In [166]:
WB_size

,Баркод,Стоимость логистики,Стоимость хранения
0,7432141550840,54.0,0.32
1,7432141559167,54.0,-0.16
2,7432141950561,230.4,3.20
3,7432141950483,54.0,0.32
4,7432141950484,54.0,0.32
...,...,...,...
1358,4651867882221,117.0,1.04
1359,4652126684235,117.0,1.04
1360,4652210873651,117.0,1.04
1361,4652433706750,117.0,1.04


In [167]:
WB_orders_metrics_copy_markt = WB_orders_metrics.copy()

WB_orders_metrics_copy_markt = WB_orders_metrics_copy_markt.merge(WB_size_copy_market, on='Баркод', how='left')
WB_orders_metrics_copy_markt['Стоимость логистики'] = WB_orders_metrics_copy_markt['Стоимость логистики'].fillna(0)
WB_orders_metrics_copy_markt['Стоимость хранения'] = WB_orders_metrics_copy_markt['Стоимость хранения'].fillna(0)


for i in range(len(WB_orders_metrics_copy_markt)):
    
    row = WB_orders_metrics_copy_markt.loc[i]['Артикул']
    
    if len(WB_orders_metrics_copy_markt[WB_orders_metrics_copy_markt['Артикул']==row]) > 1 and WB_orders_metrics_copy_markt.loc[i]['Стоимость логистики'] == 0:
        
         WB_orders_metrics_copy_markt['Стоимость логистики'].loc[i] = WB_orders_metrics_copy_markt[(WB_orders_metrics_copy_markt['Артикул']==row) & (WB_orders_metrics_copy_markt['Стоимость логистики'] !=0)]['Стоимость логистики'].values[0]
            
            
            
for i in range(len(WB_orders_metrics_copy_markt)):
    
    row = WB_orders_metrics_copy_markt.loc[i]['Артикул']
    
    if len(WB_orders_metrics_copy_markt[WB_orders_metrics_copy_markt['Артикул']==row]) > 1 and WB_orders_metrics_copy_markt.loc[i]['Стоимость хранения'] == 0:
        
         WB_orders_metrics_copy_markt['Стоимость хранения'].loc[i] = WB_orders_metrics_copy_markt[(WB_orders_metrics_copy_markt['Артикул']==row) & (WB_orders_metrics_copy_markt['Стоимость хранения'] !=0)]['Стоимость хранения'].values[0]
            
            
            
WB_orders_metrics_copy_markt = add_cost_price(WB_orders_metrics_copy_markt)

try: 
    WB_orders_metrics_copy_markt = add_price(WB_orders_metrics_copy_markt, price)
except:
    print(WB_orders_metrics_copy_markt)

WB_orders_metrics_copy_markt = creating_calculated_features(WB_orders_metrics_copy_markt)

WB_orders_metrics_copy_markt.to_excel(path_downloads + 'Расчетные признаки по вб на этапе вычисления наценки.xlsx', index=False)

WB_orders_metrics_copy_markt = None
WB_size_copy_market = None




In [168]:
WB_size

,Баркод,Стоимость логистики,Стоимость хранения
0,7432141550840,54.0,0.32
1,7432141559167,54.0,-0.16
2,7432141950561,230.4,3.20
3,7432141950483,54.0,0.32
4,7432141950484,54.0,0.32
...,...,...,...
1358,4651867882221,117.0,1.04
1359,4652126684235,117.0,1.04
1360,4652210873651,117.0,1.04
1361,4652433706750,117.0,1.04


In [169]:
# for row in ['G0287', 'G0288']:
#     add_art_for_cost_price([row], [0])

In [170]:
WB_size[WB_size['Баркод']==4600436646801]

,Баркод,Стоимость логистики,Стоимость хранения
1034,4600436646801,117.0,1.04


In [171]:
WB_size

,Баркод,Стоимость логистики,Стоимость хранения
0,7432141550840,54.0,0.32
1,7432141559167,54.0,-0.16
2,7432141950561,230.4,3.20
3,7432141950483,54.0,0.32
4,7432141950484,54.0,0.32
...,...,...,...
1358,4651867882221,117.0,1.04
1359,4652126684235,117.0,1.04
1360,4652210873651,117.0,1.04
1361,4652433706750,117.0,1.04


In [172]:
WB_orders_metrics = WB_orders_metrics.merge(WB_size, on='Баркод', how='left')
WB_orders_metrics['Стоимость логистики'] = WB_orders_metrics['Стоимость логистики'].fillna(0)
WB_orders_metrics['Стоимость хранения'] = WB_orders_metrics['Стоимость хранения'].fillna(0)

In [173]:
for i in range(len(WB_orders_metrics)):
    
    row = WB_orders_metrics.loc[i]['Артикул']
    
    if len(WB_orders_metrics[WB_orders_metrics['Артикул']==row]) > 1 and WB_orders_metrics.loc[i]['Стоимость логистики'] == 0:
        
         WB_orders_metrics['Стоимость логистики'].loc[i] = WB_orders_metrics[(WB_orders_metrics['Артикул']==row) & (WB_orders_metrics['Стоимость логистики'] !=0)]['Стоимость логистики'].values[0]

In [174]:
for i in range(len(WB_orders_metrics)):
    
    row = WB_orders_metrics.loc[i]['Артикул']
    
    if len(WB_orders_metrics[WB_orders_metrics['Артикул']==row]) > 1 and WB_orders_metrics.loc[i]['Стоимость хранения'] == 0:
        
         WB_orders_metrics['Стоимость хранения'].loc[i] = WB_orders_metrics[(WB_orders_metrics['Артикул']==row) & (WB_orders_metrics['Стоимость хранения'] !=0)]['Стоимость хранения'].values[0]

In [175]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [176]:
# WB_size['Стоимость логистики (по литрам)'] = Q
# WB_size = WB_size.drop('Объем, л', axis=1)
# WB_orders_metrics = WB_orders_metrics.merge(WB_size, on='Баркод', how='left')
# WB_orders_metrics['Стоимость логистики (по литрам)'] = WB_orders_metrics['Стоимость логистики (по литрам)'].fillna(0)
# WB_orders_metrics['Стоимость логистики'] = WB_orders_metrics['Стоимость логистики (по литрам)']
# WB_orders_metrics = WB_orders_metrics.drop('Стоимость логистики (по литрам)', axis=1)
# for i in range(len(WB_orders_metrics)):
    
#     row = WB_orders_metrics.loc[i]['Артикул']
    
#     if len(WB_orders_metrics[WB_orders_metrics['Артикул']==row]) > 1 and WB_orders_metrics.loc[i]['Стоимость логистики'] == 0:
        
#          WB_orders_metrics['Стоимость логистики'].loc[i] = WB_orders_metrics[(WB_orders_metrics['Артикул']==row) & (WB_orders_metrics['Стоимость логистики'] !=0)]['Стоимость логистики'].values[0]
            


In [177]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [178]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Предмет,Процент комиссии,Стоимость логистики,Стоимость хранения
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,Чехлы для планшетов,23,54.0,0.32
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,0,0,0,1,1,...,0,0,0,0,0,0,Чехлы для планшетов,23,54.0,0.32
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,2,0,0,1,0,...,0,0,0,0,0,0,Чехлы для планшетов,23,54.0,0.32
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,Чехлы для планшетов,23,54.0,0.32
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,1,3,1,0,2,...,2,3,5,7,4,615,Чехлы для наушников,23,54.0,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,0,0,1,0,2,0,Расчески,23,54.0,0.32
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,1,2,1,2,8,204,Плитка зеркальная,19,54.0,0.32
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,1,219,Подушки автомобильные,17,255.6,3.68
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,469,Фонари спортивные,18,66.6,0.08


In [179]:
WB_orders_metrics['Баркод'] = WB_orders_metrics['Баркод'].astype(int)

In [180]:
WB_orders_metrics666 = WB_orders_metrics.copy()
WB_orders_metrics666 = WB_orders_metrics666.drop('Стоимость логистики', axis=1)

### Подгрузка "объема" к метрикам по объемам

In [181]:
WB_size_metrics = redact_WB_size_metrics(WB_size_for_metric, WB_size_day, WB_orders_metrics)

### Добавление признака "Себестоимость"

In [182]:
WB_orders_metrics = add_cost_price(WB_orders_metrics)

### Проверка 

In [183]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Предмет,Процент комиссии,Стоимость логистики,Стоимость хранения,Себестоимость
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,Чехлы для планшетов,23,54.0,0.32,243.71
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,0,0,0,1,1,...,0,0,0,0,0,Чехлы для планшетов,23,54.0,0.32,243.71
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,2,0,0,1,0,...,0,0,0,0,0,Чехлы для планшетов,23,54.0,0.32,243.71
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,Чехлы для планшетов,23,54.0,0.32,243.71
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,1,3,1,0,2,...,3,5,7,4,615,Чехлы для наушников,23,54.0,0.32,50.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,0,1,0,2,0,Расчески,23,54.0,0.32,74.78
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,2,1,2,8,204,Плитка зеркальная,19,54.0,0.32,73.12
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,1,219,Подушки автомобильные,17,255.6,3.68,390.38
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,469,Фонари спортивные,18,66.6,0.08,878.91


In [184]:
# for row in ['A4507', '17031', '17030']:
#     add_art_for_cost_price([row], [0])

In [185]:
# add_art_for_cost_price(['50456'], [0])
# add_art_for_cost_price(['A4503'], [0])

In [186]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [187]:
for i in range(len(WB_orders_metrics)):
    b = WB_orders_metrics.loc[i]['Баркод']
    if len(WB_orders_metrics[WB_orders_metrics['Баркод']==b]) > 1:
        display(WB_orders_metrics[WB_orders_metrics['Баркод']==b])
        exit()

In [188]:
dfc = 'WB_size_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 62


In [189]:
# add_art_for_cost_price(['16952'], [0])

In [190]:
# for row in ['50711',
#   '50710',
#   'J0073']:
#     add_art_for_cost_price([row], [0])

In [191]:
# for row in ['B0156',
#   'A2910',
#   '50961',
#   'B0170',
#   '29720',
#   'G0139',
#   'B0150',
#   '28810',
#   'B0252',
#   '50333',
#   '50960',
#   'B0149',
#   '50342',
#   'B0250',
#   '29724',
#   'G0109',
#   'G0123',
#   '22810',
#   'G0345',
#   'B0155',
#   '22814',
#   'B0157',
#   '50970',
#   '22812',
#   'G0042',
#   'J0005',
#   'G0041',
#   'G0121',
#   '22813']:
#     add_art_for_cost_price([row], [0])

In [192]:
# for row in ['G0114',
#   '50341',
#   'B0260',
#   'G0126',
#   'G0122',
#   '29723',
#   '29721',
#   'G0128',
#   'G0113',
#   'J0004',
#   'G0125',
#   'B0147',
#   '28812',
#   'G0127',
#   '50343',
#   'G0124',
#   'B0251',
#   'G0120',
#   'B0148',
#   '50340']:
#     add_art_for_cost_price([row], [0])

In [193]:
# add_art_for_cost_price(['G0335'], [0])

### Добавление признака "Цена"

In [194]:
WB_orders_metrics = add_price(WB_orders_metrics, price)

In [195]:
for i in range(len(WB_orders_metrics)):
    art = WB_orders_metrics.loc[i]['Артикул']
    if art == 'G0709' and WB_orders_metrics.loc[i]['Цена'] == 0:
        WB_orders_metrics.loc[i, 'Цена'] = 780
    elif art == 'G0800' and WB_orders_metrics.loc[i]['Цена'] == 0:
        WB_orders_metrics.loc[i, 'Цена'] = 340

In [196]:
for i in range(len(WB_orders_metrics)):
    art = WB_orders_metrics.loc[i]['Артикул']
    if len(WB_orders_metrics[WB_orders_metrics['Артикул']==art]) > 1:
        p = [x for x in list(WB_orders_metrics[WB_orders_metrics['Артикул']==art]['Цена']) if x != 0]
        if len(p) > 0:
            p = p[-1]
        else:   
            display(WB_orders_metrics[WB_orders_metrics['Артикул']==art])

In [197]:
for i in range(len(WB_orders_metrics)):
    art = WB_orders_metrics.loc[i]['Артикул']
    if len(WB_orders_metrics[WB_orders_metrics['Артикул']==art]) > 1:
        
        p = [x for x in list(WB_orders_metrics[WB_orders_metrics['Артикул']==art]['Цена']) if x != 0][-1]
        for ind in list(WB_orders_metrics[WB_orders_metrics['Артикул']==art].index):
            if WB_orders_metrics.loc[ind]['Цена'] == 0:
                WB_orders_metrics.loc[ind, 'Цена'] = p
                display(WB_orders_metrics.loc[ind:ind])

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,08-April-2024,Остаток,Предмет,Процент комиссии,Стоимость логистики,Стоимость хранения,Себестоимость,Цена до скидки,Скидка,Цена
93,27701,"Диск балансировочный, графит",11478762,7432141550851,Shark Fit,0,0,0,0,0,...,0,0,Подушки балансировочные,18,117.0,1.04,359.45,0.0,0.0,528.0


,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,08-April-2024,Остаток,Предмет,Процент комиссии,Стоимость логистики,Стоимость хранения,Себестоимость,Цена до скидки,Скидка,Цена
1133,G0069,Застежка для уменьшения брюк латунь,165118565,4644647421062,JewelryMeverly,0,0,0,0,0,...,0,0,Ремни,23,54.0,0.32,66.07,0.0,0.0,301.0


,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,08-April-2024,Остаток,Предмет,Процент комиссии,Стоимость логистики,Стоимость хранения,Себестоимость,Цена до скидки,Скидка,Цена
1119,G0413,Спортивный пояс размер XXL,37338500,7432141557675,Shark Fit,14,8,2,3,2,...,3,309,Пояса для похудения,20,54.0,0.32,139.67,0.0,0.0,449.0


,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,08-April-2024,Остаток,Предмет,Процент комиссии,Стоимость логистики,Стоимость хранения,Себестоимость,Цена до скидки,Скидка,Цена
1118,G0636,Фонарь-прожектор,165763226,4647277236624,Ocean of Light,0,0,0,0,0,...,0,0,Фонари спортивные,18,66.6,0.08,878.91,0.0,0.0,1970.0


In [198]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


### Проверка

In [199]:
ccc = 0
for row in list(WB_orders_metrics['Цена'].unique()):
    if round(row) < row:
        ccc +=1
        print(row)

In [200]:
if ccc == 0:
    WB_orders_metrics['Цена'] = WB_orders_metrics['Цена'].astype(int)

In [201]:
ccc = 0
for row in list(WB_orders_metrics['Цена до скидки'].unique()):
    if round(row) < row:
        ccc +=1
        print(row)

In [202]:
if ccc == 0:
    WB_orders_metrics['Цена до скидки'] = WB_orders_metrics['Цена до скидки'].astype(int)

In [203]:
## Цена до скидки

dop_price = pd.read_excel(path_price,
                          usecols=['Артикул продавца', 'Текущая розн. цена (до скидки)'],
                          dtype={'Артикул продавца': 'str',
                                 'Текущая розн. цена (до скидки)': 'int'})
dop_price.columns = ['Артикул', 'Цена до скидки']
dop_price['Артикул'] = [x.replace('/', '') for x in list(dop_price['Артикул'])]
dop_price = dop_price.groupby('Артикул').mean().reset_index(drop=False)

WB_orders_metrics['Цена до скидки'] = list(dop_price.merge(WB_orders_metrics[['Артикул']], on='Артикул', how='right')['Цена до скидки'])

# WB_full_df.insert(list(WB_full_df.columns).index('Цена')+1, 
#                   'Цена до скидки', 
#                   list(WB_full_df[['Артикул']]\
#                        .merge(dop_price, 
#                               on='Артикул', 
#                               how='left')['Цена до скидки']))

In [204]:
WB_orders_metrics[WB_orders_metrics['Артикул']=='G0800']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,08-April-2024,Остаток,Предмет,Процент комиссии,Стоимость логистики,Стоимость хранения,Себестоимость,Цена до скидки,Скидка,Цена
1227,G0800,"Шнурок гайтан кожаный 2 мм, 40см",168229542,4655849865141,JewelryMeverly,0,0,0,0,0,...,0,0,Шнурки для кулонов,19,54.0,0.32,66.01,315.0,0.0,340
1229,G0800,"Шнурок гайтан кожаный 2 мм, 40см",168229542,4655750572909,JewelryMeverly,0,0,0,0,0,...,0,0,Шнурки для кулонов,19,54.0,0.32,66.01,315.0,0.0,340
1230,G0800,"Шнурок гайтан кожаный 2 мм, 40см",168229542,4655752763770,JewelryMeverly,0,0,1,0,0,...,0,1,Шнурки для кулонов,19,54.0,0.32,66.01,315.0,0.0,340
1233,G0800,"Шнурок гайтан кожаный 2 мм, 40см",168229542,4655637993650,JewelryMeverly,0,0,0,0,0,...,0,0,Шнурки для кулонов,19,54.0,0.32,66.01,315.0,0.0,340
1235,G0800,"Шнурок гайтан кожаный 2 мм, 40см",168229542,4655587563330,JewelryMeverly,0,0,0,0,0,...,0,0,Шнурки для кулонов,19,54.0,0.32,66.01,315.0,0.0,340
1236,G0800,"Шнурок гайтан кожаный 2 мм, 40см",168229542,4656268479094,JewelryMeverly,0,0,0,0,0,...,0,0,Шнурки для кулонов,19,54.0,0.32,66.01,315.0,0.0,315
1242,G0800,"Шнурок гайтан кожаный 2 мм, 40см",168229542,4655529556437,JewelryMeverly,0,0,0,0,0,...,0,0,Шнурки для кулонов,19,54.0,0.32,66.01,315.0,0.0,340
1249,G0800,"Шнурок гайтан кожаный 2 мм, 40см",168229542,4655854909717,JewelryMeverly,0,0,0,0,0,...,0,0,Шнурки для кулонов,19,54.0,0.32,66.01,315.0,0.0,340


In [205]:
## Редактирование цены и цены до скидки у размерных товаров

for row in [['G0270', 'G0271', 'G0272','G0273'], 
            ['G0282', 'G0280', 'G0281'], 
            ['G0370', 'G0371', 'G0372', 'G0373', 'G03734'],
            ['G0400', 'G0401', 'G0402', 'G0403', 'G0404'],
            ['G0410', 'G0411', 'G0412', 'G0413'],
            ['G0275', 'G0274', 'G0276'],
            ['G0284', 'G0283', 'G0285'],
            ['G0700', 'G0701', 'G0702'], 
            ['G0703', 'G0704', 'G0705'], 
            ['G0706', 'G0707', 'G0708'], 
            ['G0709', 'G0710']]:
    l_p = list(WB_orders_metrics[WB_orders_metrics['Артикул'].isin(row)]['Цена'])
    l_p = [x for x in l_p if x > 0 and str(x) != 'nan']
    l_pd = list(WB_orders_metrics[WB_orders_metrics['Артикул'].isin(row)]['Цена до скидки'])
    l_pd = [x for x in l_pd if x > 0 and str(x) != 'nan']
    l_p = l_p[0]
    l_pd = l_pd[0]
    for i in list(WB_orders_metrics[WB_orders_metrics['Артикул'].isin(row)].index):
        WB_orders_metrics.loc[i, 'Цена'] = l_p
        WB_orders_metrics.loc[i, 'Цена до скидки'] = l_pd


In [206]:
dop_price_2 = WB_orders_metrics[['Артикул', 'Цена до скидки']].copy()

In [207]:
dop_price_2

,Артикул,Цена до скидки
0,16801,5940.0
1,16802,2300.0
2,16803,3144.0
3,16804,4236.0
4,18301,1150.0
...,...,...
1245,50457,390.0
1246,J0007,646.0
1247,G0950,1590.0
1248,G0636,2627.0


In [208]:
len(dop_price_2) - len(list(set(dop_price_2['Артикул'])))

16

In [209]:
dop_price_2

,Артикул,Цена до скидки
0,16801,5940.0
1,16802,2300.0
2,16803,3144.0
3,16804,4236.0
4,18301,1150.0
...,...,...
1245,50457,390.0
1246,J0007,646.0
1247,G0950,1590.0
1248,G0636,2627.0


In [210]:
flag_stop_nac

False

In [211]:
if flag_stop_nac == True:
    exit()

In [212]:
# def nac_pr_wb(art, price):
#     data = WB_orders_metrics[WB_orders_metrics['Артикул']==art].reset_index(drop=True).copy()
#     data.loc[0, 'Цена'] = price
#     data = creating_calculated_features(data)[['Артикул', 'Цена', 'Наценка']]
    
#     return data

In [213]:
# nac_pr_wb('G0650', 1374)

In [214]:
# wb_markups.creating_calculated_features(['G0650'], [1374])

In [215]:
# article_barcodes.merge(WB_orders_metrics_for_ac[['Баркод', 'Стоимость хранения']].rename(columns={'Баркод': 'barcode'}),
#                        on='barcode',
#                        how='right').to_excel(path_downloads + 'Стомость хранения вб.xlsx', index=False)

In [216]:
# WB_orders_metrics['Процент комиссии'] = WB_orders_metrics['Процент комиссии'] + 4.5

In [217]:
WB_orders_metrics_for_ac = \
WB_orders_metrics[['Баркод', 'Процент комиссии', 'Стоимость логистики', 'Стоимость хранения', 'Себестоимость']]\
.copy() 
WB_orders_metrics_for_ac

,Баркод,Процент комиссии,Стоимость логистики,Стоимость хранения,Себестоимость
0,7432141949918,23,54.0,0.32,243.71
1,7432141949925,23,54.0,0.32,243.71
2,7432141949932,23,54.0,0.32,243.71
3,7432141550639,23,54.0,0.32,243.71
4,7432141649603,23,54.0,0.32,50.42
...,...,...,...,...,...
1245,4673613441636,23,54.0,0.32,74.78
1246,4664807383498,19,54.0,0.32,73.12
1247,4659437081876,17,255.6,3.68,390.38
1248,4674582725086,18,66.6,0.08,878.91


In [218]:
# LOG

In [219]:
df_for_param_nc = WB_orders_metrics_for_ac.drop('Себестоимость', axis=1).copy()
wwm.unique_barcode(df_for_param_nc)
df_for_param_nc['date'] = wwm.dt_now.strftime('%d-%B-%Y')
df_for_param_nc = mwm.redact_columns_with_bd_table(df_for_param_nc)
mwm.update_or_create_table_date(df_for_param_nc,
                                'wb_params_for_markups',
                                ['Баркод', 'date'],
                                'date')

In [220]:
# article_barcodes = mwm.select_table('article_barcodes')[['article_name', 'barcode']]
# article_barcodes.columns = ['Артикул', 'barcode']
# article_barcodes['Артикул'] = article_barcodes['Артикул'].astype(str)
# article_barcodes['barcode'] = article_barcodes['barcode'].astype(int)

# wb = mwm.select_table('dds_wb_orders', schema=mwm.dds_schema)
# wb = wwm.expand_df(wb[(wb['date'] >= datetime.date(2024, 1, 1)) \
#                       & (wb['date'] <= datetime.date(2024, 1, 28)) \
#                       & (wb['account']=='ir')].drop('account', axis=1).reset_index(drop=True),
#                    key_column='barcode',
#                    result_column='orders')
# date_cols = dt_col(wb)
# wb = wb.query(f"index in {[x for x in list(wb.index) if wb.iloc[x, 1:].sum() > 0]}")\
# .reset_index(drop=True)
# wb['sum_orders'] = [wb.iloc[x, 1:].sum() for x in list(wb.index)]

# wbp = mwm.select_table('dds_wb_prices', schema=mwm.dds_schema)
# wbp = wwm.expand_df(wbp[(wbp['date'] >= datetime.date(2024, 1, 1)) \
#                       & (wbp['date'] <= datetime.date(2024, 1, 28)) \
#                       & (wbp['account']=='ir')].drop('account', axis=1).reset_index(drop=True),
#                    key_column='barcode',
#                    result_column='prices',
#                    func='mean')
# wbp_dop = pd.read_excel(wwm.path_full_WB_df,
#                         sheet_name='WB_price_metric')
# wbp_dop = wbp_dop[['Баркод']+[x for x in dt_col(wbp_dop) if x in date_cols]]
# wbp_dop = wbp_dop[wbp_dop['Баркод']==7432141550851].reset_index(drop=True)
# wbp_dop.loc[0, 'Баркод'] = 7432141550855
# wbp_dop = wbp_dop.rename(columns={'Баркод': 'barcode'})
# wbp_dop_median = np.median(list(wbp_dop.iloc[:, 1:].values[0]))
# for row in date_cols:
#     if row not in dt_col(wbp_dop):
#         wbp_dop[row] = wbp_dop_median
# wbp_dop = wbp_dop[['barcode']+[y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in dt_col(wbp_dop)])]]
# wbp_dop = wbp_dop.astype(int)
# wbp = pd.concat([wbp, wbp_dop], ignore_index=True)
# wbp = wbp[wbp['barcode'].isin(list(set(wb['barcode'])))].reset_index(drop=True)
# wbp['mean_price'] = [round(np.mean([y for y in list(wbp.iloc[x, 1:]) if y > 0 ])) for x in list(wbp.index)]

# wb = wb.sort_values('barcode').reset_index(drop=True)
# wbp = wbp.sort_values('barcode').reset_index(drop=True)

# mwm.unique_barcode(wb.rename(columns={'barcode': 'Баркод'}))
# mwm.unique_barcode(wbp.rename(columns={'barcode': 'Баркод'}))
# if len(set(wb['barcode']) - set(wbp['barcode'])) != 0:
#     exit()
    
# wbv = wb[['barcode']].copy()
# for col in date_cols:
#     wbv[col] = None
# for i in range(len(wbv)):
#     for col in date_cols:
#         wbv.loc[i, col] = wb.loc[i][col] * wbp.loc[i][col]
# for row in list(set(wbp['barcode'])):
#     if row not in list(set(wb['barcode'])):
#         print(row)
# wbv['sum_earnings'] = [wbv.iloc[x, 1:].sum() for x in list(wbv.index)]

# cp = mwm.select_table_with_last_date_sort_list('cost_price', 'date')
# cp['Артикул'] = cp['Артикул'].astype('str')
# cp = cp[['Артикул', 'Себестоимость']]
# cp = cp[cp['Себестоимость']>0].reset_index(drop=True)
# cp = mwm.clear_article(cp, 'Артикул').reset_index(drop=True)
# Q = []
# for i in range(len(cp)):
#     art = cp.loc[i]['Артикул']
#     if len(cp[cp['Артикул']==art])> 1:
#         sub_df = cp[cp['Артикул']==art]
#         sub_df = sub_df.sort_values('Себестоимость')
#         for ind in list(sub_df.index)[:-1]:
#             Q.append(ind)
# Q = list(set(Q))
# cp = cp.query('index not in @Q').reset_index(drop=True)
# mwm.unique_items(cp)
# cp = cp.merge(article_barcodes, on='Артикул', how='left').dropna()
# cp['barcode'] = cp['barcode'].astype(int)
# cp = cp[['barcode', 'Себестоимость']]
# mwm.unique_barcode(cp.rename(columns={'barcode': 'Баркод'}))
# cp = cp[cp['barcode'].isin(list(set(wb['barcode'])))].sort_values('barcode').reset_index(drop=True)
# for col in date_cols:
#     cp[col] = None
#     for i in range(len(cp)):
#         cp.loc[i, col] = round((cp.loc[i]['Себестоимость'] * wb.loc[i][col]), 2)
# cp['sum_cost_price'] = [cp.iloc[x, 2:].sum() for x in list(cp.index)]

# WB_orders_metrics_for_ac_copy = WB_orders_metrics_for_ac.copy()
# WB_orders_metrics_for_ac_copy = WB_orders_metrics_for_ac_copy.drop('Себестоимость', axis=1)
# WB_orders_metrics_for_ac_copy.columns = ['barcode', 'Процент комиссии', 'Стоимость логистики', 'Стоимость хранения']
# WB_orders_metrics_for_ac_copy = WB_orders_metrics_for_ac_copy.merge(cp[['barcode', 'Себестоимость']], 
#                                                                     on='barcode',
#                                                                     how='right')
# wb_dop = wb[['barcode']].copy()
# for col in date_cols:
#     df1 = creating_calculated_features(WB_orders_metrics_for_ac_copy\
#                                  .merge(wbp[['barcode', col]].rename(columns={col: 'Цена'}),
#                                         on='barcode',
#                                         how='left'))[['barcode', 'Стоимость логистики',
#                                                       'Комиссия в рублях',
#                                                       'Стоимость хранения',
#                                                       'НДС',
#                                                       'Налог на прибыль']]\
#     .rename(columns={'Стоимость логистики': col + ' Логистика',
#                      'Комиссия в рублях': col + ' Комиссия',
#                      'Стоимость хранения': col + ' Хранение',
#                      'НДС': col + ' НДС',
#                      'Налог на прибыль': col + ' Налог на прибыль'})
#     df2 = wbp[['barcode', col]].rename(columns={col: 'Цена'})
#     df2['Эквайринг'] = df2['Цена'] / 100 * 0.75
#     df2 = df2[['barcode', 'Эквайринг']].rename(columns={'Эквайринг': col + ' Эквайринг'})
    
#     df1 = df1.merge(df2, on='barcode', how='left')
    
#     wb_dop = wb_dop.merge(df1,
#                           on='barcode',
#                           how='inner')

In [221]:
# wb_logistic = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'Логистика' in x]].copy()
# wb_logistic.columns = ['barcode'] + date_cols
# wb_komm = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'Комиссия' in x]].copy()
# wb_komm.columns = ['barcode'] + date_cols
# wb_hranenie = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'Хранение' in x]].copy()
# wb_hranenie.columns = ['barcode'] + date_cols
# wb_nds = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'НДС' in x]].copy()
# wb_nds.columns = ['barcode'] + date_cols
# wb_nnp = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'Налог на прибыль' in x]].copy()
# wb_nnp.columns = ['barcode'] + date_cols

# wb_ecv = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'Эквайринг' in x]].copy()
# wb_ecv.columns = ['barcode'] + date_cols

# for i in range(len(wb_logistic)):
#     for col in date_cols:
#         wb_logistic.loc[i, col] = round((wb_logistic.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_komm)):
#     for col in date_cols:
#         wb_komm.loc[i, col] = round((wb_komm.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_hranenie)):
#     for col in date_cols:
#         wb_hranenie.loc[i, col] = round((wb_hranenie.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_nds)):
#     for col in date_cols:
#         wb_nds.loc[i, col] = round((wb_nds.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_nnp)):
#     for col in date_cols:
#         wb_nnp.loc[i, col] = round((wb_nnp.loc[i][col] * wb.loc[i][col]), 2)
        
# for i in range(len(wb_ecv)):
#     for col in date_cols:
#         wb_ecv.loc[i, col] = round((wb_ecv.loc[i][col] * wb.loc[i][col]), 2)
        
# wb_logistic['sum_logistic'] = [wb_logistic.iloc[x, 1:].sum() for x in list(wb_logistic.index)]
# wb_komm['sum_komm'] = [wb_komm.iloc[x, 1:].sum() for x in list(wb_komm.index)]
# wb_hranenie['sum_hranenie'] = [wb_hranenie.iloc[x, 1:].sum() for x in list(wb_hranenie.index)]
# wb_nds['sum_nds'] = [wb_nds.iloc[x, 1:].sum() for x in list(wb_nds.index)]
# wb_nnp['sum_nnp'] = [wb_nnp.iloc[x, 1:].sum() for x in list(wb_nnp.index)]

# wb_ecv['sum_ecv'] = [wb_ecv.iloc[x, 1:].sum() for x in list(wb_ecv.index)]

# final_df = article_barcodes\
#           .merge(wb[['barcode']], on='barcode', how='right')\
#           .merge(wwm.Reestr(), on='Артикул', how='left')\
#           .merge(pd.read_excel(wwm.path_st,
#                                dtype={'Артикул': 'str'}), on='Артикул', how='left')\
#           .merge(wb[['barcode', 'sum_orders']], on='barcode', how='right')\
#           .merge(wbp[['barcode', 'mean_price']], on='barcode', how='right')\
#           .merge(wbv[['barcode', 'sum_earnings']], on='barcode', how='right')\
#           .merge(cp[['barcode', 'sum_cost_price']], on='barcode', how='right')\
#           .merge(wb_logistic[['barcode', 'sum_logistic']], on='barcode', how='right')\
#           .merge(wb_komm[['barcode', 'sum_komm']], on='barcode', how='right')\
#           .merge(wb_hranenie[['barcode', 'sum_hranenie']], on='barcode', how='right')\
#           .merge(wb_nds[['barcode', 'sum_nds']], on='barcode', how='right')\
#           .merge(wb_nnp[['barcode', 'sum_nnp']], on='barcode', how='right')\
#           .merge(wb_ecv[['barcode', 'sum_ecv']], on='barcode', how='right')

# with pd.ExcelWriter(path_downloads + 'Вб, заказы и расходы, 1-28 января 2024.xlsx') as writer:
#     wb.to_excel(writer, sheet_name="Заказы", index=False)
#     for column in wb:
#         column_width = max(wb[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb.columns.get_loc(column)
#         writer.sheets['Заказы'].set_column(col_idx, col_idx, column_width)
#     wbp.to_excel(writer, sheet_name="Цены", index=False)
#     for column in wbp:
#         column_width = max(wbp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wbp.columns.get_loc(column)
#         writer.sheets['Цены'].set_column(col_idx, col_idx, column_width)
#     wbv.to_excel(writer, sheet_name="Выручка", index=False)
#     for column in wbv:
#         column_width = max(wbv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wbv.columns.get_loc(column)
#         writer.sheets['Выручка'].set_column(col_idx, col_idx, column_width)
#     cp.to_excel(writer, sheet_name="Себестоимость", index=False)
#     for column in cp:
#         column_width = max(cp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = cp.columns.get_loc(column)
#         writer.sheets['Себестоимость'].set_column(col_idx, col_idx, column_width)
#     wb_logistic.to_excel(writer, sheet_name="Логистика", index=False)
#     for column in wb_logistic:
#         column_width = max(wb_logistic[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_logistic.columns.get_loc(column)
#         writer.sheets['Логистика'].set_column(col_idx, col_idx, column_width)
#     wb_komm.to_excel(writer, sheet_name="Комиссия", index=False)
#     for column in wb_komm:
#         column_width = max(wb_komm[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_komm.columns.get_loc(column)
#         writer.sheets['Комиссия'].set_column(col_idx, col_idx, column_width)
#     wb_hranenie.to_excel(writer, sheet_name="Хранение", index=False)
#     for column in wb_hranenie:
#         column_width = max(wb_hranenie[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_hranenie.columns.get_loc(column)
#         writer.sheets['Хранение'].set_column(col_idx, col_idx, column_width)
#     wb_nds.to_excel(writer, sheet_name="НДС", index=False)
#     for column in wb_nds:
#         column_width = max(wb_nds[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_nds.columns.get_loc(column)
#         writer.sheets['НДС'].set_column(col_idx, col_idx, column_width)
        
#     wb_nnp.to_excel(writer, sheet_name="Налог на прибыль", index=False)
#     for column in wb_nnp:
#         column_width = max(wb_nnp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_nnp.columns.get_loc(column)
#         writer.sheets['Налог на прибыль'].set_column(col_idx, col_idx, column_width)

#     wb_ecv.to_excel(writer, sheet_name="Эквайринг", index=False)
#     for column in wb_ecv:
#         column_width = max(wb_ecv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_ecv.columns.get_loc(column)
#         writer.sheets['Эквайринг'].set_column(col_idx, col_idx, column_width)
        
#     final_df.to_excel(writer, sheet_name="Итоговая", index=False)
#     for column in final_df:
#         column_width = max(final_df[column].astype(str).map(len).max()+2, len(column))
#         col_idx = final_df.columns.get_loc(column)
#         writer.sheets['Итоговая'].set_column(col_idx, col_idx, column_width)
# writer.save()

In [222]:
# article_barcodes = mwm.select_table('article_barcodes')[['article_name', 'barcode']]
# article_barcodes.columns = ['Артикул', 'barcode']
# article_barcodes['Артикул'] = article_barcodes['Артикул'].astype(str)
# article_barcodes['barcode'] = article_barcodes['barcode'].astype(int)

# wb = mwm.select_table('dds_wb_orders', schema=mwm.dds_schema)
# wb = wwm.expand_df(wb[(wb['date'] >= datetime.date(2024, 1, 1)) \
#                       & (wb['date'] <= datetime.date(2024, 1, 28)) \
#                       & (wb['account']=='ir')].drop('account', axis=1).reset_index(drop=True),
#                    key_column='barcode',
#                    result_column='orders')
# date_cols = dt_col(wb)
# wb = wb.query(f"index in {[x for x in list(wb.index) if wb.iloc[x, 1:].sum() > 0]}")\
# .reset_index(drop=True)
# wb['sum_orders'] = [wb.iloc[x, 1:].sum() for x in list(wb.index)]

# wbp = mwm.select_table('dds_wb_prices', schema=mwm.dds_schema)
# wbp = wwm.expand_df(wbp[(wbp['date'] >= datetime.date(2024, 1, 1)) \
#                       & (wbp['date'] <= datetime.date(2024, 1, 28)) \
#                       & (wbp['account']=='ir')].drop('account', axis=1).reset_index(drop=True),
#                    key_column='barcode',
#                    result_column='prices',
#                    func='mean')
# wbp_dop = pd.read_excel(wwm.path_full_WB_df,
#                         sheet_name='WB_price_metric')
# wbp_dop = wbp_dop[['Баркод']+[x for x in dt_col(wbp_dop) if x in date_cols]]
# wbp_dop = wbp_dop[wbp_dop['Баркод']==7432141550851].reset_index(drop=True)
# wbp_dop.loc[0, 'Баркод'] = 7432141550855
# wbp_dop = wbp_dop.rename(columns={'Баркод': 'barcode'})
# wbp_dop_median = np.median(list(wbp_dop.iloc[:, 1:].values[0]))
# for row in date_cols:
#     if row not in dt_col(wbp_dop):
#         wbp_dop[row] = wbp_dop_median
# wbp_dop = wbp_dop[['barcode']+[y.strftime('%d-%B-%Y') for y in sorted([pd.to_datetime(x) for x in dt_col(wbp_dop)])]]
# wbp_dop = wbp_dop.astype(int)
# wbp = pd.concat([wbp, wbp_dop], ignore_index=True)
# wbp = wbp[wbp['barcode'].isin(list(set(wb['barcode'])))].reset_index(drop=True)
# wbp['mean_price'] = [round(np.mean([y for y in list(wbp.iloc[x, 1:]) if y > 0 ])) for x in list(wbp.index)]

# wb = wb.sort_values('barcode').reset_index(drop=True)
# wbp = wbp.sort_values('barcode').reset_index(drop=True)

# mwm.unique_barcode(wb.rename(columns={'barcode': 'Баркод'}))
# mwm.unique_barcode(wbp.rename(columns={'barcode': 'Баркод'}))
# if len(set(wb['barcode']) - set(wbp['barcode'])) != 0:
#     exit()
    
# wbv = wb[['barcode']].copy()
# for col in date_cols:
#     wbv[col] = None
# for i in range(len(wbv)):
#     for col in date_cols:
#         wbv.loc[i, col] = wb.loc[i][col] * wbp.loc[i][col]
# for row in list(set(wbp['barcode'])):
#     if row not in list(set(wb['barcode'])):
#         print(row)
# wbv['sum_earnings'] = [wbv.iloc[x, 1:].sum() for x in list(wbv.index)]

# cp = mwm.select_table_with_last_date_sort_list('cost_price', 'date')
# cp['Артикул'] = cp['Артикул'].astype('str')
# cp = cp[['Артикул', 'Себестоимость']]
# cp = cp[cp['Себестоимость']>0].reset_index(drop=True)
# cp = mwm.clear_article(cp, 'Артикул').reset_index(drop=True)
# Q = []
# for i in range(len(cp)):
#     art = cp.loc[i]['Артикул']
#     if len(cp[cp['Артикул']==art])> 1:
#         sub_df = cp[cp['Артикул']==art]
#         sub_df = sub_df.sort_values('Себестоимость')
#         for ind in list(sub_df.index)[:-1]:
#             Q.append(ind)
# Q = list(set(Q))
# cp = cp.query('index not in @Q').reset_index(drop=True)
# mwm.unique_items(cp)
# cp = cp.merge(article_barcodes, on='Артикул', how='left').dropna()
# cp['barcode'] = cp['barcode'].astype(int)
# cp = cp[['barcode', 'Себестоимость']]
# mwm.unique_barcode(cp.rename(columns={'barcode': 'Баркод'}))
# cp = cp[cp['barcode'].isin(list(set(wb['barcode'])))].sort_values('barcode').reset_index(drop=True)
# for col in date_cols:
#     cp[col] = None
#     for i in range(len(cp)):
#         cp.loc[i, col] = round((cp.loc[i]['Себестоимость'] * wb.loc[i][col]), 2)
# cp['sum_cost_price'] = [cp.iloc[x, 2:].sum() for x in list(cp.index)]

# WB_orders_metrics_for_ac_copy = WB_orders_metrics_for_ac.copy()
# WB_orders_metrics_for_ac_copy = WB_orders_metrics_for_ac_copy.drop('Себестоимость', axis=1)
# WB_orders_metrics_for_ac_copy.columns = ['barcode', 'Процент комиссии', 'Стоимость логистики', 'Стоимость хранения']
# WB_orders_metrics_for_ac_copy = WB_orders_metrics_for_ac_copy.merge(cp[['barcode', 'Себестоимость']], 
#                                                                     on='barcode',
#                                                                     how='right')

# wb_dop = wb[['barcode']].copy()
# for col in date_cols:
#     wb_dop = wb_dop.merge(creating_calculated_features(WB_orders_metrics_for_ac_copy\
#                                  .merge(wbp[['barcode', col]].rename(columns={col: 'Цена'}),
#                                         on='barcode',
#                                         how='left'))[['barcode', 'Стоимость логистики',
#                                                       'Комиссия в рублях',
#                                                       'Стоимость хранения',
#                                                       'НДС',
#                                                       'Налог на прибыль']]\
#     .rename(columns={'Стоимость логистики': col + ' Логистика',
#                      'Комиссия в рублях': col + ' Комиссия',
#                      'Стоимость хранения': col + ' Хранение',
#                      'НДС': col + ' НДС',
#                      'Налог на прибыль': col + ' Налог на прибыль'}),
#                           on='barcode',
#                           how='inner')
    
# wb_logistic = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'Логистика' in x]].copy()
# wb_logistic.columns = ['barcode'] + date_cols
# wb_komm = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'Комиссия' in x]].copy()
# wb_komm.columns = ['barcode'] + date_cols
# wb_hranenie = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'Хранение' in x]].copy()
# wb_hranenie.columns = ['barcode'] + date_cols
# wb_nds = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'НДС' in x]].copy()
# wb_nds.columns = ['barcode'] + date_cols
# wb_nnp = wb_dop[['barcode']+[x for x in list(wb_dop.columns) if 'Налог на прибыль' in x]].copy()
# wb_nnp.columns = ['barcode'] + date_cols
# for i in range(len(wb_logistic)):
#     for col in date_cols:
#         wb_logistic.loc[i, col] = round((wb_logistic.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_komm)):
#     for col in date_cols:
#         wb_komm.loc[i, col] = round((wb_komm.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_hranenie)):
#     for col in date_cols:
#         wb_hranenie.loc[i, col] = round((wb_hranenie.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_nds)):
#     for col in date_cols:
#         wb_nds.loc[i, col] = round((wb_nds.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_nnp)):
#     for col in date_cols:
#         wb_nnp.loc[i, col] = round((wb_nnp.loc[i][col] * wb.loc[i][col]), 2)
# wb_logistic['sum_logistic'] = [wb_logistic.iloc[x, 1:].sum() for x in list(wb_logistic.index)]
# wb_komm['sum_komm'] = [wb_komm.iloc[x, 1:].sum() for x in list(wb_komm.index)]
# wb_hranenie['sum_hranenie'] = [wb_hranenie.iloc[x, 1:].sum() for x in list(wb_hranenie.index)]
# wb_nds['sum_nds'] = [wb_nds.iloc[x, 1:].sum() for x in list(wb_nds.index)]
# wb_nnp['sum_nnp'] = [wb_nnp.iloc[x, 1:].sum() for x in list(wb_nnp.index)]

# final_df = article_barcodes\
#           .merge(wb[['barcode']], on='barcode', how='right')\
#           .merge(wwm.Reestr(), on='Артикул', how='left')\
#           .merge(pd.read_excel(wwm.path_st,
#                                dtype={'Артикул': 'str'}), on='Артикул', how='left')\
#           .merge(wb[['barcode', 'sum_orders']], on='barcode', how='right')\
#           .merge(wbp[['barcode', 'mean_price']], on='barcode', how='right')\
#           .merge(wbv[['barcode', 'sum_earnings']], on='barcode', how='right')\
#           .merge(cp[['barcode', 'sum_cost_price']], on='barcode', how='right')\
#           .merge(wb_logistic[['barcode', 'sum_logistic']], on='barcode', how='right')\
#           .merge(wb_komm[['barcode', 'sum_komm']], on='barcode', how='right')\
#           .merge(wb_hranenie[['barcode', 'sum_hranenie']], on='barcode', how='right')\
#           .merge(wb_nds[['barcode', 'sum_nds']], on='barcode', how='right')\
#           .merge(wb_nnp[['barcode', 'sum_nnp']], on='barcode', how='right')

# with pd.ExcelWriter(path_downloads + 'Вб, заказы и расходы, 1-28 января 2024.xlsx') as writer:
#     wb.to_excel(writer, sheet_name="Заказы", index=False)
#     for column in wb:
#         column_width = max(wb[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb.columns.get_loc(column)
#         writer.sheets['Заказы'].set_column(col_idx, col_idx, column_width)
#     wbp.to_excel(writer, sheet_name="Цены", index=False)
#     for column in wbp:
#         column_width = max(wbp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wbp.columns.get_loc(column)
#         writer.sheets['Цены'].set_column(col_idx, col_idx, column_width)
#     wbv.to_excel(writer, sheet_name="Выручка", index=False)
#     for column in wbv:
#         column_width = max(wbv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wbv.columns.get_loc(column)
#         writer.sheets['Выручка'].set_column(col_idx, col_idx, column_width)
#     cp.to_excel(writer, sheet_name="Себестоимость", index=False)
#     for column in cp:
#         column_width = max(cp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = cp.columns.get_loc(column)
#         writer.sheets['Себестоимость'].set_column(col_idx, col_idx, column_width)
#     wb_logistic.to_excel(writer, sheet_name="Логистика", index=False)
#     for column in wb_logistic:
#         column_width = max(wb_logistic[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_logistic.columns.get_loc(column)
#         writer.sheets['Логистика'].set_column(col_idx, col_idx, column_width)
#     wb_komm.to_excel(writer, sheet_name="Комиссия", index=False)
#     for column in wb_komm:
#         column_width = max(wb_komm[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_komm.columns.get_loc(column)
#         writer.sheets['Комиссия'].set_column(col_idx, col_idx, column_width)
#     wb_hranenie.to_excel(writer, sheet_name="Хранение", index=False)
#     for column in wb_hranenie:
#         column_width = max(wb_hranenie[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_hranenie.columns.get_loc(column)
#         writer.sheets['Хранение'].set_column(col_idx, col_idx, column_width)
#     wb_nds.to_excel(writer, sheet_name="НДС", index=False)
#     for column in wb_nds:
#         column_width = max(wb_nds[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_nds.columns.get_loc(column)
#         writer.sheets['НДС'].set_column(col_idx, col_idx, column_width)
#     wb_nnp.to_excel(writer, sheet_name="Налог на прибыль", index=False)
#     for column in wb_nnp:
#         column_width = max(wb_nnp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_nnp.columns.get_loc(column)
#         writer.sheets['Налог на прибыль'].set_column(col_idx, col_idx, column_width)
#     final_df.to_excel(writer, sheet_name="Итоговая", index=False)
#     for column in final_df:
#         column_width = max(final_df[column].astype(str).map(len).max()+2, len(column))
#         col_idx = final_df.columns.get_loc(column)
#         writer.sheets['Итоговая'].set_column(col_idx, col_idx, column_width)
# writer.save()

In [223]:
# cp = wb_art.merge(cp, on='Артикул', how='inner')
# cp = cp[['Баркод', 'Себестоимость']]
# mwm.unique_barcode(cp)
# cp = cp[cp['Баркод'].isin(list(set(wb['Баркод'])))].sort_values('Баркод').reset_index(drop=True)
# for col in date_cols:
#     cp[col] = None
#     for i in range(len(cp)):
#         cp.loc[i, col] = round((cp.loc[i]['Себестоимость'] * wb.loc[i][col]), 2)
# cp['sum_cost_price'] = [cp.iloc[x, 2:].sum() for x in list(cp.index)]

In [224]:
# import maslow_working_module as mwm
# wb = pd.read_excel(wwm.path_full_WB_df,
#                    dtype={'Баркод': 'int',
#                           'Артикул': 'str'})
# date_cols = [x for x in dt_col(wb) if 'November-2023' in x and int(x[:2]) in list(range(20, 27))]
# wb_art = wb[['Артикул', 'Баркод']].copy()
# wb = wb[['Баркод']+date_cols]
# wb = wb.query(f"index in {[x for x in list(wb.index) if wb.iloc[x, 1:].sum() > 0]}")\
# .reset_index(drop=True)
# wb['sum_orders'] = [wb.iloc[x, 1:].sum() for x in list(wb.index)]
# wbp = mwm.get_tabl_from_bd('wb_prices', flag=1)
# wbp['price'] = (wbp['price'] * (100 - wbp['discount'])) / 100
# wbp = wbp[['Баркод', 'price', 'date']]
# wbp['price'] = wbp['price'].astype(int)
# wbp['date'] = [pd.to_datetime(x).strftime('%d-%B-%Y') for x in list(wbp['date'])]
# wbp = wbp[wbp['date'].isin(date_cols)].reset_index(drop=True)
# df_0 = pd.DataFrame(list(set(wbp['Баркод'])), columns=['Баркод'])
# for row in date_cols:
#     df_0 = df_0.merge(wbp[wbp['date']==row].groupby('Баркод')['price'].mean()\
#     .reset_index(drop=False).rename(columns={'price': row}), on='Баркод', how='left')
# df_0 = df_0.fillna(0)
# wbp, df_0 = df_0, None
# wbp = wbp.astype(int)
# wbp = wbp[wbp['Баркод'].isin(list(set(wb['Баркод'])))].reset_index(drop=True)
# wbp['mean_price'] = [round(np.mean([y for y in list(wbp.iloc[x, 1:]) if y > 0])) for x in list(wbp.index)]
# wbp = wbp.sort_values('Баркод').reset_index(drop=True)
# wb = wb.sort_values('Баркод').reset_index(drop=True)
# wbv = wb[['Баркод']].copy()
# for col in date_cols:
#     wbv[col] = None
# for i in range(len(wbv)):
#     for col in date_cols:
#         wbv.loc[i, col] = wb.loc[i][col] * wbp.loc[i][col]
# for row in list(set(wbp['Баркод'])):
#     if row not in list(set(wb['Баркод'])):
#         print(row)
# wbv['sum_earnings'] = [wbv.iloc[x, 1:].sum() for x in list(wbv.index)]
# cp = mwm.select_table_with_last_date_sort_list('cost_price', 'date')
# cp['Артикул'] = cp['Артикул'].astype('str')
# cp = cp[['Артикул', 'Себестоимость']]
# cp = cp[cp['Себестоимость']>0].reset_index(drop=True)
# cp = mwm.clear_article(cp, 'Артикул').reset_index(drop=True)
# Q = []
# for i in range(len(cp)):
#     art = cp.loc[i]['Артикул']
#     if len(cp[cp['Артикул']==art])> 1:
#         sub_df = cp[cp['Артикул']==art]
#         sub_df = sub_df.sort_values('Себестоимость')
#         for ind in list(sub_df.index)[:-1]:
#             Q.append(ind)
# Q = list(set(Q))
# cp = cp.query('index not in @Q').reset_index(drop=True)
# mwm.unique_items(cp)
# cp = wb_art.merge(cp, on='Артикул', how='inner')
# cp = cp[['Баркод', 'Себестоимость']]
# mwm.unique_barcode(cp)
# cp = cp[cp['Баркод'].isin(list(set(wb['Баркод'])))].sort_values('Баркод').reset_index(drop=True)
# for col in date_cols:
#     cp[col] = None
#     for i in range(len(cp)):
#         cp.loc[i, col] = round((cp.loc[i]['Себестоимость'] * wb.loc[i][col]), 2)
# cp['sum_cost_price'] = [cp.iloc[x, 2:].sum() for x in list(cp.index)]
# WB_orders_metrics_for_ac = WB_orders_metrics_for_ac.drop('Себестоимость', axis=1)
# WB_orders_metrics_for_ac = WB_orders_metrics_for_ac.merge(cp[['Баркод', 'Себестоимость']], 
#                                                           on='Баркод',
#                                                           how='right')
# wb_dop = wb[['Баркод']].copy()
# for col in date_cols:
#     wb_dop = wb_dop.merge(creating_calculated_features(WB_orders_metrics_for_ac\
#                                  .merge(wbp[['Баркод', col]].rename(columns={col: 'Цена'}),
#                                         on='Баркод',
#                                         how='left'))[['Баркод', 'Стоимость логистики',
#                                                       'Комиссия в рублях',
#                                                       'Стоимость хранения',
#                                                       'НДС',
#                                                       'Налог на прибыль']]\
#     .rename(columns={'Стоимость логистики': col + ' Логистика',
#                      'Комиссия в рублях': col + ' Комиссия',
#                      'Стоимость хранения': col + ' Хранение',
#                      'НДС': col + ' НДС',
#                      'Налог на прибыль': col + ' Налог на прибыль'}),
#                           on='Баркод',
#                           how='inner')
# wb_logistic = wb_dop[['Баркод']+[x for x in list(wb_dop.columns) if 'Логистика' in x]].copy()
# wb_logistic.columns = ['Баркод'] + date_cols
# wb_komm = wb_dop[['Баркод']+[x for x in list(wb_dop.columns) if 'Комиссия' in x]].copy()
# wb_komm.columns = ['Баркод'] + date_cols
# wb_hranenie = wb_dop[['Баркод']+[x for x in list(wb_dop.columns) if 'Хранение' in x]].copy()
# wb_hranenie.columns = ['Баркод'] + date_cols
# wb_nds = wb_dop[['Баркод']+[x for x in list(wb_dop.columns) if 'НДС' in x]].copy()
# wb_nds.columns = ['Баркод'] + date_cols
# wb_nnp = wb_dop[['Баркод']+[x for x in list(wb_dop.columns) if 'Налог на прибыль' in x]].copy()
# wb_nnp.columns = ['Баркод'] + date_cols
# for i in range(len(wb_logistic)):
#     for col in date_cols:
#         wb_logistic.loc[i, col] = round((wb_logistic.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_komm)):
#     for col in date_cols:
#         wb_komm.loc[i, col] = round((wb_komm.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_hranenie)):
#     for col in date_cols:
#         wb_hranenie.loc[i, col] = round((wb_hranenie.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_nds)):
#     for col in date_cols:
#         wb_nds.loc[i, col] = round((wb_nds.loc[i][col] * wb.loc[i][col]), 2)
# for i in range(len(wb_nnp)):
#     for col in date_cols:
#         wb_nnp.loc[i, col] = round((wb_nnp.loc[i][col] * wb.loc[i][col]), 2)
# wb_logistic['sum_logistic'] = [wb_logistic.iloc[x, 1:].sum() for x in list(wb_logistic.index)]
# wb_komm['sum_komm'] = [wb_komm.iloc[x, 1:].sum() for x in list(wb_komm.index)]
# wb_hranenie['sum_hranenie'] = [wb_hranenie.iloc[x, 1:].sum() for x in list(wb_hranenie.index)]
# wb_nds['sum_nds'] = [wb_nds.iloc[x, 1:].sum() for x in list(wb_nds.index)]
# wb_nnp['sum_nnp'] = [wb_nnp.iloc[x, 1:].sum() for x in list(wb_nnp.index)]
# final_df = wb_art.merge(wwm.Reestr(), on='Артикул', how='left')\
#                  .merge(pd.read_excel(wwm.path_st,
#                                       dtype={'Артикул': 'str'}), on='Артикул', how='left')\
#                  .merge(wb[['Баркод', 'sum_orders']], on='Баркод', how='right')\
#                  .merge(wbp[['Баркод', 'mean_price']], on='Баркод', how='right')\
#                  .merge(wbv[['Баркод', 'sum_earnings']], on='Баркод', how='right')\
#                  .merge(cp[['Баркод', 'sum_cost_price']], on='Баркод', how='right')\
#                  .merge(wb_logistic[['Баркод', 'sum_logistic']], on='Баркод', how='right')\
#                  .merge(wb_komm[['Баркод', 'sum_komm']], on='Баркод', how='right')\
#                  .merge(wb_hranenie[['Баркод', 'sum_hranenie']], on='Баркод', how='right')\
#                  .merge(wb_nds[['Баркод', 'sum_nds']], on='Баркод', how='right')\
#                  .merge(wb_nnp[['Баркод', 'sum_nnp']], on='Баркод', how='right')
# with pd.ExcelWriter(path_downloads + 'Вб, заказы и расходы, 20-26 ноября.xlsx') as writer:
#     wb.to_excel(writer, sheet_name="Заказы", index=False)
#     for column in wb:
#         column_width = max(wb[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb.columns.get_loc(column)
#         writer.sheets['Заказы'].set_column(col_idx, col_idx, column_width)
#     wbp.to_excel(writer, sheet_name="Цены", index=False)
#     for column in wbp:
#         column_width = max(wbp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wbp.columns.get_loc(column)
#         writer.sheets['Цены'].set_column(col_idx, col_idx, column_width)
#     wbv.to_excel(writer, sheet_name="Выручка", index=False)
#     for column in wbv:
#         column_width = max(wbv[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wbv.columns.get_loc(column)
#         writer.sheets['Выручка'].set_column(col_idx, col_idx, column_width)
#     cp.to_excel(writer, sheet_name="Себестоимость", index=False)
#     for column in cp:
#         column_width = max(cp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = cp.columns.get_loc(column)
#         writer.sheets['Себестоимость'].set_column(col_idx, col_idx, column_width)
#     wb_logistic.to_excel(writer, sheet_name="Логистика", index=False)
#     for column in wb_logistic:
#         column_width = max(wb_logistic[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_logistic.columns.get_loc(column)
#         writer.sheets['Логистика'].set_column(col_idx, col_idx, column_width)
#     wb_komm.to_excel(writer, sheet_name="Комиссия", index=False)
#     for column in wb_komm:
#         column_width = max(wb_komm[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_komm.columns.get_loc(column)
#         writer.sheets['Комиссия'].set_column(col_idx, col_idx, column_width)
#     wb_hranenie.to_excel(writer, sheet_name="Хранение", index=False)
#     for column in wb_hranenie:
#         column_width = max(wb_hranenie[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_hranenie.columns.get_loc(column)
#         writer.sheets['Хранение'].set_column(col_idx, col_idx, column_width)
#     wb_nds.to_excel(writer, sheet_name="НДС", index=False)
#     for column in wb_nds:
#         column_width = max(wb_nds[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_nds.columns.get_loc(column)
#         writer.sheets['НДС'].set_column(col_idx, col_idx, column_width)
#     wb_nnp.to_excel(writer, sheet_name="Налог на прибыль", index=False)
#     for column in wb_nnp:
#         column_width = max(wb_nnp[column].astype(str).map(len).max()+2, len(column))
#         col_idx = wb_nnp.columns.get_loc(column)
#         writer.sheets['Налог на прибыль'].set_column(col_idx, col_idx, column_width)
#     final_df.to_excel(writer, sheet_name="Итоговая", index=False)
#     for column in final_df:
#         column_width = max(final_df[column].astype(str).map(len).max()+2, len(column))
#         col_idx = final_df.columns.get_loc(column)
#         writer.sheets['Итоговая'].set_column(col_idx, col_idx, column_width)
# writer.save()

In [225]:
wb_markups.creating_calculated_features(['G0650'], [1374])

[['G0650', 287.47]]

In [226]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,08-April-2024,Остаток,Предмет,Процент комиссии,Стоимость логистики,Стоимость хранения,Себестоимость,Цена до скидки,Скидка,Цена
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,0,0,0,0,0,...,0,0,Чехлы для планшетов,23,54.0,0.32,243.71,5940.0,90.0,594
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,0,0,0,1,1,...,0,0,Чехлы для планшетов,23,54.0,0.32,243.71,2300.0,79.0,483
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,2,0,0,1,0,...,0,0,Чехлы для планшетов,23,54.0,0.32,243.71,3144.0,84.0,503
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,0,0,...,0,0,Чехлы для планшетов,23,54.0,0.32,243.71,4236.0,81.0,804
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,1,3,1,0,2,...,4,615,Чехлы для наушников,23,54.0,0.32,50.42,1150.0,90.0,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,2,0,Расчески,23,54.0,0.32,74.78,390.0,0.0,390
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,8,204,Плитка зеркальная,19,54.0,0.32,73.12,646.0,48.0,335
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,1,219,Подушки автомобильные,17,255.6,3.68,390.38,1590.0,0.0,1590
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,469,Фонари спортивные,18,66.6,0.08,878.91,2627.0,25.0,1970


In [227]:
WB_orders_metrics = creating_calculated_features(WB_orders_metrics)

### Проверка 

In [228]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,Цена до скидки,Скидка,Цена,Комиссия в рублях,НДС,Налог на прибыль,Налог,Прибыль,Наценка,Процент налога
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,0,0,0,0,0,...,5940.0,90.0,594,154.44,23.641667,22.750667,46.392333,91.002667,37.34,7.810157
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,0,0,0,1,1,...,2300.0,79.0,483,125.58,9.951667,9.227167,19.178833,36.908667,15.14,3.970773
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,2,0,0,1,0,...,3144.0,84.0,503,130.78,12.418333,11.663833,24.082167,46.655333,19.14,4.787707
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,0,0,...,4236.0,81.0,804,209.04,49.541667,48.335667,97.877333,193.342667,79.33,12.173798
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,1,3,1,0,2,...,1150.0,90.0,115,29.90,-3.220000,-3.392500,-6.612500,-13.570000,-26.91,-5.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,390.0,0.0,390,101.40,26.636667,26.051667,52.688333,104.206667,139.35,13.509829
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,646.0,48.0,335,73.70,22.363333,21.860833,44.224167,87.443333,119.59,13.201244
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,1590.0,0.0,1590,318.00,104.336667,101.951667,206.288333,407.806667,104.46,12.974109
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,2627.0,25.0,1970,413.70,101.798333,98.843333,200.641667,395.373333,44.98,10.184856


In [229]:
dfc = 'WB_orders_metrics'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [230]:
def okr(y):
    try:
        y = round(y, 2)
    except:
        None
    return y

In [231]:
def dop_st_log(y, x, z):
    s = 0
    if y <= 5 and y > 0:
        s = x
    elif y > 5:
        s = ((round(y) - 5) * z) + x
    else:
        None
    return s   

In [232]:
LOG['монопалета'] = [[okr(LOG.loc[i]['Стоимость логистики монопалета (база)']),
                    okr(LOG.loc[i]['Стоимость логистики монопалета (доп. литр)'])] for i in list(range(len(LOG)))]

LOG['короб'] = [[okr(LOG.loc[i]['Стоимость логистики короб (база)']), 
                    okr(LOG.loc[i]['Стоимость логистики короб (доп. литр)'])] for i in list(range(len(LOG)))]

LOG = LOG[['Склад', 'монопалета', 'короб']]

In [233]:
WB_size666['Баркод'] = WB_size666['Баркод'].astype(int)
WB_orders_metrics666['Баркод'] = WB_orders_metrics666['Баркод'].astype(int)
if len(WB_size666) - len(list(set(WB_size666['Баркод']))) != 0 or len(WB_orders_metrics666) - len(list(set(WB_orders_metrics666['Баркод']))) != 0:
    print('Ошибка')
    float(Nan)
else:
    WB_orders_metrics666 = WB_orders_metrics666.merge(WB_size666, on='Баркод', how='inner').reset_index(drop=True)
    
for i in range(len(WB_orders_metrics666)):
    if WB_orders_metrics666.loc[i]['Объем, л'] == 0:
        WB_orders_metrics666['Объем, л'].loc[i] = 0.1
        
WB_orders_metrics666 = WB_orders_metrics666[WB_orders_metrics666['Объем, л']>0].reset_index(drop=True)
WB_orders_metrics666 = WB_orders_metrics666[list(WB_orders_metrics666.columns[:5])+['Процент комиссии', 'Объем, л']]

In [234]:
dfl = WB_orders_metrics666[['Баркод']].copy()
for col in ['монопалета', 'короб']:
    for i in tqdm(range(len(LOG))):
        if str(LOG.loc[i][col]) != '[nan, nan]':
            WB_orders_metrics666['Стоимость логистики'] = list(map(lambda x: dop_st_log(WB_orders_metrics666.loc[x]['Объем, л'], LOG.loc[i][col][0], LOG.loc[i][col][1]), list(WB_orders_metrics666.index)))
            WB_orders_metrics666 = add_cost_price(WB_orders_metrics666)
            WB_orders_metrics666 = add_price(WB_orders_metrics666, price)
            WB_orders_metrics666 = creating_calculated_features(WB_orders_metrics666)
            dfl = dfl.merge(WB_orders_metrics666[['Баркод', 'Наценка']], on='Баркод', how='outer')
            dfl = dfl.rename(columns={'Наценка': f"Наценка_{LOG.loc[i]['Склад']}_{col}"})
            WB_orders_metrics666 = WB_orders_metrics666[['Артикул', 'Наименование', 'Номенклатура', 'Баркод', 'ТМ','Процент комиссии', 'Объем, л']]
        else:
            None
        
WB_orders_metrics666 = WB_orders_metrics666[['Артикул', 'Наименование', 'Номенклатура', 'Баркод', 'ТМ']]
dfl = WB_orders_metrics666.merge(dfl, on='Баркод', how='right')

100%|███████████████████████████████████████████| 44/44 [00:07<00:00,  5.84it/s]


In [235]:
for i in range(len(dfl)):
    for row in dfl.columns[5:]:
        if str(dfl.loc[i][row]) == 'inf':
            dfl[row].loc[i] = 0

In [236]:
Q = []
W = []
E = []
for i in range(len(dfl)):
    R = {}
    for row in dfl.columns[5:]:
        R[row] = dfl.loc[i][row]
    R = dict((sorted(R.items(), key=itemgetter(1))[::-1]))
    Q.append(list(R.keys())[0] + f": {R[list(R.keys())[0]]}")
    W.append(list(R.keys())[1] + f": {R[list(R.keys())[1]]}")
    E.append(list(R.keys())[2] + f": {R[list(R.keys())[2]]}")

        
        
        
dfl['          '] = None
dfl['Лучшая наценка_1'] = Q
dfl['Лучшая наценка_2'] = W
dfl['Лучшая наценка_3'] = E
Q = None
W = None
E = None
R = None   

### Проверка метрик на русские буквы в "Артикулах"

In [237]:
len(WB_orders_metrics['Артикул'].unique())

1234

In [238]:
WB_orders_metrics = letter_substitution(WB_orders_metrics)

In [239]:
len(WB_orders_metrics['Артикул'].unique())

1234

In [240]:
len(WB_orders_metrics) - len(WB_orders_metrics['Баркод'].unique())

0

In [241]:
WB_orders_metrics['Баркод'] = WB_orders_metrics['Баркод'].astype(int)

### Создание метрик с полными расчетными признаками

In [242]:
WB_full_features_metric = WB_orders_metrics.copy()

### Удаление ненужных для репрезентативной таблицы признаков

In [243]:
WB_orders_metrics = drop_columns_for_presentation_df(WB_orders_metrics)

### Добавление "РЦ"

In [244]:
WB_orders_metrics = add_rc(WB_orders_metrics)

### Проверка 

In [245]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена,Наценка,Остаток на РЦ
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,594,37.34,3.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,0,0,0,1,1,...,0,0,0,0,0,0,0,483,15.14,4.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,2,0,0,1,0,...,0,0,0,0,0,0,0,503,19.14,4.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,804,79.33,0.0
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,1,3,1,0,2,...,2,2,3,5,7,4,615,115,-26.91,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,1,0,0,1,0,2,0,390,139.35,0.0
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,1,2,1,2,8,204,335,119.59,2234.0
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,1,219,1590,104.46,830.0
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,469,1970,44.98,1669.0


In [246]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена,Наценка,Остаток на РЦ
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,594,37.34,3.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,0,0,0,1,1,...,0,0,0,0,0,0,0,483,15.14,4.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,2,0,0,1,0,...,0,0,0,0,0,0,0,503,19.14,4.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,804,79.33,0.0
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,1,3,1,0,2,...,2,2,3,5,7,4,615,115,-26.91,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,1,0,0,1,0,2,0,390,139.35,0.0
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,1,2,1,2,8,204,335,119.59,2234.0
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,1,219,1590,104.46,830.0
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,469,1970,44.98,1669.0


### Добавление признака "Статус"

In [247]:
Q = list(pd.read_excel(wwm.path_st,
         dtype={'Артикул': 'str'}).merge(WB_orders_metrics[['Артикул']], 
                                         on='Артикул',
                                         how='right')['Статус'])
WB_orders_metrics.insert(5, 'Статус', Q)

In [248]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена,Наценка,Остаток на РЦ
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,отказ,0,0,0,0,...,0,0,0,0,0,0,0,594,37.34,3.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,отказ,0,0,0,1,...,0,0,0,0,0,0,0,483,15.14,4.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,отказ,2,0,0,1,...,0,0,0,0,0,0,0,503,19.14,4.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,архив,0,0,0,0,...,0,0,0,0,0,0,0,804,79.33,0.0
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,отказ,1,3,1,0,...,2,2,3,5,7,4,615,115,-26.91,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,товары в пути,0,0,0,0,...,1,0,0,1,0,2,0,390,139.35,0.0
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,новинка Даша,0,0,0,0,...,0,1,2,1,2,8,204,335,119.59,2234.0
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,новинка Даша,0,0,0,0,...,0,0,0,0,0,1,219,1590,104.46,830.0
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,текущий Дарья,0,0,0,0,...,0,0,0,0,0,0,469,1970,44.98,1669.0


In [249]:
# WB_orders_metrics = add_status(WB_orders_metrics)

### Проверка 

In [250]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024,Остаток,Цена,Наценка,Остаток на РЦ
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,отказ,0,0,0,0,...,0,0,0,0,0,0,0,594,37.34,3.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,отказ,0,0,0,1,...,0,0,0,0,0,0,0,483,15.14,4.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,отказ,2,0,0,1,...,0,0,0,0,0,0,0,503,19.14,4.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,архив,0,0,0,0,...,0,0,0,0,0,0,0,804,79.33,0.0
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,отказ,1,3,1,0,...,2,2,3,5,7,4,615,115,-26.91,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,товары в пути,0,0,0,0,...,1,0,0,1,0,2,0,390,139.35,0.0
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,новинка Даша,0,0,0,0,...,0,1,2,1,2,8,204,335,119.59,2234.0
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,новинка Даша,0,0,0,0,...,0,0,0,0,0,1,219,1590,104.46,830.0
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,текущий Дарья,0,0,0,0,...,0,0,0,0,0,0,469,1970,44.98,1669.0


### Добавление расчетных признаков "Суммы за неделю"

In [251]:
WB_orders_metrics = weekly_amount(WB_orders_metrics)

### Проверка 

In [252]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,04-April-2024,05-April-2024,06-April-2024,07-April-2024,01-April-07-April,08-April-2024,Остаток,Цена,Наценка,Остаток на РЦ
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,отказ,0,0,0,0,...,0,0,0,0,0,0,0,594,37.34,3.0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,отказ,0,0,0,1,...,0,0,0,0,0,0,0,483,15.14,4.0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,отказ,2,0,0,1,...,0,0,0,0,0,0,0,503,19.14,4.0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,архив,0,0,0,0,...,0,0,0,0,0,0,0,804,79.33,0.0
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,отказ,1,3,1,0,...,2,3,5,7,22,4,615,115,-26.91,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,товары в пути,0,0,0,0,...,0,0,1,0,2,2,0,390,139.35,0.0
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,новинка Даша,0,0,0,0,...,1,2,1,2,6,8,204,335,119.59,2234.0
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,новинка Даша,0,0,0,0,...,0,0,0,0,0,1,219,1590,104.46,830.0
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,текущий Дарья,0,0,0,0,...,0,0,0,0,0,0,469,1970,44.98,1669.0


In [253]:
WB_orders_metrics['Баркод'] = WB_orders_metrics['Баркод'].astype(int)

### Добавление строки "Итого" внизу таблицы и пустого столбца

In [254]:
WB_orders_metrics = add_total(WB_orders_metrics)

### Проверка 

In [255]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,05-April-2024,06-April-2024,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ
0,16000,"Гель для рук антисептический с витамином Е, 66...",11922227.0,4.627181e+12,SANITEX,архив,0,0,0,0,...,0,0,0,0,0,,0,51.0,-inf,0.0
1,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567.0,7.432142e+12,SS&Y Group,отказ,0,0,0,0,...,0,0,0,0,0,,0,594.0,37.34,3.0
2,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568.0,7.432142e+12,SS&Y Group,отказ,0,0,0,1,...,0,0,0,0,0,,0,483.0,15.14,4.0
3,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569.0,7.432142e+12,SS&Y Group,отказ,2,0,0,1,...,0,0,0,0,0,,0,503.0,19.14,4.0
4,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570.0,7.432142e+12,SS&Y Group,архив,0,0,0,0,...,0,0,0,0,0,,0,804.0,79.33,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,Z0401,Ваза для цветов и сухоцветов решетка,149328830.0,4.601664e+12,Limpide Side,текущий Дарья,6,7,4,4,...,1,6,3,18,1,,433,1619.0,68.56,0.0
1247,Z0500,Декантер с бокалами,150246240.0,4.602723e+12,Limpide Side,текущий Дарья,5,5,3,0,...,1,1,2,7,0,,295,3099.0,80.68,4.0
1248,Z0501,Графин для воды,158358449.0,4.630113e+12,Limpide Side,текущий Дарья,10,2,1,3,...,1,4,1,21,3,,24,1745.0,100.35,7.0
1249,c20502,"Мышечный перкуссионный массажер, ударный писто...",11416406.0,8.112390e+12,SS&Y Group,архив,0,0,0,0,...,0,0,0,0,0,,0,0.0,0.00,0.0


### Создание репрезентативных метрик

In [256]:
WB_full_df = WB_orders_metrics.copy()

### Загрузка "исторических" метрик с ценами

In [257]:
WB_price_metric = pd.read_csv(path_WB_price_metric, sep=';')

In [258]:
WB_price_metric

,Артикул,Наименование,Номенклатура,Баркод,ТМ,30-August-2022,31-August-2022,01-September-2022,02-September-2022,05-September-2022,...,26-March-2024,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,813,813,813,864,864,...,849,849,849,849,849,849,849,849,594,594
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,862,862,862,862,862,...,690,690,690,690,690,690,690,690,483,483
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,882,882,882,882,882,...,719,719,719,719,719,719,719,719,503,503
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,889,889,889,889,889,...,804,804,804,804,804,804,804,804,804,804
4,18102,Защитное полноэкранное 3D стекло для для iPhon...,8461127,7432141550590,SS&Y Group,132,132,132,132,132,...,132,132,132,132,132,132,132,132,132,132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,0,0,0,0,390,390,390,390,390,390
1247,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,0,0,0,650,650,650,650,335,335
1248,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,0,1590,1590,1590,1590
1249,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1970


In [259]:
WB_price_metric

,Артикул,Наименование,Номенклатура,Баркод,ТМ,30-August-2022,31-August-2022,01-September-2022,02-September-2022,05-September-2022,...,26-March-2024,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,813,813,813,864,864,...,849,849,849,849,849,849,849,849,594,594
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,862,862,862,862,862,...,690,690,690,690,690,690,690,690,483,483
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,882,882,882,882,882,...,719,719,719,719,719,719,719,719,503,503
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,889,889,889,889,889,...,804,804,804,804,804,804,804,804,804,804
4,18102,Защитное полноэкранное 3D стекло для для iPhon...,8461127,7432141550590,SS&Y Group,132,132,132,132,132,...,132,132,132,132,132,132,132,132,132,132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,0,0,0,0,390,390,390,390,390,390
1247,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,0,0,0,650,650,650,650,335,335
1248,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,0,1590,1590,1590,1590
1249,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1970


### Сведение новых цен и метрик по ценам

In [260]:
WB_full_features_metric

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,Цена до скидки,Скидка,Цена,Комиссия в рублях,НДС,Налог на прибыль,Налог,Прибыль,Наценка,Процент налога
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,0,0,0,0,0,...,5940.0,90.0,594,154.44,23.641667,22.750667,46.392333,91.002667,37.34,7.810157
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,0,0,0,1,1,...,2300.0,79.0,483,125.58,9.951667,9.227167,19.178833,36.908667,15.14,3.970773
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,2,0,0,1,0,...,3144.0,84.0,503,130.78,12.418333,11.663833,24.082167,46.655333,19.14,4.787707
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,0,0,...,4236.0,81.0,804,209.04,49.541667,48.335667,97.877333,193.342667,79.33,12.173798
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,1,3,1,0,2,...,1150.0,90.0,115,29.90,-3.220000,-3.392500,-6.612500,-13.570000,-26.91,-5.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,390.0,0.0,390,101.40,26.636667,26.051667,52.688333,104.206667,139.35,13.509829
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,646.0,48.0,335,73.70,22.363333,21.860833,44.224167,87.443333,119.59,13.201244
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,1590.0,0.0,1590,318.00,104.336667,101.951667,206.288333,407.806667,104.46,12.974109
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,2627.0,25.0,1970,413.70,101.798333,98.843333,200.641667,395.373333,44.98,10.184856


In [261]:
WB_price_metric = add_price_in_WB_price_metric(WB_price_metric, WB_full_features_metric)

In [262]:
for i in range(len(WB_price_metric)):
    art = WB_price_metric.loc[i]['Артикул']
    if len(WB_price_metric[WB_price_metric['Артикул']==art]) > 1:
        p = [x for x in list(WB_price_metric[WB_price_metric['Артикул']==art][list(WB_price_metric.columns)[-1]]) if x != 0][-1]
        for ind in list(WB_price_metric[WB_price_metric['Артикул']==art].index):
            if WB_price_metric.loc[ind][list(WB_price_metric.columns)[-1]] == 0:
                WB_price_metric.loc[ind, list(WB_price_metric.columns)[-1]] = p
                display(WB_price_metric.loc[ind:ind])
            

In [263]:
dfc = 'WB_price_metric'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 23


### Проверка 

In [264]:
len([x for x in list(WB_price_metric['04-December-2023']) if x == 0])

93

In [265]:
len([x for x in list(WB_price_metric['05-December-2023']) if x == 0])

87

In [266]:
len([x for x in list(pd.read_excel(wwm.path_full_WB_df,
              sheet_name='WB_full_df')['Цена']) if x == 0])

4

In [267]:
WB_price_metric

,Артикул,Наименование,Номенклатура,Баркод,ТМ,30-August-2022,31-August-2022,01-September-2022,02-September-2022,05-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,813,813,813,864,864,...,849,849,849,849,849,849,849,594,594,594
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,862,862,862,862,862,...,690,690,690,690,690,690,690,483,483,483
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,882,882,882,882,882,...,719,719,719,719,719,719,719,503,503,503
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,889,889,889,889,889,...,804,804,804,804,804,804,804,804,804,804
4,18102,Защитное полноэкранное 3D стекло для для iPhon...,8461127,7432141550590,SS&Y Group,132,132,132,132,132,...,132,132,132,132,132,132,132,132,132,132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,0,0,0,390,390,390,390,390,390,390
1247,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,0,0,650,650,650,650,335,335,335
1248,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,1590,1590,1590,1590,1590
1249,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1970,1970


In [268]:
WB_orders_metrics = WB_orders_metrics_final

## Проверка всех метрик

### Основные "исторические" метрики

In [269]:
WB_orders_metrics

,Артикул,Наименование,Номенклатура,Баркод,ТМ,23-February-2020,24-February-2020,25-February-2020,26-February-2020,27-February-2020,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,2,1,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,7,2,4,1,3,...,2,2,1,2,2,2,3,5,7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,3,1,0,0,1,0,0,1,0,2
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,3,0,0,0,1,2,1,2,8
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [270]:
WB_orders_metrics[WB_orders_metrics['Артикул']=='Z0011']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,23-February-2020,24-February-2020,25-February-2020,26-February-2020,27-February-2020,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
576,Z0011,"Кружка с двойными стенками, 355 мл",57464211,7432141559107,Limpide Side,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
577,Z0011,"Кружка с двойными стенками, 355 мл",49349914,7432141559106,Limpide Side,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Метрки по ценам 

In [271]:
WB_price_metric

,Артикул,Наименование,Номенклатура,Баркод,ТМ,30-August-2022,31-August-2022,01-September-2022,02-September-2022,05-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,813,813,813,864,864,...,849,849,849,849,849,849,849,594,594,594
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,862,862,862,862,862,...,690,690,690,690,690,690,690,483,483,483
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,882,882,882,882,882,...,719,719,719,719,719,719,719,503,503,503
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,889,889,889,889,889,...,804,804,804,804,804,804,804,804,804,804
4,18102,Защитное полноэкранное 3D стекло для для iPhon...,8461127,7432141550590,SS&Y Group,132,132,132,132,132,...,132,132,132,132,132,132,132,132,132,132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,0,0,0,390,390,390,390,390,390,390
1247,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,0,0,650,650,650,650,335,335,335
1248,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,1590,1590,1590,1590,1590
1249,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1970,1970


In [272]:
WB_price_metric[WB_price_metric['Артикул']=='Z0011']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,30-August-2022,31-August-2022,01-September-2022,02-September-2022,05-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
667,Z0011,"Кружка с двойными стенками, 355 мл",57464211,7432141559107,Limpide Side,411,411,411,411,424,...,500,500,500,500,500,500,500,350,350,350
668,Z0011,"Кружка с двойными стенками, 355 мл",49349914,7432141559106,Limpide Side,418,418,418,418,418,...,433,433,433,433,433,433,433,303,303,303


In [273]:
WB_price_metric[WB_price_metric[WB_price_metric.columns[-1]]==0]

,Артикул,Наименование,Номенклатура,Баркод,ТМ,30-August-2022,31-August-2022,01-September-2022,02-September-2022,05-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
245,50241,Вставка 14 см (доп ширина),38342223,7432141557103,Kids Zone,1522,1522,1522,1522,1522,...,0,0,0,0,0,0,0,0,0,0
697,с20502,"Мышечный перкуссионный массажер, ударный писто...",11416406,8112389834753,SS&Y Group,4410,4410,4410,4410,4410,...,0,0,0,0,0,0,0,0,0,0
698,27002,"Бесконтактный инфракрасный термометр, электрон...",11706753,8112389054199,SS&Y Group,819,819,819,819,819,...,0,0,0,0,0,0,0,0,0,0
757,20100,Паровая сауна для лица,8424485,8112389884815,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
841,c20502,"Мышечный перкуссионный массажер, ударный писто...",11416406,7432141550853,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Метрики по остаткам

In [274]:
stocks_WB_metric

,Артикул,Наименование,Номенклатура,Баркод,ТМ,30-August-2022,31-August-2022,01-September-2022,02-September-2022,05-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,68,66,66,66,65,...,0,0,0,0,0,0,0,0,0,0
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,58,59,59,58,54,...,0,0,0,0,0,0,0,0,0,0
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,27,27,27,27,28,...,0,0,0,0,0,0,0,0,0,0
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,30,30,30,29,28,...,0,0,0,0,0,0,0,0,0,0
4,18102,Защитное полноэкранное 3D стекло для для iPhon...,8461127,7432141550590,SS&Y Group,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,0,0,0,0,0,108,106,210,204
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,2,120,120,208,219
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,0,0,469,469


In [275]:
stocks_WB_metric[stocks_WB_metric['Артикул']=='Z0011']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,30-August-2022,31-August-2022,01-September-2022,02-September-2022,05-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
670,Z0011,"Кружка с двойными стенками, 355 мл",57464211,7432141559107,Limpide Side,165,165,163,160,162,...,0,0,0,0,0,0,0,0,0,0
671,Z0011,"Кружка с двойными стенками, 355 мл",49349914,7432141559106,Limpide Side,166,164,164,164,167,...,0,0,0,0,0,0,0,0,0,0


### Метрики по "обезличенным" товарам

In [276]:
WB_depersonalized_metric

,Артикул,07-February-2023,08-February-2023,09-February-2023,10-February-2023,11-February-2023,12-February-2023,13-February-2023,14-February-2023,15-February-2023,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
0,27701ba,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"2,86091E+11",1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29720-слоновая кость,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28800Mod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,286091234541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,30000-сиреневый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
6,Неопознанный товар,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [277]:
Q = []

for i in range(len(WB_depersonalized_metric)):
    
    if WB_depersonalized_metric.iloc[i, 4:].sum() == 0:
        
        Q.append(i)

In [278]:
WB_depersonalized_metric = WB_depersonalized_metric.query('index not in @Q').reset_index(drop=True)

In [279]:
WB_depersonalized_metric = WB_depersonalized_metric.drop_duplicates().reset_index(drop=True)

### Метрики с полными расчетными признаками

In [280]:
WB_full_features_metric[WB_full_features_metric['Артикул']=='Z0011']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,Цена до скидки,Скидка,Цена,Комиссия в рублях,НДС,Налог на прибыль,Налог,Прибыль,Наценка,Процент налога
576,Z0011,"Кружка с двойными стенками, 355 мл",57464211,7432141559107,Limpide Side,0,0,0,0,0,...,1422.5,72.0,350,80.50,8.811667,8.286667,17.098333,33.146667,26.55,4.885238
577,Z0011,"Кружка с двойными стенками, 355 мл",49349914,7432141559106,Limpide Side,0,0,0,0,0,...,1422.5,81.0,303,69.69,6.980000,6.525500,13.505500,26.102000,20.91,4.457261


### Финальные метрики для презентации

In [281]:
WB_full_df[WB_full_df['Артикул']=='Z0011']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,05-April-2024,06-April-2024,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ
1207,Z0011,"Кружка с двойными стенками, 355 мл",57464211.0,7.432142e+12,Limpide Side,отказ,0,0,0,0,...,0,0,0,0,0,,0,350.0,26.55,1.0
1208,Z0011,"Кружка с двойными стенками, 355 мл",49349914.0,7.432142e+12,Limpide Side,отказ,0,0,0,0,...,0,0,0,0,0,,0,303.0,20.91,1.0


## Добавление к финальной таблице для презентации дополнительных признаков

In [282]:
dop_f = WB_full_features_metric.copy()
dop_f = dop_f[['Баркод', 'Стоимость логистики', 'Предмет']]
dop_f['Баркод'] = dop_f['Баркод'].astype(int)

WB_full_df['          '] = None
WB_full_df = WB_full_df.merge(dop_f, on='Баркод', how='outer')
WB_full_df['Стоимость логистики'].loc[len(WB_full_df)-1] = 0
WB_full_df['Стоимость логистики'] = WB_full_df['Стоимость логистики'].astype(float)

WB_full_df['Предмет'] = WB_full_df['Предмет'].astype(str)

WB_full_df = WB_full_df.drop('Предмет', axis=1)

### Проверка

In [283]:
WB_full_df

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ,,Стоимость логистики
0,16000,"Гель для рук антисептический с витамином Е, 66...",11922227.0,4.627181e+12,SANITEX,архив,0,0,0,0,...,0,0,0,,0,51.0,-inf,0.0,None,54.0
1,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567.0,7.432142e+12,SS&Y Group,отказ,0,0,0,0,...,0,0,0,,0,594.0,37.34,3.0,None,54.0
2,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568.0,7.432142e+12,SS&Y Group,отказ,0,0,0,1,...,0,0,0,,0,483.0,15.14,4.0,None,54.0
3,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569.0,7.432142e+12,SS&Y Group,отказ,2,0,0,1,...,0,0,0,,0,503.0,19.14,4.0,None,54.0
4,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570.0,7.432142e+12,SS&Y Group,архив,0,0,0,0,...,0,0,0,,0,804.0,79.33,0.0,None,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,Z0401,Ваза для цветов и сухоцветов решетка,149328830.0,4.601664e+12,Limpide Side,текущий Дарья,6,7,4,4,...,3,18,1,,433,1619.0,68.56,0.0,None,91.8
1247,Z0500,Декантер с бокалами,150246240.0,4.602723e+12,Limpide Side,текущий Дарья,5,5,3,0,...,2,7,0,,295,3099.0,80.68,4.0,None,268.2
1248,Z0501,Графин для воды,158358449.0,4.630113e+12,Limpide Side,текущий Дарья,10,2,1,3,...,1,21,3,,24,1745.0,100.35,7.0,None,66.6
1249,c20502,"Мышечный перкуссионный массажер, ударный писто...",11416406.0,8.112390e+12,SS&Y Group,архив,0,0,0,0,...,0,0,0,,0,0.0,0.00,0.0,None,0.0


In [284]:
WB_full_df[WB_full_df['Артикул']=='Z0011']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ,,Стоимость логистики
1207,Z0011,"Кружка с двойными стенками, 355 мл",57464211.0,7.432142e+12,Limpide Side,отказ,0,0,0,0,...,0,0,0,,0,350.0,26.55,1.0,None,91.8
1208,Z0011,"Кружка с двойными стенками, 355 мл",49349914.0,7.432142e+12,Limpide Side,отказ,0,0,0,0,...,0,0,0,,0,303.0,20.91,1.0,None,66.6


In [285]:
# WB_full_df = WB_full_df[WB_full_df['Артикул']!='50090'].reset_index(drop=True)

### Расчет и добавление "выручки", "доли", "итоговой наценки"

In [286]:
df = None

df = WB_full_features_metric[['Баркод', 'Прибыль', 'Себестоимость', 'Наценка', dt_col(WB_full_features_metric)[-1]]]\
     .rename(columns={dt_col(WB_full_features_metric)[-1]: 'Продажи'})

df = df[(df['Себестоимость'].astype(str).isin(['0', 'nan', '0.0'])==False)\
        &(df['Продажи']>0)]\
     .reset_index(drop=True)

ITOG_NAC = round((((df['Прибыль'] * df['Продажи']).sum() / (df['Себестоимость'] *  df['Продажи']).sum()) * 100), 2)

df = None

In [287]:
WB_full_df

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ,,Стоимость логистики
0,16000,"Гель для рук антисептический с витамином Е, 66...",11922227.0,4.627181e+12,SANITEX,архив,0,0,0,0,...,0,0,0,,0,51.0,-inf,0.0,None,54.0
1,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567.0,7.432142e+12,SS&Y Group,отказ,0,0,0,0,...,0,0,0,,0,594.0,37.34,3.0,None,54.0
2,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568.0,7.432142e+12,SS&Y Group,отказ,0,0,0,1,...,0,0,0,,0,483.0,15.14,4.0,None,54.0
3,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569.0,7.432142e+12,SS&Y Group,отказ,2,0,0,1,...,0,0,0,,0,503.0,19.14,4.0,None,54.0
4,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570.0,7.432142e+12,SS&Y Group,архив,0,0,0,0,...,0,0,0,,0,804.0,79.33,0.0,None,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,Z0401,Ваза для цветов и сухоцветов решетка,149328830.0,4.601664e+12,Limpide Side,текущий Дарья,6,7,4,4,...,3,18,1,,433,1619.0,68.56,0.0,None,91.8
1247,Z0500,Декантер с бокалами,150246240.0,4.602723e+12,Limpide Side,текущий Дарья,5,5,3,0,...,2,7,0,,295,3099.0,80.68,4.0,None,268.2
1248,Z0501,Графин для воды,158358449.0,4.630113e+12,Limpide Side,текущий Дарья,10,2,1,3,...,1,21,3,,24,1745.0,100.35,7.0,None,66.6
1249,c20502,"Мышечный перкуссионный массажер, ударный писто...",11416406.0,8.112390e+12,SS&Y Group,архив,0,0,0,0,...,0,0,0,,0,0.0,0.00,0.0,None,0.0


In [288]:
WB_full_df

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,07-April-2024,01-April-07-April,08-April-2024,,Остаток,Цена,Наценка,Остаток на РЦ,,Стоимость логистики
0,16000,"Гель для рук антисептический с витамином Е, 66...",11922227.0,4.627181e+12,SANITEX,архив,0,0,0,0,...,0,0,0,,0,51.0,-inf,0.0,None,54.0
1,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567.0,7.432142e+12,SS&Y Group,отказ,0,0,0,0,...,0,0,0,,0,594.0,37.34,3.0,None,54.0
2,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568.0,7.432142e+12,SS&Y Group,отказ,0,0,0,1,...,0,0,0,,0,483.0,15.14,4.0,None,54.0
3,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569.0,7.432142e+12,SS&Y Group,отказ,2,0,0,1,...,0,0,0,,0,503.0,19.14,4.0,None,54.0
4,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570.0,7.432142e+12,SS&Y Group,архив,0,0,0,0,...,0,0,0,,0,804.0,79.33,0.0,None,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,Z0401,Ваза для цветов и сухоцветов решетка,149328830.0,4.601664e+12,Limpide Side,текущий Дарья,6,7,4,4,...,3,18,1,,433,1619.0,68.56,0.0,None,91.8
1247,Z0500,Декантер с бокалами,150246240.0,4.602723e+12,Limpide Side,текущий Дарья,5,5,3,0,...,2,7,0,,295,3099.0,80.68,4.0,None,268.2
1248,Z0501,Графин для воды,158358449.0,4.630113e+12,Limpide Side,текущий Дарья,10,2,1,3,...,1,21,3,,24,1745.0,100.35,7.0,None,66.6
1249,c20502,"Мышечный перкуссионный массажер, ударный писто...",11416406.0,8.112390e+12,SS&Y Group,архив,0,0,0,0,...,0,0,0,,0,0.0,0.00,0.0,None,0.0


In [289]:
ITOG_NAC

56.99

In [290]:
# Q = []
# for row in WB_full_df.columns[-20:]:
    
#     try:
#         row = pd.to_datetime(row)
        
#         Q.append(row)
        
#     except:
        
#         None
        
# Q.sort()

# Q = list(map(lambda x: x.strftime(format='%d-%B-%Y'), Q))

WB_full_df['Выручка'] = WB_full_df['Цена'] * WB_full_df[dt_col(WB_full_df)[-1]] 
WB_full_df['Выручка'].loc[len(WB_full_df)-1] = 0
WB_full_df['Выручка'] = WB_full_df['Выручка'].astype(float)
s = WB_full_df['Выручка'].sum()
WB_full_df['Доля выручки'] = (WB_full_df['Выручка'] * 100) / s
WB_full_df['Выручка'].loc[len(WB_full_df)-1] = s
# WB_full_df['Итого наценка'] = WB_full_df['Доля выручки'] / 100 * WB_full_df['Наценка']
# # e = WB_full_df['Итого наценка'].sum()
# e = WB_full_df[WB_full_df['Итого наценка']!=np.inf]['Итого наценка'].sum()

WB_full_df['Итого наценка'] = None
WB_full_df['Итого наценка'].loc[len(WB_full_df)-1] = ITOG_NAC


In [291]:
len(WB_full_df) - len(list(set(WB_full_df['Баркод'])))

0

In [292]:
len(WB_full_df.iloc[:-1, :]) - len(WB_full_df.iloc[:-1, 3].astype(int).unique())

0

In [293]:
WB_full_df['Итого наценка']

0        None
1        None
2        None
3        None
4        None
        ...  
1246     None
1247     None
1248     None
1249     None
1250    56.99
Name: Итого наценка, Length: 1251, dtype: object

In [294]:
WB_full_df['Выручка']

0             0.0
1             0.0
2             0.0
3             0.0
4             0.0
          ...    
1246       1619.0
1247          0.0
1248       5235.0
1249          0.0
1250    2154682.0
Name: Выручка, Length: 1251, dtype: float64

### Доп. признак для маркетинга

In [295]:
# dp = pd.read_excel(path_dp)

# dp['Артикул'] = dp['Артикул'].astype(str)
# WB_full_df['                       '] = None
# for row in list(set(dp['Артикул'])):
    
#     if row in list(set(WB_full_df['Артикул'])): 
#         if len(WB_full_df[WB_full_df['Артикул']==row]) > 1:
            
#             print(row)

            
# dp = dp[['Артикул', 'Дата публикации SEO', 'Группа SEO']]

In [296]:
# for row in list(set(dp['Артикул'])):
    
#     if len(dp[dp['Артикул']==row]) > 1:
            
#             print(row)

In [297]:
# WB_full_df = WB_full_df.merge(dp, on='Артикул', how='left')

In [298]:
# WB_full_df

In [299]:
# list(filter(lambda x: re.compile('[^а-яА-Я ]').sub('', x), list(dp['Артикул'])))

In [300]:
# len(dp) - len(list(set(dp['Артикул'])))

In [301]:
# len(WB_full_df) - WB_full_df['Группа SEO'].isna().sum()

In [302]:
# for row in list(set(dp['Артикул'])):
    
#     if row not in list(set(WB_full_df['Артикул'])):
        
#         print(row)

In [303]:
nbar

,Баркод,Новый баркод
0,7432141950648,4612161840522
1,7432141559442,4610532500976
2,7432141950153,4624597893829
3,7432141557135,4664347683263
4,7432141557141,4676952882856
...,...,...
582,7432141556092,4644958833981
583,7432141550897,4657545794589
584,7432141556600,4645755754820
585,7432141557045,4629624513251


In [304]:
for i in range(len(nbar)):
    if len(nbar[nbar['Новый баркод']==nbar.loc[i]['Новый баркод']]) > 1:
        display(nbar[nbar['Новый баркод']==nbar.loc[i]['Новый баркод']])

,Баркод,Новый баркод
58,7432141550511,4658622962440
60,7432141550512,4658622962440


,Баркод,Новый баркод
58,7432141550511,4658622962440
60,7432141550512,4658622962440


,Баркод,Новый баркод
143,7432141949956,4640248746955
555,7432141550861,4640248746955


,Баркод,Новый баркод
143,7432141949956,4640248746955
555,7432141550861,4640248746955


In [305]:
Q = []
for i in range(len(nbar)):
    if len(nbar[nbar['Новый баркод']==nbar.loc[i]['Новый баркод']]) > 1:
        Q.append(nbar[nbar['Новый баркод']==nbar.loc[i]['Новый баркод']].index[1])

nbar = nbar.query("index not in @Q").reset_index(drop=True)

In [306]:
for i in range(len(nbar)):
    if len(nbar[nbar['Новый баркод']==nbar.loc[i]['Новый баркод']]) > 1:
        display(nbar[nbar['Новый баркод']==nbar.loc[i]['Новый баркод']])
        
        
        

In [307]:
len(nbar) - len(list(set(nbar['Новый баркод'])))

0

## Работа с метриками ОСТАТКИ FBS

In [308]:
WB_ofbs = pd.read_excel(path_WB_ofbs)

In [309]:
WB_ofbs = WB_ofbs[['Баркод', 'Количество']]


In [310]:
WB_ofbs.query(f"Баркод in {list(set(nbar['Баркод']))}")

,Баркод,Количество
56,7432141550511,0
1327,7432141550511,0
1862,7432141550511,21


In [311]:
len(list(filter(lambda x: x[0]== '4', list(WB_ofbs['Баркод'].astype(str)))))

2235

In [312]:
for i in range(len(WB_ofbs)):
    bar = WB_ofbs.loc[i]['Баркод']
    if bar in list(set(nbar['Новый баркод'])):
        WB_ofbs['Баркод'].loc[i] = nbar[nbar['Новый баркод']==bar]['Баркод'].values[0]

In [313]:
len(list(filter(lambda x: x[0]== '4', list(WB_ofbs['Баркод'].astype(str)))))

702

In [314]:
len(WB_ofbs) - len(list(set(WB_ofbs['Баркод'])))

1807

In [315]:
WB_ofbs = WB_ofbs[['Баркод', 'Количество']]
WB_ofbs['Баркод'] = WB_ofbs['Баркод'].astype(int)
WB_ofbs['Количество'] = WB_ofbs['Количество'].fillna(0)
WB_ofbs['Количество'] = WB_ofbs['Количество'].astype(int)
WB_ofbs = WB_ofbs.groupby('Баркод')['Количество'].sum().reset_index(drop=False)

In [316]:
len(WB_ofbs) - len(list(set(WB_ofbs['Баркод'])))

0

In [317]:
WB_ofbs_metrics = pd.read_csv(path_WB_ofbs_metrics, sep=';')
WB_ofbs_metrics = WB_ofbs_metrics.astype(int)
WB_ofbs_metrics

,Баркод,24-October-2022,25-October-2022,27-October-2022,28-October-2022,31-October-2022,01-November-2022,02-November-2022,03-November-2022,07-November-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,4600275286909,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4600434586475,0,0,0,0,0,0,0,0,0,...,34,34,34,25,25,25,21,20,19,19
2,4600436646801,0,0,0,0,0,0,0,0,0,...,0,0,0,77,77,77,77,15,16,16
3,4600478190874,0,0,0,0,0,0,0,0,0,...,5,6,6,7,7,8,9,10,14,14
4,4601024336050,0,0,0,0,0,0,0,0,0,...,154,182,183,189,190,192,194,195,200,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,7432141950750,0,0,0,0,0,0,0,0,0,...,96,96,96,101,100,100,103,105,105,105
1313,7432141950753,0,0,0,0,0,0,0,0,0,...,3,3,3,4,3,3,3,3,3,3
1314,7432141950760,0,0,0,0,0,0,0,0,0,...,2,1,179,129,129,129,129,98,98,98
1315,7432141950763,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [318]:
len(list(filter(lambda x: x[0]== '4', list(WB_ofbs_metrics['Баркод'].astype(str)))))

300

In [319]:
for i in range(len(WB_ofbs_metrics)):
    bar = WB_ofbs_metrics.loc[i]['Баркод']
    if bar in list(set(nbar['Новый баркод'])):
        WB_ofbs_metrics['Баркод'].loc[i] = nbar[nbar['Новый баркод']==bar]['Баркод'].values[0]

In [320]:
len(list(filter(lambda x: x[0]== '4', list(WB_ofbs_metrics['Баркод'].astype(str)))))

300

In [321]:
WB_ofbs_metrics.query(f"Баркод in {list(set(nbar['Баркод']))}")

,Баркод,24-October-2022,25-October-2022,27-October-2022,28-October-2022,31-October-2022,01-November-2022,02-November-2022,03-November-2022,07-November-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
304,7432141550503,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,7432141550504,10,10,10,10,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
306,7432141550505,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307,7432141550506,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
308,7432141550507,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,7432141950750,0,0,0,0,0,0,0,0,0,...,96,96,96,101,100,100,103,105,105,105
1313,7432141950753,0,0,0,0,0,0,0,0,0,...,3,3,3,4,3,3,3,3,3,3
1314,7432141950760,0,0,0,0,0,0,0,0,0,...,2,1,179,129,129,129,129,98,98,98
1315,7432141950763,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [322]:
WB_ofbs_metrics = WB_ofbs_metrics.groupby('Баркод')[list(WB_ofbs_metrics.columns[1:])].sum().reset_index(drop=False)

In [323]:
len(WB_ofbs_metrics) - len(list(set(WB_ofbs_metrics['Баркод'])))

0

In [324]:
WB_ofbs_metrics

,Баркод,24-October-2022,25-October-2022,27-October-2022,28-October-2022,31-October-2022,01-November-2022,02-November-2022,03-November-2022,07-November-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,4600275286909,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4600434586475,0,0,0,0,0,0,0,0,0,...,34,34,34,25,25,25,21,20,19,19
2,4600436646801,0,0,0,0,0,0,0,0,0,...,0,0,0,77,77,77,77,15,16,16
3,4600478190874,0,0,0,0,0,0,0,0,0,...,5,6,6,7,7,8,9,10,14,14
4,4601024336050,0,0,0,0,0,0,0,0,0,...,154,182,183,189,190,192,194,195,200,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,7432141950750,0,0,0,0,0,0,0,0,0,...,96,96,96,101,100,100,103,105,105,105
1313,7432141950753,0,0,0,0,0,0,0,0,0,...,3,3,3,4,3,3,3,3,3,3
1314,7432141950760,0,0,0,0,0,0,0,0,0,...,2,1,179,129,129,129,129,98,98,98
1315,7432141950763,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [325]:
WB_full_df = WB_full_df.merge(WB_ofbs, on='Баркод', how='left')
WB_full_df = WB_full_df.rename(columns={WB_ofbs.columns[-1]: 'Остатки FBS'})
s = list(WB_full_df['Остатки FBS']).copy()
WB_full_df = WB_full_df.drop('Остатки FBS', axis=1)
WB_full_df.insert(WB_full_df.columns.get_loc('Остаток') + 1, 'Остатки FBS', s)

In [326]:
datetime.datetime.today().strftime('%d-%B-%Y')

'09-April-2024'

In [327]:
f = '0'

if len(WB_ofbs) - len(list(set(WB_ofbs['Баркод']))) != 0:
    
    f = 'Баркоды не уникальны!'
    
    print(f)
    
else: 
    
    WB_ofbs = WB_ofbs.rename(columns={WB_ofbs.columns[-1]: datetime.datetime.today().strftime('%d-%B-%Y')})
    
    

In [328]:
WB_ofbs

,Баркод,09-April-2024
0,4600275286909,0
1,4600434586475,19
2,4600436646801,16
3,4600478190874,14
4,4601024336050,200
...,...,...
1299,7432141950750,105
1300,7432141950753,3
1301,7432141950760,98
1302,7432141950763,0


In [329]:
if WB_ofbs.columns[-1] in list(WB_ofbs_metrics.columns[1:]):
    
    if WB_ofbs[WB_ofbs.columns[-1]].sum() != WB_ofbs_metrics[WB_ofbs.columns[-1]].sum():
        
        print('В метриках есть день с данными из "новых" остатков fbs, но сумма остатков разная!')
        
        f = '1'
        
        
    else:
        
        print('В метриках есть день с данными из "новых" остатков fbs, сумма остатков одинаковая!')
        
        f = '2'
        
else:
    
    if f == '0':
    
        WB_ofbs_metrics = WB_ofbs_metrics.merge(WB_ofbs, on='Баркод', how='outer')

        WB_ofbs_metrics = WB_ofbs_metrics.fillna(0)
        
        WB_ofbs_metrics = WB_ofbs_metrics.astype(int)
        


В метриках есть день с данными из "новых" остатков fbs, сумма остатков одинаковая!


### Добавление дополнительных признаков по акциям

In [330]:
AC = WB_full_df[WB_full_df['Баркод'].astype(str)!='nan'][['Баркод']].reset_index(drop=True).copy()
AC['Баркод'] = AC['Баркод'].astype(int)

In [331]:
WB_ac

['Товары для акции_Распродажа неходового товара_08.04.2024 08.32.42.xlsx',
 'Товары для акции_СКИДКИ ПРОСТО КОСМОС_08.04.2024 08.32.52.xlsx']

In [332]:
if len(WB_ac) > 0:
    for row in WB_ac:
        ac_name = row[:-25]
        ac = pd.read_excel(path_downloads + row)[['Последний баркод', 'Плановая цена для акции']]\
             .rename(columns={'Последний баркод': 'Баркод', 'Плановая цена для акции': 'Цена'})
        wwm.unique_barcode(ac)
        ac = WB_orders_metrics_for_ac.copy().merge(ac, on='Баркод', how='right')
        ac = creating_calculated_features(ac)
        ac = ac[['Баркод', 'Цена', 'Наценка']]
        ac.columns = ['Баркод', 'Цена'+'_'+ac_name+'_идет', 'Наценка'+'_'+ac_name+'_идет']
        AC = AC.merge(ac, on='Баркод', how='left')
else:
    None

In [333]:
WB_ac_feature

[]

In [334]:
if len(WB_ac_feature) > 0:
    for row in WB_ac_feature:
        ac_name = row[:-25]
        ac = pd.read_excel(path_downloads + row)[['Последний баркод', 'Плановая цена для акции']]\
             .rename(columns={'Последний баркод': 'Баркод', 'Плановая цена для акции': 'Цена'})
        wwm.unique_barcode(ac)
        ac = WB_orders_metrics_for_ac.copy().merge(ac, on='Баркод', how='right')
        ac = creating_calculated_features(ac)
        ac = ac[['Баркод', 'Цена', 'Наценка']]
        ac.columns = ['Баркод', 'Цена'+'_'+ac_name, 'Наценка'+'_'+ac_name]
        AC = AC.merge(ac, on='Баркод', how='left')
else:
    None

In [335]:
AC

,Баркод,Цена_Товары для акции_Распродажа неходового товара_идет,Наценка_Товары для акции_Распродажа неходового товара_идет,Цена_Товары для акции_СКИДКИ ПРОСТО КОСМОС_идет,Наценка_Товары для акции_СКИДКИ ПРОСТО КОСМОС_идет
0,4627180760348,NaN,NaN,NaN,NaN
1,7432141949918,NaN,NaN,666.0,51.74
2,7432141949925,NaN,NaN,NaN,NaN
3,7432141949932,NaN,NaN,683.0,55.14
4,7432141550639,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1245,4601497102794,NaN,NaN,889.0,79.14
1246,4601663851785,NaN,NaN,1521.0,59.72
1247,4602723427780,NaN,NaN,2869.0,68.34
1248,4630112870930,NaN,NaN,1536.0,79.29


In [336]:
# if len(WB_ac) > 0:
#     for key in list(WB_ac.keys()):
#         if ';' in key:
#             ac = pd.DataFrame(columns=['Баркод', 'Цена'])
#             for path in key.split(';'):
#                 xlrd.xlsx.ensure_elementtree_imported(False, None)
#                 xlrd.xlsx.Element_has_iter = True
#                 wb = xlrd.open_workbook(path_downloads + path)
#                 sh = wb.sheet_by_name('Отчёт по скидкам для акции')
#                 header = sh.row_values(0, start_colx=0, end_colx=None)
#                 ac_0 = pd.DataFrame()
#                 for i in range(sh.ncols):
#                     ac_0[header[i]] = sh.col_values(i, start_rowx=1, end_rowx=None)
#                 ac_0 = ac_0[['Последний\nбаркод', 'Плановая цена для акции']]
#                 ac_0.columns = ['Баркод', 'Цена']
#                 ac_0['Баркод'] = ac_0['Баркод'].astype(int)
#                 ac = pd.concat([ac, ac_0], ignore_index=True)
#         else:
#             path = key
#             xlrd.xlsx.ensure_elementtree_imported(False, None)
#             xlrd.xlsx.Element_has_iter = True
#             wb = xlrd.open_workbook(path_downloads + path)
#             sh = wb.sheet_by_name('Отчёт по скидкам для акции')
#             header = sh.row_values(0, start_colx=0, end_colx=None)
#             ac_0 = pd.DataFrame()
#             for i in range(sh.ncols):
#                 ac_0[header[i]] = sh.col_values(i, start_rowx=1, end_rowx=None)
#             ac_0 = ac_0[['Последний\nбаркод', 'Плановая цена для акции']]
#             ac_0.columns = ['Баркод', 'Цена']
#             ac_0['Баркод'] = ac_0['Баркод'].astype(int)
#             ac, ac_0 = ac_0, None
#             ac = ac.drop_duplicates().reset_index(drop=True)
#             if len(ac) - len(list(set(ac['Баркод']))) != 0:
#                 exit()   
#         ac_komm = WB_ac[key].split(';')[0]
#         ac_name = WB_ac[key].split(';')[1]
#         ac = WB_orders_metrics_for_ac.copy().merge(ac, on='Баркод', how='right')
#         if ac_komm != 'бк':
#             ac['Процент комиссии'] = ac['Процент комиссии'] - float(ac_komm[:-1])
#         ac = creating_calculated_features(ac)
#         ac = ac[['Баркод', 'Цена', 'Наценка']]
#         ac.columns = ['Баркод', 'Цена'+'_'+ac_name+'_идет', 'Наценка'+'_'+ac_name+'_идет']
#         AC = AC.merge(ac, on='Баркод', how='left')
# else:
#     None
    
                
                
                
        

In [337]:
# if len(WB_ac_feature) > 0:
#     for key in list(WB_ac_feature.keys()):
#         if ';' in key:
#             ac = pd.DataFrame(columns=['Баркод', 'Цена'])
#             for path in key.split(';'):
#                 xlrd.xlsx.ensure_elementtree_imported(False, None)
#                 xlrd.xlsx.Element_has_iter = True
#                 wb = xlrd.open_workbook(path_downloads + path)
#                 sh = wb.sheet_by_name('Отчёт по скидкам для акции')
#                 header = sh.row_values(0, start_colx=0, end_colx=None)
#                 ac_0 = pd.DataFrame()
#                 for i in range(sh.ncols):
#                     ac_0[header[i]] = sh.col_values(i, start_rowx=1, end_rowx=None)
#                 ac_0 = ac_0[['Последний\nбаркод', 'Плановая цена для акции']]
#                 ac_0.columns = ['Баркод', 'Цена']
#                 ac_0['Баркод'] = ac_0['Баркод'].astype(int)
#                 ac = pd.concat([ac, ac_0], ignore_index=True)
#         else:
#             path = key
#             xlrd.xlsx.ensure_elementtree_imported(False, None)
#             xlrd.xlsx.Element_has_iter = True
#             wb = xlrd.open_workbook(path_downloads + path)
#             sh = wb.sheet_by_name('Отчёт по скидкам для акции')
#             header = sh.row_values(0, start_colx=0, end_colx=None)
#             ac_0 = pd.DataFrame()
#             for i in range(sh.ncols):
#                 ac_0[header[i]] = sh.col_values(i, start_rowx=1, end_rowx=None)
#             ac_0 = ac_0[['Последний\nбаркод', 'Плановая цена для акции']]
#             ac_0.columns = ['Баркод', 'Цена']
#             ac_0['Баркод'] = ac_0['Баркод'].astype(int)
#             ac, ac_0 = ac_0, None
#             ac = ac.drop_duplicates().reset_index(drop=True)
#             if len(ac) - len(list(set(ac['Баркод']))) != 0:
#                 exit()   
#         ac_komm = WB_ac_feature[key].split(';')[0]
#         ac_name = WB_ac_feature[key].split(';')[1]
#         ac = WB_orders_metrics_for_ac.copy().merge(ac, on='Баркод', how='right')
#         if ac_komm != 'бк':
#             ac['Процент комиссии'] = ac['Процент комиссии'] - float(ac_komm[:-1])
#         ac = creating_calculated_features(ac)
#         ac = ac[['Баркод', 'Цена', 'Наценка']]
#         ac.columns = ['Баркод', 'Цена'+'_'+ac_name, 'Наценка'+'_'+ac_name]
#         AC = AC.merge(ac, on='Баркод', how='left')
# else:
#     None

In [338]:
pd.DataFrame(list(AC.columns[1:]), 
             columns=['Col_for_not_del']).to_excel(path_downloads + 'Col_for_not_del_wb.xlsx', index=False)

In [339]:
pd.DataFrame(list(AC.columns[1:]), 
             columns=['Col_for_del']).to_excel(path_downloads + 'Col_for_del_wb.xlsx', index=False)

In [340]:
# df_name = {}
# for i in range(len(list(WB_ac.keys()))):
#     df_name[list(WB_ac.keys())[i]] = f"p_{i+1}" 

In [341]:
# w = WB_full_features_metric.copy()


# col = ['Процент комиссии', 
#        'Стоимость логистики', 
#        'Стоимость хранения',
#        'Себестоимость', 
#        'Цена']


# w = w[list(w.columns[:5])+col]
# w['Баркод'] = w['Баркод'].astype(int)
# w1 = w.copy()
# w1 = w1[['Баркод']+col]
# w1 = w1.drop('Цена', axis=1)


# def creating_calculated_features_V2(data):
    
#     data['Комиссия в рублях'] = (data['Цена'] / 100) * data['Процент комиссии']
    
#     data['НДС'] =  ((data['Цена'] * 0.2 / 1.2) - (data['Себестоимость'] * 0.2 / 1.2) - 
#                     (data['Комиссия в рублях'] * 0.2 / 1.2) - 
#                     (data['Стоимость логистики'] * 0.2 / 1.2))


#     R = ((data['Цена'] - (data['Цена'] * 0.2 / 1.2)) - 
#          (data['Себестоимость'] - (data['Себестоимость'] * 0.2 / 1.2)) - 
#          (data['Комиссия в рублях'] - (data['Комиссия в рублях'] * 0.2 / 1.2)) - 
#          (data['Стоимость логистики'] - (data['Стоимость логистики'] * 0.2 / 1.2)))
    
    
#     data['Налог на прибыль'] = R*0.2
    
#     data['Налог'] = data['НДС'] + data['Налог на прибыль']
    
#     data['Прибыль'] = (data['Цена'] - 
#                        data['Себестоимость'] - 
#                        data['Комиссия в рублях'] - 
#                        data['Стоимость логистики'] - 
#                        data['Налог'])
    
    
#     data['Наценка'] = (data['Прибыль'] / data['Себестоимость']) * 100
    
#     data['Процент налога'] = (data['Налог'] * 100) / data['Цена']
    
    
    
#     for i in range(len(data)):
        
#         if data['Цена'][i] <= 0:
#             data['Налог на прибыль'][i] = 0
#             data['Прибыль'][i] = 0
#             data['Процент налога'][i] = 0
#             data['Процент налога'][i] = 0
        
        
        
#     data['Наценка'] = data['Наценка'].round(2)
    
#     if 'Артикул' in list(data.columns) and 'ТМ' in list(data.columns):
    
#         data = data[list(data.columns[:5])+['Цена', 'Наценка', 'Процент комиссии']]
        
#     else:
        
#         data = data[['Баркод', 'Цена', 'Наценка', 'Процент комиссии']]
    
    
#     return data






# w = creating_calculated_features_V2(w)


# w['           '] = None

In [342]:
# df = None
# R = None
# R = []
# for df in WB_ac:
    
    
#     path_P = path_downloads + df
#     xlrd.xlsx.ensure_elementtree_imported(False, None)
#     xlrd.xlsx.Element_has_iter = True
#     wb = xlrd.open_workbook(path_P)
#     sh = wb.sheet_by_name('Отчёт по скидкам для акции')
#     header = sh.row_values(0, start_colx=0, end_colx=None)
#     locals()[df_name[df]] = pd.DataFrame()
#     for i in range(sh.ncols):
#         locals()[df_name[df]][header[i]] = sh.col_values(i, start_rowx=1, end_rowx=None)
        
        
        
#     locals()[df_name[df]] = locals()[df_name[df]][['Последний\nбаркод', 'Плановая цена для акции']]
#     locals()[df_name[df]] = locals()[df_name[df]].rename(columns={locals()[df_name[df]].columns[0]: 'Баркод', locals()[df_name[df]].columns[-1]: 'Цена'})
#     locals()[df_name[df]] = locals()[df_name[df]].dropna().reset_index(drop=True)
#     locals()[df_name[df]]['Баркод'] = locals()[df_name[df]]['Баркод'].astype(int)
#     locals()[df_name[df]]['Цена'] = locals()[df_name[df]]['Цена'].astype(int)
    
    
    
    
#     locals()[df_name[df]] = w1.merge(locals()[df_name[df]], on='Баркод', how='inner')
    
    
#     if WB_ac[df] != 'бк':
        
#         for i in range(len(locals()[df_name[df]])):

#             e = locals()[df_name[df]].loc[i]['Процент комиссии'] - int(WB_ac[df][:-1])

#             if e < 3:

#                 locals()[df_name[df]]['Процент комиссии'].loc[i] = 3
#             else:
#                 locals()[df_name[df]]['Процент комиссии'].loc[i] = e
                
#     else:
        
#         None
        
        
        
#     locals()[df_name[df]] = creating_calculated_features_V2(locals()[df_name[df]]) 
    
    
#     locals()[df_name[df]] = locals()[df_name[df]].rename(columns={'Цена': f"Цена-{WB_ac[df]+'_'+df_name[df][-1]}", 'Наценка': f"Наценка-{WB_ac[df]+'_'+df_name[df][-1]}", 'Процент комиссии': f"Процент комиссии-{WB_ac[df]+'_'+df_name[df][-1]}"})
#     if WB_ac[df] != 'бк':
#         R.append(f"Наценка-{WB_ac[df]+'_'+df_name[df][-1]}")

#     if int(df_name[df][-1]) != len(WB_ac):
        

    
#         locals()[df_name[df]][f"{'           ' + (' '* int(df_name[df][-1]))}"] = None 
        
#     else:
        
#         None
    
#     w = w.merge(locals()[df_name[df]], on='Баркод', how='outer')
    
#     w = w.fillna(0)
    
#     for col_ in w.columns:
        
#         if '  ' in col_:
#             w[col_] = None
#         else:
#             None
            
            
# W = None
# E = None  
# Q = None
# R = ['Наценка'] + R
# Q = []


# for i in range(len(w)):
#     W = []
#     n = -1000000000000000
#     for row in R:
#         if row != R[0]:
#             W.append(w.loc[i][row])
#         else:
#             None
#         if w.loc[i][row] > n:
#             n = w.loc[i][row]
#         else:
#             None
#     if n == -1000000000000000:
#         Q.append(None)
#     elif n == w.loc[i][R[0]]:
#         Q.append('Не учавств. в акции')
#     elif n in W:
#         Q.append(', '.join(list(filter(lambda x: n == w.loc[i][x], R[1:]))))
#     else:
#         Q.append(None)
        
# W = None
# E = None

# w['              '] = None
# w['Лучшая наценка'] = Q

# Q = None

# del_col = ['Артикул', 'Наименование', 'Номенклатура', 'ТМ']

# w = w.drop(del_col, axis=1)
# # w = w[['Баркод']+list(w.columns[-13:])]


# w['Баркод'] = w['Баркод'].astype(int)

# w = w.rename(columns={'Цена': 'Цена_основн.', 'Наценка': 'Наценка_основн.', 'Процент комиссии': 'Процент комиссии_основн.'})

# WB_full_df['                                 '] = None

# WB_full_df = WB_full_df.merge(w, on='Баркод', how='left')

# WB_ac = None
# W = None

# for row in df_name:
#     locals()[df_name[row]] = None
# df_name = None

In [343]:
len(WB_full_df) - len(list(set(WB_full_df['Баркод'])))

0

In [344]:
for i in range(len(WB_full_df)):
    
    if len(WB_full_df[WB_full_df['Баркод']==WB_full_df.loc[i]['Баркод']]) >1:
        display(WB_full_df[WB_full_df['Баркод']==WB_full_df.loc[i]['Баркод']])

In [345]:
if len(AC) - len(list(set(AC['Баркод']))) !=0:
    exit()

In [346]:
if len(WB_full_df) - len(list(set(WB_full_df['Баркод']))) !=0:
    exit()

In [347]:
WB_full_df = WB_full_df.merge(AC, on='Баркод', how='left')

In [348]:
# dict_for_metric_wb = {}
# dict_for_metric_wb['Col_for_del'] = [x for x in list(w.columns) if '  ' not in x and 'Баркод' not in x]
# pd.DataFrame(dict_for_metric_wb).to_excel(path_downloads + 'Col_for_del_wb.xlsx', index=False)
# dict_for_metric_wb = None

### Дополнительный признак "сервис" (спринт) 

In [349]:
spr_wb = pd.read_excel(path_spr_wb, 
                       dtype={'Артикул': 'str'})

spr_oz = pd.read_excel(path_spr_oz, 
                       dtype={'Артикул': 'str'})

WB_full_df = WB_full_df.merge(spr_wb, on='Артикул', how='left')\
             .merge(spr_oz, on='Артикул', how='left')

In [350]:
# Serv['Артикул'] = Serv['Артикул'].astype(str)

# WB_full_df = WB_full_df.merge(Serv, on='Артикул', how='left')

In [351]:
# for i in range(len(WB_orders_metrics)):
    
#     if len(WB_orders_metrics[WB_orders_metrics['Баркод']==WB_orders_metrics.loc[i]['Баркод']]) >1:
#         display(WB_orders_metrics[WB_orders_metrics['Баркод']==WB_orders_metrics.loc[i]['Баркод']])

### Доп.

In [352]:
try:
    Q = []

    for i in range(len(WB_full_df)):

        if WB_full_df.loc[i]['Артикул'] in list(set(dop_ac['Артикул'])):

            Q.append('Участвует в акции')
        else:
            Q.append(None)

    WB_full_df['Участие в акции'] = Q
    Q = None
except:
    None

In [353]:
# WB_full_df = WB_full_df.merge(zac, on='Артикул', how='left')

### Наценка "Невинномысск" и остатки этого склада

In [354]:
nvm = nvm_stocks.merge(dfl[['Баркод', 'Наценка_Невинномысск_монопалета']], on='Баркод', how='outer')

In [355]:
nvm = nvm.fillna(0)

In [356]:
len(WB_full_df.columns)

78

In [357]:
WB_full_df = WB_full_df.merge(nvm, on='Баркод', how='left')
WB_full_df['                                  '] = None
WB_full_df = WB_full_df[list(WB_full_df.columns[:WB_full_df.columns.get_loc('Остаток на РЦ')+1]) + ['                                  '] + list(WB_full_df.columns[-3:-1]) + list(WB_full_df.columns[WB_full_df.columns.get_loc('Остаток на РЦ')+1:-3])]
WB_full_df['Остаток в Невинномысске'].loc[len(WB_full_df)-1] = WB_full_df['Остаток в Невинномысске'][:-1].sum()

In [358]:
# WB_full_df = WB_full_df.merge(wb_rank_feature_metrics, on='Баркод', how='left')

### Добавление остатков Екатеринбург 2 и Новосибирск 

In [359]:
WB_full_df = WB_full_df.merge(ekb2_nsk_stocks, on='Баркод', how='left')

In [360]:
WB_full_df = WB_full_df[list(WB_full_df.columns[:WB_full_df.columns.get_loc('Наценка_Невинномысск_монопалета')+1])\
                        + ['Остаток в Екатеринбург - Испытателей 14г', 
                           'Остаток в Екатеринбург - Перспективный 12', 
                           'Остаток в Новосибирске']\
                        + list(WB_full_df.columns[WB_full_df.columns.get_loc('Наценка_Невинномысск_монопалета')+1:-3])]

In [361]:
WB_full_df.loc[WB_full_df[WB_full_df['Артикул']=='Итого'].index[0], 
               'Остаток в Екатеринбург - Испытателей 14г'] = WB_full_df[['Остаток в Екатеринбург - Испытателей 14г']].iloc[:-1, :].sum().sum()

WB_full_df.loc[WB_full_df[WB_full_df['Артикул']=='Итого'].index[0], 
               'Остаток в Екатеринбург - Перспективный 12'] = WB_full_df[['Остаток в Екатеринбург - Перспективный 12']].iloc[:-1, :].sum().sum()


WB_full_df.loc[WB_full_df[WB_full_df['Артикул']=='Итого'].index[0], 
               'Остаток в Новосибирске'] = WB_full_df[['Остаток в Новосибирске']].iloc[:-1, :].sum().sum()

### В резерве

In [362]:
vr = pd.read_excel(path_vr)
vr['Артикул'] = vr['Артикул'].astype(str)

WB_full_df = WB_full_df.merge(vr, on='Артикул', how='left')

WB_full_df = WB_full_df[list(WB_full_df.columns[:WB_full_df.columns.get_loc('Остаток на РЦ')+1]) + ['В резерве'] + list(WB_full_df.columns[WB_full_df.columns.get_loc('Остаток на РЦ')+1:-1])]

#### Новый баркод, справвочно

In [363]:
# nbar = pd.read_excel(path_downloads + 'Реестр (1).xlsx')
# nbar = nbar.dropna()
# nbar = nbar.drop_duplicates().reset_index(drop=True)
# nbar['Баркод'] = nbar['Баркод'].astype(int)
# nbar['Новый баркод'] = nbar['Новый баркод'].astype(int)
# len(nbar) - len(list(set(nbar['Баркод'])))

In [364]:
WB_full_df = WB_full_df.merge(nbar, on='Баркод', how='left')

### Добавление аналитики по ценам за последние несколько дней

In [365]:
wbap = WB_price_metric[['Баркод']+dt_col(WB_price_metric)].copy()
# wbap[dt_col(wbap)] = wbap[dt_col(wbap)].astype(float)


def list_none(List):
    for i in range(len(List)):
        if List[i] == 0:
            List[i] = None
    return List
    
# Разница в цене предыдущего дня
Q = []
Q = list_none(wbap[dt_col(WB_price_metric)[-1]] - wbap[dt_col(WB_price_metric)[-2]])
    
# Стандартное отклонение цен за последние 7 дней
W = []
W = list_none([np.std(list(wbap.iloc[x, -7:])).round(2) for x in list(wbap.index)])

# Стандартное отклонение цен за последние 3 дня
E = []
E = list_none([np.std(list(wbap.iloc[x, -3:])).round(2) for x in list(wbap.index)])

# Отличие максимальной цены от минимальной за 3 дня 

R = []
R = list_none([wbap.iloc[x, -3:].max() - wbap.iloc[x, -3:].min() for x in list(wbap.index)])



wbap = wbap[['Баркод']]
wbap['Разница в цене предыдущего дня'] = Q
wbap['Стандартное отклонение цен за последние 7 дней'] = W
wbap['Стандартное отклонение цен за последние 3 дня'] = E
wbap['Отличие максимальной цены от минимальной за 3 дня'] = R


WB_full_df = WB_full_df.merge(wbap, on='Баркод', how='left')

In [366]:
rc_irc = list(WB_full_df[['Артикул']]\
         .merge(pd.read_excel(path_downloads + 'РЦ Иркутск.xlsx', 
                dtype={'Артикул': 'str'}).groupby('Артикул')['Остаток РЦ Иркутск'].sum()\
                .reset_index(drop=False), 
                on='Артикул', 
                how='left')['Остаток РЦ Иркутск'])

WB_full_df.insert(WB_full_df.columns.get_loc('Остаток на РЦ')+1, 'Остаток РЦ Иркутск', rc_irc)

## Создание таблиц по остаткам FBS WB (по складам) - метод: выгрузка по API

In [367]:
header_params = {
    'Authorization': const.WB_API_KEY,
    'Content-Type': 'application/json'
    }



url_param = f'https://suppliers-api.wildberries.ru/api/v3/warehouses'

response = requests.get(
    url_param,
    headers=header_params,
)

list_warehouseId = {}
df = pd.DataFrame(response.json())
for row in df['name'].unique():
    list_warehouseId[row] = df[df['name']==row]['id'].values[0]

list_bar = [x for x in list(set(pd.read_excel(path_WB_size,
                                usecols=['Баркод'],
                                dtype={'Баркод': 'str'})['Баркод'])) if str(x) != 'nan']
wb_fbs_stocks = None

Sklad_list = list(list_warehouseId.keys())

for row in Sklad_list:

    warehouseId = list_warehouseId[row]

    url_param = f'https://suppliers-api.wildberries.ru/api/v3/stocks/{warehouseId}'

    wb_fbs_stocks_sklad = None

    for butch in list_bar[:1000], list_bar[1000:]:

        response = requests.post(
            url_param,
            headers=header_params,
            json = {'skus': butch},
        )

        if butch == list_bar[:1000]:
            wb_fbs_stocks_sklad = pd.DataFrame(response.json()['stocks'])
#             print('wb_fbs_stocks_sklad_0')
        else:
            wb_fbs_stocks_sklad = pd.concat([wb_fbs_stocks_sklad, pd.DataFrame(response.json()['stocks'])])
#             print('wb_fbs_stocks_sklad_1')

    if len(wb_fbs_stocks_sklad) > 0:
        wb_fbs_stocks_sklad.columns = ['Баркод', row.replace(' ', '_')]
    else:
        None

#     display(wb_fbs_stocks_sklad)
#     print(wb_fbs_stocks_sklad[row].sum())

    if row == Sklad_list[0]:
        if len(wb_fbs_stocks_sklad) > 0:
            wb_fbs_stocks = wb_fbs_stocks_sklad.copy()
        else:
            Sklad_list = Sklad_list[1:]
    else:
        if len(wb_fbs_stocks_sklad) > 0:
            wb_fbs_stocks = wb_fbs_stocks.merge(wb_fbs_stocks_sklad, on='Баркод', how='outer')
        else:
            None
            
wb_fbs_stocks['Остатки FBS'] = [wb_fbs_stocks.iloc[x, 1:].fillna(0).sum() for x in list(wb_fbs_stocks.index)]

wb_fbs_stocks['Баркод'] = wb_fbs_stocks['Баркод'].astype(int)
if len(wb_fbs_stocks) - len(list(set(wb_fbs_stocks['Баркод']))) != 0:
    print('error')
    int('error')
else:
    None
if 'Остатки FBS' in WB_full_df.columns:
    WB_full_df = WB_full_df.drop('Остатки FBS', axis=1)
WB_full_df = WB_full_df.merge(wb_fbs_stocks, on='Баркод', how='left')
wb_fbs_stocks = None
WB_full_df = WB_full_df[list(WB_full_df.columns)[:list(WB_full_df.columns).index('Остаток')+1]\
+ ['Остатки FBS'] + list(WB_full_df.columns)[list(WB_full_df.columns).index('Остаток')+1:-1]]

In [368]:
wb_fbs_stocks

In [369]:
WB_full_df

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,Спринт озон,Новый баркод,Разница в цене предыдущего дня,Стандартное отклонение цен за последние 7 дней,Стандартное отклонение цен за последние 3 дня,Отличие максимальной цены от минимальной за 3 дня,Склад_Иркутск,Склад_Омск,ПСР,Склад_Поставщик_Химки
0,16000,"Гель для рук антисептический с витамином Е, 66...",11922227.0,4.627181e+12,SANITEX,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0
1,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567.0,7.432142e+12,SS&Y Group,отказ,0,0,0,0,...,NaN,NaN,NaN,126.19,NaN,NaN,0.0,NaN,NaN,0.0
2,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568.0,7.432142e+12,SS&Y Group,отказ,0,0,0,1,...,NaN,NaN,NaN,102.44,NaN,NaN,0.0,NaN,NaN,0.0
3,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569.0,7.432142e+12,SS&Y Group,отказ,2,0,0,1,...,NaN,NaN,NaN,106.89,NaN,NaN,0.0,NaN,NaN,0.0
4,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570.0,7.432142e+12,SS&Y Group,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,Z0401,Ваза для цветов и сухоцветов решетка,149328830.0,4.601664e+12,Limpide Side,текущий Дарья,6,7,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,37.0,NaN,0.0,0.0
1247,Z0500,Декантер с бокалами,150246240.0,4.602723e+12,Limpide Side,текущий Дарья,5,5,3,0,...,ТЗ 30 (16.10 - 25.10),NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0
1248,Z0501,Графин для воды,158358449.0,4.630113e+12,Limpide Side,текущий Дарья,10,2,1,3,...,ТЗ 34 (21.11-01.12),NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0
1249,c20502,"Мышечный перкуссионный массажер, ударный писто...",11416406.0,8.112390e+12,SS&Y Group,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [370]:
WB_full_df.iloc[:, -4:].sum().sum()

32257.0

In [371]:
WB_full_df['Остатки FBS'].sum()

32257.0

### Добавление "статус рекламы"

In [372]:
WB_full_df = WB_full_df.merge(st_r, on='Артикул', how='left')

In [373]:
WB_full_df['статус рекламы'].unique()

array([nan, 'активна', 'приостановлена'], dtype=object)

### Проверка метрик по "объемам"

In [374]:
WB_size_metrics

,Артикул,Наименование,Номенклатура,Баркод,Предмет,Бренд,30-November-2022,01-December-2022,02-December-2022,05-December-2022,...,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024,Diff
0,27005,Бесконтактный термометр,12818125,7432141550840,Термометры электронные,36_6,0.89,0.89,0.89,0.89,...,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.0
1,B0001,Набор мезoроллер + масло,39973778,7432141559167,Косметические наборы для ухода,BearsBeard,1.04,1.04,1.04,1.04,...,1.10,1.10,1.10,1.10,1.10,1.10,1.10,1.10,1.10,0.0
2,50210,"Контейнер для лекарств, аптечка",128685705,7432141950561,Аптечки,Birdhouse,23.09,23.09,23.09,21.28,...,14.78,14.78,14.78,14.78,14.78,14.78,14.78,14.78,14.78,0.0
3,J0003,"Бордюрная лента для ванны 3,3м",128450645,7432141950483,Бордюры для ванны,Birdhouse,0.40,0.40,0.40,0.40,...,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.40,0.0
4,29706,"Вакуумный пакет для одежды 70х100 см, 3 шт",17580917,7432141550912,Вакуумные пакеты,Birdhouse,2.76,2.76,2.76,2.76,...,2.70,2.70,2.70,2.70,2.70,2.70,2.70,2.70,2.70,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,A8304,Набор для специй 6 баночек,216744369,4674444646573,Наборы для специй,Shiny Kitchen,NaN,NaN,NaN,NaN,...,1.54,1.54,1.54,1.54,1.54,1.54,1.54,1.54,1.54,0.0
1409,A1301-выбеленный,NaN,220961206,2039754400789,Диспенсеры кухонные,Shiny Kitchen,NaN,NaN,NaN,NaN,...,NaN,NaN,1.95,1.95,1.95,1.95,1.95,1.95,1.95,0.0
1410,G0637,NaN,222264458,4674549145483,Прожекторы,Ocean of Light,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.38,2.38,0.0
1411,G0636N,NaN,222296446,4674912969500,Прожекторы,Ocean of Light,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.38,2.38,0.0


In [375]:
WB_size_metrics = WB_size_metrics.drop_duplicates().reset_index(drop=True)

In [376]:
# for i in range(len(WB_size_metrics)):
#     art = WB_size_metrics.loc[i]['Артикул']
#     if len(WB_size_metrics[WB_size_metrics['Артикул']==art]) > 1:
#         display(WB_size_metrics[WB_size_metrics['Артикул']==art])

In [377]:
dfc = 'WB_price_metric'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 23


In [378]:
dfc = 'stocks_WB_metric'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [379]:
dfc = 'WB_full_df'

print(f"Дублей в стобце Баркод: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Баркод'])))}")
print(f"Дублей в стобце Артикул: {len(locals()[dfc]) - len(list(set(locals()[dfc]['Артикул'])))}")

Дублей в стобце Баркод: 0
Дублей в стобце Артикул: 16


In [380]:
WB_full_features_metric['Остаток'].isna().sum()

0

In [381]:
WB_full_df['Остаток'].isna().sum()

0

In [382]:
WB_full_df.columns

Index(['Артикул', 'Наименование', 'Номенклатура', 'Баркод', 'ТМ', 'Статус',
       '20-February-2024', '21-February-2024', '22-February-2024',
       '23-February-2024', '24-February-2024', '25-February-2024',
       '26-February-2024', '27-February-2024', '28-February-2024',
       '29-February-2024', '01-March-2024', '02-March-2024', '03-March-2024',
       '26-February-03-March', '04-March-2024', '05-March-2024',
       '06-March-2024', '07-March-2024', '08-March-2024', '09-March-2024',
       '10-March-2024', '04-March-10-March', '11-March-2024', '12-March-2024',
       '13-March-2024', '14-March-2024', '15-March-2024', '16-March-2024',
       '17-March-2024', '11-March-17-March', '18-March-2024', '19-March-2024',
       '20-March-2024', '21-March-2024', '22-March-2024', '23-March-2024',
       '24-March-2024', '18-March-24-March', '25-March-2024', '26-March-2024',
       '27-March-2024', '28-March-2024', '29-March-2024', '30-March-2024',
       '31-March-2024', '25-March-31-March'

In [383]:
WB_full_features_metric

,Артикул,Наименование,Номенклатура,Баркод,ТМ,20-February-2024,21-February-2024,22-February-2024,23-February-2024,24-February-2024,...,Цена до скидки,Скидка,Цена,Комиссия в рублях,НДС,Налог на прибыль,Налог,Прибыль,Наценка,Процент налога
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,0,0,0,0,0,...,5940.0,90.0,594,154.44,23.641667,22.750667,46.392333,91.002667,37.34,7.810157
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,0,0,0,1,1,...,2300.0,79.0,483,125.58,9.951667,9.227167,19.178833,36.908667,15.14,3.970773
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,2,0,0,1,0,...,3144.0,84.0,503,130.78,12.418333,11.663833,24.082167,46.655333,19.14,4.787707
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,0,0,0,0,0,...,4236.0,81.0,804,209.04,49.541667,48.335667,97.877333,193.342667,79.33,12.173798
4,18301,"Чехол для airpods 2 с карабином, белый",7490270,7432141649603,SS&Y Group,1,3,1,0,2,...,1150.0,90.0,115,29.90,-3.220000,-3.392500,-6.612500,-13.570000,-26.91,-5.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,390.0,0.0,390,101.40,26.636667,26.051667,52.688333,104.206667,139.35,13.509829
1246,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,646.0,48.0,335,73.70,22.363333,21.860833,44.224167,87.443333,119.59,13.201244
1247,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,1590.0,0.0,1590,318.00,104.336667,101.951667,206.288333,407.806667,104.46,12.974109
1248,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,2627.0,25.0,1970,413.70,101.798333,98.843333,200.641667,395.373333,44.98,10.184856


In [384]:
# dop_ost = WB_ofbs_metrics[['Баркод', WB_ofbs_metrics.columns[-1]]].copy()
# dop_ost['Баркод'] = dop_ost['Баркод'].astype(int)
# dop_ost[dop_ost.columns[-1]] = dop_ost[dop_ost.columns[-1]].astype(int)
# dop_ost.columns = ['Баркод', 'Доп. ост']
# WB_full_df = WB_full_df.merge(dop_ost, on='Баркод', how='left')
# WB_full_df['Остаток'].loc[len(WB_full_df)-1] = 0
# WB_full_df['Остаток'] = WB_full_df['Остаток'] + WB_full_df['Доп. ост']
# WB_full_df = WB_full_df.drop('Доп. ост', axis=1)
# WB_full_df['Остаток'].loc[len(WB_full_df)-1] = sum(list(WB_full_df['Остаток'])[:-1])

# WB_full_features_metric = WB_full_features_metric.merge(dop_ost, on='Баркод', how='left')
# WB_full_features_metric['Остаток'] = WB_full_features_metric['Остаток'] + WB_full_features_metric['Доп. ост']
# WB_full_features_metric = WB_full_features_metric.drop('Доп. ост', axis=1)

In [385]:
# WB_full_df['Остоток_доп'] = WB_full_df['Остатки FBS']
# WB_full_df['Остоток_доп'] = WB_full_df['Остоток_доп'].fillna(0)
# WB_full_df['Остаток'].loc[len(WB_full_df)-1] = 0
# WB_full_df['Остаток'] = WB_full_df['Остаток'] + WB_full_df['Остоток_доп']
# WB_full_df = WB_full_df.drop('Остоток_доп', axis=1)
# WB_full_df['Остаток'].loc[len(WB_full_df)-1] = sum(list(WB_full_df['Остаток']))
# WB_ost_dop = WB_full_df[['Баркод', 'Остаток']].iloc[:-1, :].copy()
# WB_ost_dop.columns = ['Баркод', 'Остаток_new']
# WB_full_features_metric = WB_full_features_metric.merge(WB_ost_dop, on='Баркод', how='left')
# WB_full_features_metric['Остаток'] = WB_full_features_metric['Остаток_new']
# WB_full_features_metric = WB_full_features_metric.drop('Остаток_new', axis=1)

In [386]:
for row in [WB_orders_metrics, 
            WB_price_metric, 
            stocks_WB_metric, 
            WB_full_features_metric, 
            WB_full_df, 
            WB_size_metrics]:
    if len(row) - len(list(set(row['Баркод']))) !=0:

        display(row)
    

## Создание таблицы с изменением цен

In [387]:
if len(WB_price_metric) - len(list(set(WB_price_metric['Баркод']))) !=0:
    print('error')
    int('error')
else:
    WB_price_difference = WB_price_metric[['Баркод']+dt_col(WB_price_metric)[-13:]].copy()
    cols_for_analysis = list(WB_price_difference.columns)[1:][::-1]
    for i in range(len(cols_for_analysis)-1):
        WB_price_difference['Изм. цены ' + pd.to_datetime(cols_for_analysis[i])\
                            .strftime('%d.%m')] = WB_price_difference[cols_for_analysis[i]]\
                                                    - WB_price_difference[cols_for_analysis[i+1]]
#         WB_price_difference['Цена '+ pd.to_datetime(cols_for_analysis[i])\
#                             .strftime('%d.%m')] = WB_price_difference[cols_for_analysis[i]]
    WB_price_difference = WB_price_difference.drop(cols_for_analysis, axis=1)
    WB_price_difference = WB_price_difference[['Баркод']\
                                             + list(WB_price_difference.columns[1:])[::-1]]
    C_price = pd.read_excel(path_downloads + 'Целевая цена.xlsx',
                            sheet_name='WB', 
                            dtype={'Баркод': 'int'})
    WB_price_difference = WB_price_difference.merge(C_price, on='Баркод', how='left')
    if len(WB_price_difference) - len(list(set(WB_price_difference['Баркод']))) !=0:
        exit()
    WB_price_difference.to_excel(path_downloads + 'Изменения цены вб.xlsx', index=False)

## Выгрузка

In [388]:
WB_price_metric

,Артикул,Наименование,Номенклатура,Баркод,ТМ,30-August-2022,31-August-2022,01-September-2022,02-September-2022,05-September-2022,...,27-March-2024,28-March-2024,29-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,08-April-2024,09-April-2024
0,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567,7432141949918,SS&Y Group,813,813,813,864,864,...,849,849,849,849,849,849,849,594,594,594
1,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568,7432141949925,SS&Y Group,862,862,862,862,862,...,690,690,690,690,690,690,690,483,483,483
2,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569,7432141949932,SS&Y Group,882,882,882,882,882,...,719,719,719,719,719,719,719,503,503,503
3,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570,7432141550639,SS&Y Group,889,889,889,889,889,...,804,804,804,804,804,804,804,804,804,804
4,18102,Защитное полноэкранное 3D стекло для для iPhon...,8461127,7432141550590,SS&Y Group,132,132,132,132,132,...,132,132,132,132,132,132,132,132,132,132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,50457,"Расческа с зеркалом матовая, зеленая",198454401,4673613441636,Revolut,0,0,0,0,0,...,0,0,0,390,390,390,390,390,390,390
1247,J0007,Зеркальная мозаика,172572285,4664807383498,Birdhouse,0,0,0,0,0,...,0,0,0,650,650,650,650,335,335,335
1248,G0950,Подушка подголовник в машину,170627282,4659437081876,Flex Drive,0,0,0,0,0,...,0,0,0,0,0,1590,1590,1590,1590,1590
1249,G0636,Фонарь-прожектор,165763226,4674582725086,Ocean of Light,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1970,1970


In [389]:
list(set(WB_price_metric[dt_col(WB_price_metric)[-1]] - WB_price_metric[dt_col(WB_price_metric)[-2]]))

[0, 33, -64, 36, 10, 11, 12, 13, 14, 115, 23, 24, 57, 27, 25]

## прибыль с единицы товара

In [390]:
WB_full_df = WB_full_df.merge(WB_full_features_metric[['Баркод', 
                                                       'Прибыль']]\
                              .rename(columns={'Прибыль': 'прибыль с единицы товара'}),
                              on='Баркод', 
                              how='left') 
WB_full_df['прибыль с единицы товара'] = WB_full_df['прибыль с единицы товара'].round(2)

## Цена до скидки

In [391]:
dop_price_2 = dop_price_2.drop_duplicates().reset_index(drop=True)
if len(dop_price_2) - len(list(set(dop_price_2['Артикул']))) != 0:
    exit()

In [392]:
WB_full_df.insert(list(WB_full_df.columns).index('Цена')+1, 
                  'Цена до скидки', 
                  list(dop_price_2\
                       .merge(WB_full_df[['Артикул']], 
                              on='Артикул', 
                              how='right')['Цена до скидки']))

In [393]:
WB_full_df

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,Разница в цене предыдущего дня,Стандартное отклонение цен за последние 7 дней,Стандартное отклонение цен за последние 3 дня,Отличие максимальной цены от минимальной за 3 дня,Склад_Иркутск,Склад_Омск,ПСР,Склад_Поставщик_Химки,статус рекламы,прибыль с единицы товара
0,16000,"Гель для рук антисептический с витамином Е, 66...",11922227.0,4.627181e+12,SANITEX,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,-11.15
1,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,8455567.0,7.432142e+12,SS&Y Group,отказ,0,0,0,0,...,NaN,126.19,NaN,NaN,0.0,NaN,NaN,0.0,NaN,91.00
2,16802,Чехол книжка-подставка Smart Case для iPad 9.7...,8455568.0,7.432142e+12,SS&Y Group,отказ,0,0,0,1,...,NaN,102.44,NaN,NaN,0.0,NaN,NaN,0.0,NaN,36.91
3,16803,Чехол книжка-подставка Smart Case для iPad 9.7...,8455569.0,7.432142e+12,SS&Y Group,отказ,2,0,0,1,...,NaN,106.89,NaN,NaN,0.0,NaN,NaN,0.0,NaN,46.66
4,16804,Чехол книжка-подставка Smart Case для iPad 9.7...,8455570.0,7.432142e+12,SS&Y Group,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,193.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,Z0401,Ваза для цветов и сухоцветов решетка,149328830.0,4.601664e+12,Limpide Side,текущий Дарья,6,7,4,4,...,NaN,NaN,NaN,NaN,37.0,NaN,0.0,0.0,NaN,390.87
1247,Z0500,Декантер с бокалами,150246240.0,4.602723e+12,Limpide Side,текущий Дарья,5,5,3,0,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,762.97
1248,Z0501,Графин для воды,158358449.0,4.630113e+12,Limpide Side,текущий Дарья,10,2,1,3,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,505.24
1249,c20502,"Мышечный перкуссионный массажер, ударный писто...",11416406.0,8.112390e+12,SS&Y Group,архив,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00


In [394]:
# ## Цена до скидки

# dop_price = pd.read_excel(path_price,
#                           usecols=['Артикул продавца', 'Текущая розн. цена (до скидки)'],
#                           dtype={'Артикул продавца': 'str',
#                                  'Текущая розн. цена (до скидки)': 'int'})
# dop_price.columns = ['Артикул', 'Цена до скидки']
# dop_price['Артикул'] = [x.replace('/', '') for x in list(dop_price['Артикул'])]
# dop_price = dop_price.groupby('Артикул').mean().reset_index(drop=False)
# WB_full_df.insert(list(WB_full_df.columns).index('Цена')+1, 
#                   'Цена до скидки', 
#                   list(WB_full_df[['Артикул']]\
#                        .merge(dop_price, 
#                               on='Артикул', 
#                               how='left')['Цена до скидки']))

### Доп признак от Славы

In [395]:
dop_columns_for_Slava = pd.read_excel(path_downloads + 'Поставки ВБ и Озон.xlsx', 
                                      sheet_name='ВБ',
                                      dtype={'Артикул': 'str'})
WB_full_df = WB_full_df.merge(dop_columns_for_Slava, on='Артикул', how='left')

In [396]:
# WB_full_df = WB_full_df.merge(pd.read_excel(path_downloads + 'Склады ВБ.xlsx',
#                                usecols=['supplierArticle', 'Количество складов'], 
#                                dtype={'supplierArticle': 'str'})\
#                  .rename(columns={'supplierArticle': 'Артикул'})\
#                  .drop_duplicates().reset_index(drop=True),
#                  on='Артикул', 
#                  how='left')

In [397]:
list(set(WB_full_df[WB_full_df['Цена']==0]['Артикул']))

['20100', 'c20502', '50241', '27002']

In [398]:
WB_full_df[WB_full_df['Артикул'].isin(list(set(WB_full_df[WB_full_df['Цена']==0]['Артикул'])))][['Артикул', 'Баркод', 'Цена']]

,Артикул,Баркод,Цена
114,20100,8.112390e+12,0.0
232,27002,8.112389e+12,0.0
423,50241,7.432142e+12,0.0
1249,c20502,8.112390e+12,0.0


In [399]:
WB_full_df[WB_full_df['Артикул']=='G0069']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,Статус,20-February-2024,21-February-2024,22-February-2024,23-February-2024,...,ПСР,Склад_Поставщик_Химки,статус рекламы,прибыль с единицы товара,Коледино 10.11,Электросталь 3.11,Екатеринбург 20.11,Екат доп,Краснодар 13.11,Шушары 8.11
873,G0069,Застежка для уменьшения брюк латунь,165118565.0,4.645471e+12,JewelryMeverly,текущий Дарья,1,0,0,1,...,NaN,9.0,NaN,66.64,NaN,NaN,NaN,NaN,100.0,NaN
874,G0069,Застежка для уменьшения брюк латунь,165118565.0,4.644647e+12,JewelryMeverly,текущий Дарья,0,0,0,0,...,NaN,NaN,NaN,66.64,NaN,NaN,NaN,NaN,100.0,NaN


In [400]:
index1 = WB_full_df[(WB_full_df['Артикул']=='G0069')&(WB_full_df['Баркод']==4645471315596)].index[0]
index2 = WB_full_df[(WB_full_df['Артикул']=='G0069')&(WB_full_df['Баркод']==4644647421062)].index[0]
for row in dt_col(WB_full_df):
    WB_full_df.loc[index1, row] = WB_full_df.loc[index1][row] + WB_full_df.loc[index2][row]
WB_full_df = WB_full_df.drop(index=index2).reset_index(drop=True)

In [401]:
index1 = WB_full_df[(WB_full_df['Артикул']=='G0636')&(WB_full_df['Баркод']==4674582725086)].index[0]
index2 = WB_full_df[(WB_full_df['Артикул']=='G0636')&(WB_full_df['Баркод']==4647277236624)].index[0]
for row in dt_col(WB_full_df):
    WB_full_df.loc[index1, row] = WB_full_df.loc[index1][row] + WB_full_df.loc[index2][row]
WB_full_df = WB_full_df.drop(index=index2).reset_index(drop=True)

In [402]:
# def xxx(data):
    
#     if len(data[(data['Артикул']=='G0283') & (data['Баркод']==4648957260304)]) != 0:
        
#         index1 = data[(data['Артикул']=='G0283') & (data['Баркод']==4648957260304)].index[0]
        
#         data['Артикул'].loc[index1] = 'G0284' 
        
        
        
        
#     if len(data[(data['Артикул']=='G0283') & (data['Баркод']==4648877486747)]) != 0:
        
#         index1 = data[(data['Артикул']=='G0283') & (data['Баркод']==4648877486747)].index[0]
        
#         data['Артикул'].loc[index1] = 'G0285'  
        
#     return data

In [403]:
# WB_orders_metrics = xxx(WB_orders_metrics)
# WB_price_metric = xxx(WB_price_metric)
# stocks_WB_metric = xxx(stocks_WB_metric)
# WB_size_metrics = xxx(WB_size_metrics)
# WB_full_features_metric = xxx(WB_full_features_metric)
# WB_full_df = xxx(WB_full_df)

In [404]:
WB_orders_metrics = wwm.rename_data_from_reestr(WB_orders_metrics, 'Наименование', reestr=reestr)
WB_price_metric = wwm.rename_data_from_reestr(WB_price_metric, 'Наименование', reestr=reestr)
stocks_WB_metric = wwm.rename_data_from_reestr(stocks_WB_metric, 'Наименование', reestr=reestr)
WB_size_metrics = wwm.rename_data_from_reestr(WB_size_metrics, 'Наименование', reestr=reestr)
WB_full_features_metric = wwm.rename_data_from_reestr(WB_full_features_metric, 'Наименование', reestr=reestr)
WB_full_df = wwm.rename_data_from_reestr(WB_full_df, 'Наименование', reestr=reestr)

In [405]:
WB_orders_metrics.to_csv(path_WB_orders_metrics, index=False, sep=";", encoding='utf-8-sig')

In [406]:
WB_price_metric.to_csv(path_WB_price_metric, index=False, sep=";", encoding='utf-8-sig')

In [407]:
stocks_WB_metric.to_csv(path_stocks_WB_metric, index=False, sep=";", encoding='utf-8-sig')

In [408]:
WB_depersonalized_metric.to_csv(path_WB_depersonalized_metric, index=False, sep=";", encoding='utf-8-sig')

In [409]:
WB_full_features_metric.to_csv(path_WB_full_features_metric, index=False, sep=";", encoding='utf-8-sig')

In [410]:
WB_full_df.to_csv(path_WB_full_df, index=False, sep=";", encoding='utf-8-sig')

In [411]:
WB_size_metrics.to_excel(path_WB_size_metrics, index=False)

In [412]:
if f == '0':
    WB_ofbs_metrics.to_csv(path_WB_ofbs_metrics, index=False, sep=";", encoding='utf-8-sig')

In [413]:
f

'2'

### Редактирование TM

In [414]:
WB_orders_metrics = add_and_redact_brend(WB_orders_metrics)
# WB_depersonalized_metric = add_and_redact_brend(WB_depersonalized_metric)
WB_size_metrics = add_and_redact_brend(WB_size_metrics)
WB_price_metric = add_and_redact_brend(WB_price_metric)
stocks_WB_metric = add_and_redact_brend(stocks_WB_metric)
WB_full_features_metric = add_and_redact_brend(WB_full_features_metric)
WB_full_df = add_and_redact_brend(WB_full_df)

In [415]:
WB_orders_metrics[WB_orders_metrics['Артикул']=='286012']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,23-February-2020,24-February-2020,25-February-2020,26-February-2020,27-February-2020,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024


In [416]:
WB_orders_metrics[WB_orders_metrics['Артикул']=='28612']

,Артикул,Наименование,Номенклатура,Баркод,ТМ,23-February-2020,24-February-2020,25-February-2020,26-February-2020,27-February-2020,...,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024,05-April-2024,06-April-2024,07-April-2024,08-April-2024
792,28612,"Валик маленький массажный, бирюзовый",121131578,7432141550814,Shark Fit,0,0,0,0,0,...,7,6,3,3,2,3,2,4,6,3


In [417]:
for i in range(len(WB_price_metric)):
    bar = WB_price_metric.loc[i]['Баркод']
    if bar in [int(x) for x in list(set(WB_full_df['Баркод'])) if str(x)!='nan']:
        WB_price_metric.loc[i, 'Артикул'] = WB_full_df[WB_full_df['Баркод']==bar]['Артикул'].values[0]

In [418]:
# data_name = 'stocks_WB_metric'
# data = stocks_WB_metric

# Q=[]
# for i in range(len(data)):
#     art = data.loc[i]['Артикул']
#     if len(data[data['Артикул']==art]) > 1:
#         Q.append(i)
# data.query('index in @Q').reset_index(drop=True).to_excel(path_downloads + data_name + '_дубли.xlsx', index=False)

## Сохранение наценки

In [419]:
wb_markup_metrics = pd.read_excel(wwm.path_wb_markup_metrics,
                                  dtype={'Артикул': 'str',
                                         'Баркод': 'int'})
if len(wb_markup_metrics) - len(list(set(wb_markup_metrics['Баркод']))) != 0:
    print('Неуникальные баркоды в wb_markup_metrics')
    exit()
wb_markup_new = WB_full_features_metric[['Артикул', 'Баркод', 'Наценка']]\
                .rename(columns={'Наценка': const.dt_now.strftime('%d-%B-%Y')}).copy()
if len(wb_markup_new) - len(list(set(wb_markup_new['Баркод']))) != 0:
    print('Неуникальные баркоды в wb_markup_new')
    exit()
col = wwm.dt_col(wb_markup_new)[0]
for i in range(len(wb_markup_new)):
    bar = wb_markup_new.loc[i]['Баркод']
    if bar in list(set(WB_price_metric['Баркод'])):
        if WB_price_metric[WB_price_metric['Баркод']==bar][col].values[0] == 0:
            wb_markup_new.loc[i, col] = None
if col in wb_markup_metrics.columns:
    wb_markup_metrics = wb_markup_metrics.drop(col, axis=1)
    wb_markup_metrics = wb_markup_metrics.merge(wb_markup_new, on='Баркод', how='outer')
    wb_markup_metrics['Артикул_x'] = wb_markup_metrics['Артикул_x'].fillna(wb_markup_metrics['Артикул_y'])
    wb_markup_metrics = wb_markup_metrics.drop('Артикул_y', axis=1).rename(columns={'Артикул_x': 'Артикул'})
else:
    wb_markup_metrics = wb_markup_metrics.merge(wb_markup_new, on='Баркод', how='outer')
    wb_markup_metrics['Артикул_x'] = wb_markup_metrics['Артикул_x'].fillna(wb_markup_metrics['Артикул_y'])
    wb_markup_metrics = wb_markup_metrics.drop('Артикул_y', axis=1).rename(columns={'Артикул_x': 'Артикул'})
if len(wb_markup_metrics) - len(list(set(wb_markup_metrics['Баркод']))) != 0:
    print('Неуникальные баркоды в wb_markup_metrics')
    exit()
wb_markup_metrics.to_excel(wwm.path_wb_markup_metrics, index=False)

In [420]:
len(WB_full_df) - len(list(set(WB_full_df['Баркод'])))

0

In [421]:
# for i in range(len(WB_full_df)):
#     art = WB_full_df.loc[i]['Артикул']
#     if art in list(pr_n.keys()):
#         WB_full_df.loc[i, 'Цена'] = pr_n[art]
#     else:
#         WB_full_df.loc[i, 'Цена'] = None
        
        
# data = wb_markups.get_param_data()

# WB_orders_metrics = WB_orders_WB_orders_metrics = WB_orders_metrics[WB_orders_metrics['Артикул'].isin(list(set(data['Артикул'])))].reset_index(drop=True)metrics[WB_orders_metrics['Артикул'].isin(list(set(data['Артикул'])))].reset_index(drop=True)

# pr_n = wb_markups.return_price(list(set(WB_orders_metrics['Артикул'])), default_nc=80)

# for i in range(len(WB_orders_metrics)):
#     WB_orders_metrics.loc[i, 'Цена'] = pr_n[WB_orders_metrics.loc[i]['Артикул']]

## Выгрузка всех метрик в одном файле

In [422]:
def unloading():
    
    with pd.ExcelWriter(path_algoritm + "Все файлы для ежедневной презентации/full_WB_df.xlsx") as writer:




        WB_orders_metrics.to_excel(writer, sheet_name="WB_orders_metrics", index=False)
        for column in WB_orders_metrics:
            column_width = max(WB_orders_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = WB_orders_metrics.columns.get_loc(column)
            writer.sheets['WB_orders_metrics'].set_column(col_idx, col_idx, column_width)



        WB_depersonalized_metric.to_excel(writer, sheet_name="WB_depersonalized_metric", index=False)
        for column in WB_depersonalized_metric:
            column_width = max(WB_depersonalized_metric[column].astype(str).map(len).max()+2, len(column))
            col_idx = WB_depersonalized_metric.columns.get_loc(column)
            writer.sheets['WB_depersonalized_metric'].set_column(col_idx, col_idx, column_width)



        WB_ofbs_metrics.to_excel(writer, sheet_name="WB_ofbs_metrics", index=False)
        for column in WB_ofbs_metrics:
            column_width = max(WB_ofbs_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = WB_ofbs_metrics.columns.get_loc(column)
            writer.sheets['WB_ofbs_metrics'].set_column(col_idx, col_idx, column_width)


        WB_size_metrics.to_excel(writer, sheet_name="WB_size_metrics", index=False)
        for column in WB_size_metrics:
            column_width = max(WB_size_metrics[column].astype(str).map(len).max(), len(column))
            col_idx = WB_size_metrics.columns.get_loc(column)
            writer.sheets['WB_size_metrics'].set_column(col_idx, col_idx, column_width)




        WB_price_metric.to_excel(writer, sheet_name="WB_price_metric", index=False)
        for column in WB_price_metric:
            column_width = max(WB_price_metric[column].astype(str).map(len).max(), len(column))
            col_idx = WB_price_metric.columns.get_loc(column)
            writer.sheets['WB_price_metric'].set_column(col_idx, col_idx, column_width)



        stocks_WB_metric.to_excel(writer, sheet_name="stocks_WB_metric", index=False)
        for column in stocks_WB_metric:
            column_width = max(stocks_WB_metric[column].astype(str).map(len).max(), len(column))
            col_idx = stocks_WB_metric.columns.get_loc(column)
            writer.sheets['stocks_WB_metric'].set_column(col_idx, col_idx, column_width)


        WB_full_features_metric.to_excel(writer, sheet_name="WB_full_features_metric", index=False)
        for column in WB_full_features_metric:
            column_width = max(WB_full_features_metric[column].astype(str).map(len).max() + 3, len(column))
            col_idx = WB_full_features_metric.columns.get_loc(column)
            writer.sheets['WB_full_features_metric'].set_column(col_idx, col_idx, column_width)



        WB_full_df.to_excel(writer, sheet_name="WB_full_df", index=False)
        for column in WB_full_df:
            column_width = max(WB_full_df[column].astype(str).map(len).max(), len(column))
            col_idx = WB_full_df.columns.get_loc(column)
            writer.sheets['WB_full_df'].set_column(col_idx, col_idx, column_width)



    writer.save()  

In [423]:
try:
    unloading()
except:
    unloading()

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [424]:
# ddf = WB_full_features_metric[['Артикул', 'Наименование', 'Номенклатура', 'Баркод', 'ТМ', 'Остаток', 'Стоимость хранения']].copy()
# ddf['Стоимость храниения всего кол-ва'] = ddf['Стоимость хранения'] * ddf['Остаток']

In [425]:
# ddf.loc[len(ddf)] = ['Всего'] + [None]*4 + [ddf['Остаток'].sum()] + [None] + [ddf['Стоимость храниения всего кол-ва'].sum()]

In [426]:
# ddf.to_excel(path_downloads + 'Стоимость хранения.xlsx', index=False)

## Обновление файла full_WB_df на корпоративном гугл диске 

In [427]:
# Подключение к гугл диску, аутентефикация

gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)


# Находим "id" папки "Дима"

id_folder_Dima = [x['id'] for x in drive\
                 .ListFile({'q': "'root' in parents and trashed=false"}).GetList() if x['title'] == "Дима"]
if len(id_folder_Dima) != 1:
    print('error')
    int('error')
else:
    id_folder_Dima = id_folder_Dima[0]

    
# Находим "id" файла "full_WB_df" в папке "Дима"

id_full_WB_df = [x['id'] for x in drive.ListFile({'q': f"'{id_folder_Dima}' in parents and trashed=false"})\
.GetList() if 'full_WB_df' in x['title']]

if len(id_full_WB_df) != 1:
    print('error')
    int('error')
else:
    id_full_WB_df = id_full_WB_df[0]

    
# Путь к файлу "full_WB_df" на локальном компьютере

path_file = path_algoritm + 'Все файлы для ежедневной презентации/full_WB_df.xlsx'


# Обновление файла "full_WB_df" на корпоративном гугл диске 

file = drive.CreateFile({'id': id_full_WB_df})
file.SetContentFile(path_file)
file.Upload()






# Находим "id" папки "Дима"

id_folder_Dima = [x['id'] for x in drive\
                 .ListFile({'q': "'root' in parents and trashed=false"}).GetList() if x['title'] == "Дима"]
if len(id_folder_Dima) != 1:
    print('error')
    int('error')
else:
    id_folder_Dima = id_folder_Dima[0]

    
# Находим "id" файла "РЦ_обработанные" в папке "Дима"

id_rc = [x['id'] for x in drive.ListFile({'q': f"'{id_folder_Dima}' in parents and trashed=false"})\
.GetList() if 'РЦ_обработанные' in x['title']]

if len(id_rc) != 1:
    print('error')
    int('error')
else:
    id_rc = id_rc[0]

    
# Путь к файлу "РЦ_обработанные" на локальном компьютере

path_file = path_algoritm + "ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/РЦ/РЦ_обработанные.xlsx"


# Обновление файла "РЦ_обработанные" на корпоративном гугл диске 

file = drive.CreateFile({'id': id_rc})
file.SetContentFile(path_file)
file.Upload()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=141351806184-lcpctltvgumjg7nedaiblgr9vtjpk98v.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [428]:
with pd.ExcelWriter(path_algoritm + "Все файлы для ежедневной презентации/Расчет наценки ВБ для разных складов.xlsx") as writer:
    
    

    
    dfl.to_excel(writer, sheet_name="Наценки_вб", index=False)
    for column in dfl:
        column_width = max(dfl[column].astype(str).map(len).max()+5, len(column))
        col_idx = dfl.columns.get_loc(column)
        writer.sheets['Наценки_вб'].set_column(col_idx, col_idx, column_width)
        
        
    
writer.save()

In [429]:
WB_full_df.to_excel(path_algoritm + 'WB_ORDERS/WB_ORDERS_NEW_ALGORITM/Метрики для презентации /WB_full_df.xlsx', index=False)

In [430]:
shutka_minutka = pyjokes.get_joke()
print(shutka_minutka)


There are 2 types of people: those who can extrapolate from incomplete data sets...


In [431]:
list(WB_full_df.columns)

['Артикул',
 'Наименование',
 'Номенклатура',
 'Баркод',
 'ТМ',
 'Статус',
 '20-February-2024',
 '21-February-2024',
 '22-February-2024',
 '23-February-2024',
 '24-February-2024',
 '25-February-2024',
 '26-February-2024',
 '27-February-2024',
 '28-February-2024',
 '29-February-2024',
 '01-March-2024',
 '02-March-2024',
 '03-March-2024',
 '26-February-03-March',
 '04-March-2024',
 '05-March-2024',
 '06-March-2024',
 '07-March-2024',
 '08-March-2024',
 '09-March-2024',
 '10-March-2024',
 '04-March-10-March',
 '11-March-2024',
 '12-March-2024',
 '13-March-2024',
 '14-March-2024',
 '15-March-2024',
 '16-March-2024',
 '17-March-2024',
 '11-March-17-March',
 '18-March-2024',
 '19-March-2024',
 '20-March-2024',
 '21-March-2024',
 '22-March-2024',
 '23-March-2024',
 '24-March-2024',
 '18-March-24-March',
 '25-March-2024',
 '26-March-2024',
 '27-March-2024',
 '28-March-2024',
 '29-March-2024',
 '30-March-2024',
 '31-March-2024',
 '25-March-31-March',
 '01-April-2024',
 '02-April-2024',
 '03-Apr

In [432]:
exit()

SystemExit: 

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
